<a href="https://colab.research.google.com/github/Cobritra/NLPpublication/blob/main/TeamNoteGen2_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#@title Imports (Ignore Error Messages)
#!pip install collections==3.9.16
#import medspacy and libraries
import warnings
warnings.filterwarnings('ignore')
!pip install medspacy==0.2.0.1 -q -q -q
warnings.filterwarnings('ignore')
!pip install https://s3-us-west-2.amazonaws.com/ai2-s2-scispacy/releases/v0.5.1/en_ner_bc5cdr_md-0.5.1.tar.gz -q -q -q
warnings.filterwarnings('ignore')
!pip uninstall xgboost -y -q -q -q
warnings.filterwarnings('ignore')
!pip install xgboost==0.82 -q -q -q
warnings.filterwarnings('ignore')
!pip install dill==0.3.3 -q -q -q
warnings.filterwarnings('ignore')
!pip install scikit-learn==1.0.2 -q -q -q

import statistics
from medspacy.sentence_splitting import PyRuSHSentencizer
import medspacy
import re
from medspacy.preprocess import PreprocessingRule, Preprocessor
from medspacy.ner import TargetRule
from medspacy.context import ConTextRule
from medspacy.section_detection import Sectionizer
from medspacy.postprocess import PostprocessingRule, PostprocessingPattern, Postprocessor
from medspacy.postprocess import postprocessing_functions
from medspacy.visualization import visualize_ent, visualize_dep
from medspacy.ner import TargetRule
from medspacy.visualization import visualize_ent
from medspacy.section_detection import SectionRule
import spacy
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import urllib.request
from sklearn import datasets
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.feature_selection import SelectFromModel
from sklearn.preprocessing import binarize
import nltk
from sklearn.feature_extraction.text import TfidfTransformer 
import pickle
#import dill as pickle
from string import digits 
from sklearn.feature_extraction.text import CountVectorizer
cols_names =  ['Number', 'Note', 'Op Note','Na', 'K', 'AG', 'Cr', 'Hgb']
import json
%matplotlib inline
warnings.filterwarnings('ignore')
#import dat          
def read_dat(key):
  return pickle.load(urllib.request.urlopen(f'https://github.com/rlancer/ai-meds/raw/main/{key}'))

warnings.filterwarnings('ignore')
xgbACS = read_dat('xgbclassMTHPACS2.dat')
xgbAAA = read_dat('xgbclassMTHPAAA2.dat')
xgbTachycardia = read_dat('xgbclassMTHPTachycardia2.dat')
xgbBradycardia = read_dat('xgbclassMTHPBradycardia2.dat')
xgbAbdominal_Pain = read_dat('xgbclassMTHPAbdominal_Pain2.dat')
xgbALF = read_dat('xgbclassMTHPALF2.dat')
xgbCHF = read_dat('xgbclassMTHPCHF2.dat')
xgbHTN_Emergency = read_dat('xgbclassMTHPHTN_Emergency2.dat')
xgbHypotensionShock = read_dat('xgbclassMTHPHypotensionShock2.dat')
xgbCardiac_arrest = read_dat('xgbclassMTHPCardiac_arrest2.dat')
xgbAirway_Obstruction = read_dat('xgbclassMTHPAirway_Obstruction2.dat')
xgbCOPD = read_dat('xgbclassMTHPCOPD2.dat')
xgbPneumonia = read_dat('xgbclassMTHPPneumonia2.dat')
xgbPneumothorax = read_dat('xgbclassMTHPPneumothorax2.dat')
xgbRespiratory_Failure = read_dat('xgbclassMTHPRespiratory_Failure2.dat')
xgbGI_Bleed = read_dat('xgbclassMTHPGI_Bleed2.dat')
xgbPancreatitis = read_dat('xgbclassMTHPPancreatitis2.dat')
xgbColitis = read_dat('xgbclassMTHPColitis2.dat')
xgbGastroenteritis = read_dat('xgbclassMTHPGastroenteritis2.dat')
xgbBacteremia = read_dat('xgbclassMTHPBacteremia2.dat')
xgbmeningitis_encephalitis = read_dat('xgbclassMTHPmeningitis_encephalitis2.dat')
xgbEndocarditis = read_dat('xgbclassMTHPEndocarditis2.dat')
xgbUTI = read_dat('xgbclassMTHPUTI2.dat')
xgbafib = read_dat('xgbclassMTHPafib2.dat')
xgbvented = read_dat('xgbclassMTHPvented2.dat')
xgbDM = read_dat('xgbclassMTHPDM2.dat')
xgbDKA = read_dat('xgbclassMTHPDKA2.dat')
xgbICH = read_dat('xgbclassMTHPICH2.dat')
xgbSDH = read_dat('xgbclassMTHPSDH2.dat')
xgbSAH = read_dat('xgbclassMTHPSAH2.dat')
xgbSeizure = read_dat('xgbclassMTHPSeizure2.dat')
xgbPE = read_dat('xgbclassMTHPPE2.dat')
xgbESRD = read_dat('xgbclassMTHPESRD2.dat')
xgbCKD = read_dat('xgbclassMTHPCKD2.dat')
xgbAKI = read_dat('xgbclassMTHPAKI2.dat')
xgbHTN = read_dat('xgbclassMTHPHTN2.dat')
xgbAsthma = read_dat('xgbclassMTHPAsthma2.dat')
xgbEtOH_WD = read_dat('xgbclassMTHPEtOH_WD2.dat')
xgbCAD = read_dat('xgbclassMTHPCAD2.dat')
xgbOSA = read_dat('xgbclassMTHPOSA2.dat')
xgbhypothyroid = read_dat('xgbclassMTHPhypothyroid2.dat')
xgbhyperthyroid = read_dat('xgbclassMTHPhyperthyroid2.dat')
xgbCVA = read_dat('xgbclassMTHPCVA2.dat')
xgbDLD = read_dat('xgbclassMTHPDLD2.dat')
xgbDelirium = read_dat('xgbclassMTHPDelirium2.dat')
xgbAI = read_dat('xgbclassMTHPAI2.dat')
xgbCushing = read_dat('xgbclassMTHPCushings2.dat')
xgbLupus = read_dat('xgbclassMTHPLupus2.dat')
xgbScleroderma = read_dat('xgbclassMTHPScleroderma2.dat')
xgbPericardial_Effusion = read_dat('xgbclassMTHPPericardial_Effusion2.dat')
xgbCholecystitis = read_dat('xgbclassMTHPCholecystitis2.dat')
xgbARDS = read_dat('xgbclassMTHPARDS2.dat')
xgbCellulitis = read_dat('xgbclassMTHPcellulitis2.dat')
xgbTBI = read_dat('xgbclassMTHPTBI2.dat')
xgbBurn = read_dat('xgbclassMTHPburn2.dat')
xgbPAD = read_dat('xgbclassMTHPPAD2.dat')
xgbBrain_Tumor = read_dat('xgbclassMTHPBrain_Tumor2.dat')
xgbGERD = read_dat('xgbclassMTHPGERD2.dat')
xgbpreopUEAngio = read_dat('xgbpreopclassMTUEAngioplasty2.dat')
xgbpreopImpella = read_dat('xgbpreopclassMTImpella2.dat')
xgbpreopCraniotomy = read_dat('xgbpreopclassMTCraniotomy2.dat')
xgbpreopCath = read_dat('xgbpreopclassMTCath2.dat')
xgbpreopCABG = read_dat('xgbpreopclassMTCABG2.dat')
xgbpreopAVR = read_dat('xgbpreopclassMTAVR2.dat')
xgbpreopAngiogram = read_dat('xgbpreopclassMTAngiogram2.dat')
xgbpreopAAA = read_dat('xgbpreopclassMTAAA_repair2.dat')


#medspacy text pipeline
warnings.filterwarnings('ignore')
nlp = spacy.load("en_ner_bc5cdr_md", disable={"tok2vec","tagger", "attribute_ruler", "lemmatizer", "parser"})
nlp = medspacy.load(nlp)
preprocessor = Preprocessor(nlp.tokenizer)
nlp.tokenizer = preprocessor
preprocess_rules = [
    PreprocessingRule(
        r"\[\*\*[\d]{1,4}-[\d]{1,2}(-[\d]{1,2})?\*\*\]",
        repl="01-01-2010",
        desc="Replace MIMIC date brackets with a generic date."
    ),
    
    PreprocessingRule(
        r"\[\*\*[\d]{4}\*\*\]",
        repl="2010",
        desc="Replace MIMIC year brackets with a generic year."
    ),
        PreprocessingRule(
        r"dx'd", 
        repl="Diagnosed", 
        desc="Replace abbreviation"
    ),
    
    PreprocessingRule(
        r"tx'd", 
        repl="Treated", 
        desc="Replace abbreviation"
    ),
    
        PreprocessingRule(
        r"\[\*\*[^\]]+\]", 
        desc="Remove all other bracketed placeholder text from MIMIC"
    ), lambda x: x.lower(),

        PreprocessingRule(
        r"~",
        repl=" ",
        desc="remove tilda"
    ),
]

#preprocessor.add(preprocess_rules)
target_matcher = nlp.get_pipe("medspacy_target_matcher")

target_rules = [
    TargetRule("hemicolectomy", "treatment"),
    TargetRule("hydrochlorothiazide", "treatment"),
    TargetRule("colon cancer", "disease"),
    TargetRule("radiotherapy", "disease",
              pattern=[{"lower": "xrt"}]),
    TargetRule("metastasis", "disease"),
    TargetRule("acs", "disease", 
              pattern=[{"lower": {"in": ["mi", "nstemi", "stemi"]}}]),
    TargetRule("heart attack", "disease"),
    TargetRule("myocardial infarction", "disease"),
    TargetRule("abdominal aortic aneurysm", "disease"),
    TargetRule("aaa", "disease"),
    TargetRule("triple a", "disease"), 
    TargetRule("aortic aneurysm", "disease"),
    TargetRule("abd aorta enlarged", "disease"),
    TargetRule("enlarged aorta", "disease"),
    TargetRule("tachycardia", "disease"),
    TargetRule("svt", "disease"),
    TargetRule("afib with rvr", "disease"),
    TargetRule("afib", "disease"),
    TargetRule("vtach", "disease"),
    TargetRule("ventricular tachycardia", "disease",
              pattern=[{"lower": {"in": ["atrial", "supraventricular", "a", "v", "sinus", "multifocal atrial", "ventricular"]}},
                       {"lower": {"in": ["tachycardia", "tach", "fibrillation", "fib", "flitter", "flutter"]}}]),
    TargetRule("atrial flutter", "disease",
              pattern=[{"lower": {"in": ["atrial", "ventricular"]}},
                       {"lower": {"in": ["tachycardia", "tach"]}}]),
    TargetRule("heart block", "disease"),
    TargetRule("wenckebach", "disease"),
    TargetRule("congestive heart failure", "disease"),
    TargetRule("heart failure", "disease"),
    TargetRule("cardiac failure", "disease"),
    TargetRule("low ejection fraction", "disease"),
    TargetRule("low ef", "disease"),
    TargetRule("congestive heart", "disease"),
    TargetRule("bradycardia", "disease"),
    TargetRule("malignant hypertension", "disease"),
    TargetRule("tachycardia", "disease",
               pattern=[{"lower": {"in": ["tachycardia", "tachycardic"]}}]),
    TargetRule("chf", "disease",
               pattern=[{"lower": {"in": ["chf", "hfref", "hfpef", "cardiomyopathy"]}}]),
    TargetRule("htn emergency", "disease",
               pattern=[{"lower": {"in":   ["htn", "hypertensive", "hypertension"]}},
                        {"lower": {"in":   ["crisis", "urgency", "emergency"]}}]),
    TargetRule("hypotension", "disease",
               pattern=[{"lower": {"in": ["hypotension", "hypotensive", "shock", "malperfusion", "hypoperfuse", "hypoperfusion"]}}]),
    TargetRule("pressors", "treatment",
               pattern=[{"lower": {"in":   ["pressors", "levophed", "pressor", "levo", "norepi", "norepinephrine", "neosynephrine", "vasopressin"]}}]),
    TargetRule("cardiac arrest", "disease",
               pattern=[{"lower": {"in": ["cardiopulmonary", "cardiac"]}},
                        {"lower": {"in": ["arrest", "death"]}}]),
    TargetRule("cardiopulmonary resuscitation", "treatment",
               pattern=[{"lower": {"in": ["cardiopulmonary",  "cardiac", "cardio"]}},
                        {"lower": {"in": ["resuscitation", "defib", "defibrillate", "defibrillation"]}}]),
    TargetRule("cpr", "treatment"),
    TargetRule("chronic obstructive lung disease", "treatment"),
    TargetRule("chronic obstructive pulmonary disease", "treatment"),
    TargetRule("tuberculosis", "treatment"),
    TargetRule("ntm", "treatment"),
    TargetRule("tb", "treatment"),
    TargetRule("tbm", "treatment"),
    TargetRule("airway obstruction", "disease",
               pattern=[{"lower": {"in": ["tracheal", "airway"]}},
                        {"lower": {"in": ["stenosis", "obstruction", "edema", "mass"]}}]),
    TargetRule("vocal chord edema", "disease",
               pattern=[{"lower": {"in": ["vocal", "airway"]}},
                        {"lower": {"in": ["chord"]}},
                        {"lower": {"in": ["swelling", "edema", "dysfunction"]}}]),
    TargetRule("copd", "disease",
               pattern=[{"lower": {"in": ["copd"]}}]),
    TargetRule("pneumonia", "disease",
               pattern=[{"lower": {"in":  ["pna", "pneumonia", "cap", "hcap", "vap"]}}]),
    TargetRule("lung infection", "treatment"),
    TargetRule("pulmonary infection", "treatment"),
    TargetRule("lung infiltrate", "treatment"),
    TargetRule("pulmonary infiltrate", "treatment"),
    TargetRule("pulmonary consolidation", "treatment"),
    TargetRule("perfusion defect", "treatment"),
    TargetRule("collapsed lung", "treatment"),
    TargetRule("air around lung", "treatment"),
    TargetRule("ptx", "disease",
               pattern=[{"lower": {"in":  ["ptx", "pneumothorax", "pneumothoraces"]}}]),
    TargetRule("respiratory failure", "disease",
               pattern=[{"lower": {"in": ["respiratory", "resp", "resp", "pulmonary"]}},
                        {"lower": {"in": ["arrest", "failure", "arrest", "distress"]}}]),  
    TargetRule("intubated", "treatment",
               pattern=[{"lower": {"in":  ["intubated", "vented", "ventilator", "ventilated"]}}]),
    TargetRule("liver failure", "disease",
               pattern=[{"lower": {"in": ["liver", "hepatic"]}},
                        {"lower": {"in": ["cirrhosis", "ascites", "cirrhosis", "cirrhotic", "varices"]}}]),
    TargetRule("left upper quadrant pain", "treatment"),
    TargetRule("right upper quadrant pain", "treatment"),
    TargetRule("right lower quadrant pain", "treatment"),
    TargetRule("hepatitis", "treatment"),
    TargetRule("transaminitis", "treatment"),
    TargetRule("abdominal pain", "disease",
               pattern=[{"lower": {"in": ["abdominal", "stomach", "gastric", "ruq", "luq", "rlq", "llq", "chest", "shoulder", "back", "neck", "ear"]}},
                        {"lower": {"in": ["pain", "discomfort", "distress", "dolor", "upset"]}}]),
    TargetRule("gi bleed", "disease",
               pattern=[{"lower": {"in":  ["gastrointestinal", "gi", "variceal", "gastric", "gastro", "intestinal", "stomach"]}},
                        {"lower": {"in":  ["bleed", "bleeding", "hemorrhage", "ooze"]}}]),
    TargetRule("gib", "disease"),
    TargetRule("hematchezia", "disease"),
    TargetRule("melena", "disease"),
    TargetRule("vomiting blood", "disease"),
    TargetRule("bright red blood per rectum", "disease"),
    TargetRule("brbpr", "disease"),
    TargetRule("inflamed pancreas", "disease"),
    TargetRule("elevated lipase", "disease"),
    TargetRule("pancreatic inflammation", "disease"),            
    TargetRule("pancreatitis", "disease",
               pattern=[{"lower": {"in":  ["pancreatitis"]}}]),
    TargetRule("colitis", "disease",
               pattern=[{"lower": {"in":  ["colitis"]}}]),
    TargetRule("bowel infection", "disease"),
    TargetRule("gut inflammation", "disease"),
    TargetRule("bowel inflammation", "disease"),
    TargetRule("inflamed bowel", "disease"),
    TargetRule("blood infection", "disease"), 
    TargetRule("positive blood culture", "disease"),
    TargetRule("gram negatives in blood", "disease"), 
    TargetRule("+ blood culture", "disease"),             
    TargetRule("gastroenteritis", "disease",
               pattern=[{"lower": {"in":  ["gastroenteritis", "enteritis"]}}]),
    TargetRule("bacteremia", "disease",
               pattern=[{"lower": {"in":  ["bacteremia", "septicemia"]}}]),
    TargetRule("meningitis", "disease",
               pattern=[{"lower": {"in":  ["meningitis"]}}]),
    TargetRule("endocarditis", "disease",
               pattern=[{"lower": {"in":  ["endocarditis", "vegetation"]}}]),
    TargetRule("nuchal rigidity ", "disease"),
    TargetRule("cns infection", "disease"),
    TargetRule("encephalitis", "disease"),
    TargetRule("septic emboli", "disease"),
    TargetRule("urinary tract infection", "disease"),
    TargetRule("positive urine", "disease"),
    TargetRule("irregularly irregular", "disease"),
    TargetRule("uti", "disease",
               pattern=[{"lower": {"in": ["uti", "dysuria"]}}]),
    TargetRule("diabetes mellitus", "disease"),
    TargetRule("dmt2", "disease"),
    TargetRule("iddm", "disease"),
    TargetRule("iddm2", "disease"),
    TargetRule("dm2", "disease"),
    TargetRule("dm", "disease",
              pattern=[{"lower": {"in": ["dm", "diabetes"]}}]),
    TargetRule("diabetic ketoacidosis", "disease",
               pattern=[{"lower": {"in":  ["diabetic", "diabetes"]}},
                        {"lower": {"in":  ["ketoacidosis", "ketoacidotic", "ketosis"]}}]),
    TargetRule("dka", "disease",
              pattern=[{"lower": {"in": ["dka", "ketoacidosis", "ketosis"]}}]),
    TargetRule("intracranial hemorrhage", "disease",
               pattern=[{"lower": {"in": ["intracranial", "intraventricular", "subdural", "dural", "subarachnoid", "arachnoid", "intracerebral", "brain", "cerebral"]}},
                        {"lower": {"in": ["hemorrhage", "hemorrhaging", "bleed", "hematoma"]}}]),
    TargetRule("ich", "disease",
               pattern=[{"lower": {"in": ["ivh", "ich", "sdh", "sdh", "sah", "sah"]}}]),
     TargetRule("seizure", "disease",
               pattern=[{"lower": {"in":   ["seizure", "epilepsy", "convulsion", "pseudoseizure"]}}]),
     TargetRule("pulmonary embolism", "disease",
               pattern=[{"lower": {"in": ["pulmonary", "saddle", "lung"]}},
                        {"lower": {"in": ["embolus", "clot", "embolisms", "embolism"]}}]),
    TargetRule("ckd", "disease"),
    TargetRule("end-stage renal disease", "disease"),
    TargetRule("end stage kidney disease", "disease"),
    TargetRule("esrd", "disease"),    
    TargetRule("chronic kidney disease", "disease",
               pattern=[{"lower": {"in": ["chronic", "acute", "endstage"]}},
                        {"lower": {"in": ["kidney", "renal", "nephrogenic"]}},
                        {"lower": {"in": ["disease", "failure", "injury", "dysfunction", "disease"]}}]),
    TargetRule("pe", "disease",
               pattern=[{"lower": {"in": ["pe"]}}]),
    TargetRule("alcohol withdrawal", "disease",
               pattern=[{"lower": {"in": ["alcohol", "etoh", "alcoholic", "delirium"]}},
                        {"lower": {"in": ["withdrawal", "tremens", "hallucinosis", "hallucination"]}}]),
    TargetRule("hypertension", "disease",
               pattern=[{"lower": {"in": ["htn", "hypertension", "hypertensive"]}}]),
    TargetRule("asthma", "disease",
               pattern=[{"lower": {"in": ["asthma", "asthmatic"]}}]),
    TargetRule("coronary artery disease", "disease",
               pattern=[{"lower": {"in":  ["coronary", "heart"]}},
                        {"lower": {"in":  ["artery", "vessel"]}},
                        {"lower": {"in":  ["disease", "occlusion", "obstruction", "spasm"]}}]),
    TargetRule("cad", "disease",
               pattern=[{"lower": {"in":  ["cad"]}}]),
    TargetRule("high blood pressure", "disease"),
    TargetRule("elevated blood pressure", "disease"),
    TargetRule("coronary obstruction", "disease"),
    TargetRule("hypothyroidism", "disease",
               pattern=[{"lower": {"in":  ["hypothyroidism", "hypothyroid", "myxedema"]}}]),
    TargetRule("hypoactive thyroid", "disease"),
    TargetRule("myxedema coma", "disease"),
    TargetRule("hyperactive thyroid", "disease"),
    TargetRule("hyperthyroidism", "disease",
               pattern=[{"lower": {"in":  ["hyperthyroidism","hyperthyroid",  "graves"]}}]),
    TargetRule("cva", "disease",
               pattern=[{"lower": {"in":  ["cva", "stroke"]}}]),
    TargetRule("ischemic stroke", "disease",
               pattern=[{"lower": {"in":  ["ischemic", "ischaemic", "mca", "pca", "aca", "hemorrhagic", "cerebrovascular"]}},
                        {"lower": {"in":  ["stroke", "accident"]}}]),
    TargetRule("osa", "disease",
               pattern=[{"lower": {"in":  ["osa"]}}]),
    TargetRule("obstructive sleep apnea", "disease"),
    TargetRule("sleep apnea", "disease"),
    TargetRule("high cholesterol", "disease"),
    TargetRule("delirium", "disease",
               pattern=[{"lower": {"in":  ["delirium", "confusion", "sundowning", "sundown", "sundowning", "confused", "delirious", "encephalopathy", "encephalopathic"]}}]),
    TargetRule("hyperlipidemia", "disease",
               pattern=[{"lower": {"in":  ["hyperlipidemia", "hld", "hld", "dld", "dld", "dyslipidemia", "dyslipidemic", "hyperlipidemic", "hypertriglyceridemia"]}}]),
    TargetRule("adrenal insufficiency", "disease"),
    TargetRule("low cortisol", "disease"),
    TargetRule("cushings disease", "disease"),
    TargetRule("hypocortisolemia", "disease",
               pattern=[{"lower": {"in":  ["hypoaldosteronism", "hypoaldosterone", "ai"]}}]),
    TargetRule("cushings", "disease",
               pattern=[{"lower": {"in":  ["cushings", "hypercortisolemia", "hyperaldosteronism", "hyperaldosterone", "cushingoid", "cushing"]}}]),
    TargetRule("lupus", "disease",
               pattern=[{"lower": {"in":  ["lupus", "sle"]}}]),
    TargetRule("systemic lupus erythematosus", "disease"),
    TargetRule("lupus erythematosus", "disease"),
    TargetRule("systemic lupus", "disease"),
    TargetRule("high cortisol", "disease"), 
    TargetRule("residual deficits", "disease"),
    TargetRule("systemic sclerosis", "disease"),
    TargetRule("hemopericardium", "disease"),
    TargetRule("pneumopericardium", "disease"),
    TargetRule("pneumoperitoneum", "disease"),
    TargetRule("hemoperitoneum", "disease"),
    TargetRule("scleroderma", "disease",
               pattern=[{"lower": {"in":  ["scleroderma", "crest", "crest", "sclerodactyly"]}}]),
    TargetRule("pericardial effusion", "disease",
               pattern=[{"lower": {"in":  ["pericardial", "epicardial", "becks"]}},
                        {"lower": {"in":  ["effusion", "tamponade",  "triad", "fluid", "hemopericardium"]}}]),
    TargetRule("cholecystitis", "disease"),
    TargetRule("cholecystitis", "disease",
               pattern=[{"lower": {"in": ["gallbladder", "inflamed", "necrotic", "hemorrhagic"]}},
                        {"lower": {"in": ["infection", "inflammation", "edema", "gallbladder", "necrosis"]}}]),
    TargetRule("ards", "disease",
               pattern=[{"lower": {"in":  ["ards"]}}]),
    TargetRule("acute respiratory distress syndrome", "disease"),
    TargetRule("respiratory distress syndrome", "disease"),            
    TargetRule("cellulitis", "disease"),
    TargetRule("skin infection", "disease",
               pattern=[{"lower": {"in":  ["skin", "nec", "necrotizing", "cellulitis", "cellulitic"]}},
                        {"lower": {"in":  ["infection", "fasc", "fasciitis", "inflammation"]}}]),
    TargetRule("traumatic brain injury", "disease"),
    TargetRule("brain injury", "disease",
               pattern=[{"lower": {"in":  ["brain", "coup", "head"]}},
                        {"lower": {"in":  ["tbi", "tbi", "trauma", "contracoup", "injury"]}}]),
    TargetRule("tbi", "disease",
               pattern=[{"lower": {"in":  ["tbi"]}}]),
    TargetRule("cancer", "disease",
               pattern=[{"lower": {"in":  ["cancer", "ca"]}}]), 
    TargetRule("burn", "disease",
               pattern=[{"lower": {"in":  ["burn"]}}]),
    TargetRule("thermal injury", "disease"),
    TargetRule("cp", "disease"),
    TargetRule("pvd", "disease"),
    TargetRule("pad", "disease"),
    TargetRule("gbm", "disease"),
    TargetRule("glioblastoma", "disease"),
    TargetRule("astrocytoma", "disease"),
    TargetRule("peripheral vascular disease", "disease",
               pattern=[{"lower": {"in":  ["peripheral", "distal"]}},
                        {"lower": {"in":  ["vascular", "artery", "arterial", "venous", "vein"]}},
                        {"lower": {"in":  ["disease", "dysfunction"]}}]),
    TargetRule("brain tumor", "disease",
               pattern=[{"lower": {"in":  ["brain", "glioblastoma", "cerebral", "supratentorial"]}},
                        {"lower": {"in":  ["met", "metastasis", "tumor", "abscess", "cyst" "multiforme", "mass"]}}]),
    TargetRule("gerd", "disease",
               pattern=[{"lower": {"in":  ["gerd"]}}]),
    TargetRule("gastroesophageal reflux disease", "disease",
               pattern=[{"lower": {"in":  ["gastroesophageal reflux disease"]}}]),
    TargetRule("gastroesophageal refluxes", "disease"),
    TargetRule("gastric reflux", "disease"),
    TargetRule("empyema", "disease"),
    TargetRule("reflux disease", "disease"),
    TargetRule("hemothorax", "disease"),
    TargetRule("pleural effusion", "disease",
               pattern=[{"lower": {"in":  ["pleural", "lung", "complicated ", "loculated"]}},
                        {"lower": {"in": ["fluid", "effusion"]}}]),
    TargetRule("vancomycin", "treatment",
               pattern=[{"lower":  {"in": ["vancomycin", "vanco", "vancocin"]}}]),
    TargetRule("ceftriaxone", "treatment",
               pattern=[{"lower": {"in": ["rocephin", "ceftriaxone"]}}]),
    TargetRule("cefepime", "treatment",
               pattern=[{"lower": {"in": ["maxipime", "cefepime"]}}]),
    TargetRule("piperacillin", "treatment",
               pattern=[{"lower": {"in": ["zosyn", "piperacillin/tazobactam", "pip", "pip/tazo", "pip-tazo", "piptazo"]}}]),
    TargetRule("tazobactam", "treatment",
               pattern=[{"lower": {"in": ["zosyn", "piperacillin/tazobactam", "tazo", "pip/tazo", "pip-tazo", "piptazo"]}}]),      
    TargetRule("tigacycline", "treatment",
               pattern=[{"lower": {"in": ["tygacil", "tigacycline"]}}]),
    TargetRule("amikacin", "treatment",
               pattern=[{"lower": {"in": ["amikacin"]}}]),
    TargetRule("meropenem", "treatment",
               pattern=[{"lower": {"in": ["meropenem"]}}]),
    TargetRule("imipenem", "treatment",
               pattern=[{"lower": {"in": ["imipenem"]}}]),
    TargetRule("ertapenem", "treatment",
               pattern=[{"lower": {"in": ["ertapenem"]}}]),
    TargetRule("ampicillin", "treatment",
               pattern=[{"lower": {"in":  ["unasyn", "ampicillin/sulbactam", "amp/sulbactam", "amp", "amp-sulbactam"]}}]),
    TargetRule("sulbactam", "treatment",
               pattern=[{"lower": {"in":  ["unasyn", "ampicillin/sulbactam", "amp/sulbactam", "amp", "amp-sulbactam"]}}]),
    TargetRule("gentamicin", "treatment",
               pattern=[{"lower": {"in": ["gentamicin"]}}]),
    TargetRule("cabg", "treatment"),
    TargetRule("cab", "treatment"),
    TargetRule("cabg x1", "treatment",
               pattern=[{"lower": {"in": ["cabg", "cab"]}},
                        {"lower": {"in": ["x1", "x2", "x3", "x4"]}}]),
    TargetRule("coronary artery bypass graft", "treatment"),
    TargetRule("coronary artery bypass grafting", "treatment"),
    TargetRule("gout", "disease"),
    TargetRule("schwannoma", "disease"),
    TargetRule("cheng", category='cheng', pattern=[{'LOWER': {'IN': ['cheng', 'cheng-hsi']}}]),
    TargetRule("suarez", category='suarez', pattern=[{'LOWER': {'IN': ['suarez']}}]),
    TargetRule("sweeney", category='sweeney', pattern=[{'LOWER': {'IN': ['sweeney', 'sweeny']}}]),
    TargetRule("george", category='george', pattern=[{'LOWER': {'IN': ['george', 'joggy']}}]),
    TargetRule("goel", category='goel', pattern=[{'LOWER': {'IN': ['goel']}}]),
    TargetRule("wolf", category='wolf', pattern=[{'LOWER': {'IN': ['wolf']}}]),
    TargetRule("guha", category='guha', pattern=[{'LOWER': {'IN': ['guha']}}]),
    TargetRule("reul", category='reul', pattern=[{'LOWER': {'IN': ['reul']}}]),
    TargetRule("kleiman", category='kleiman', pattern=[{'LOWER': {'IN': ['kleiman']}}]),
    TargetRule("ramchandani", category='ramchandani', pattern=[{'LOWER': {'IN': ['ramchandani']}}]),
    TargetRule("lawrie", category='lawrie', pattern=[{'LOWER': {'IN': ['lawrie']}}]),
    TargetRule("reardon", category='reardon', pattern=[{'LOWER': {'IN': ['reardon']}}]),
    TargetRule("steele", category='steele', pattern=[{'LOWER': {'IN': ['steele']}}]),
    TargetRule("trask", category='trask', pattern=[{'LOWER': {'IN': ['trask']}}]),
    TargetRule("taylor", category='taylor', pattern=[{'LOWER': {'IN': ['taylor']}}]),
    TargetRule("extremity balloon angioplasty", 'EXTREMITY_ANGIOGRAM'),
    TargetRule("EXTREMITY_ANGIOGRAM", "EXTREMITY_ANGIOGRAM", 
              pattern=[{'LOWER': {'IN': ['extremity', 'rue', 'rle', 'lle', 'lue']}},
                       {'LOWER': {'IN': ['angiogram', 'angio']}}]),
    TargetRule("TRANSPLANTATION_LUNG", category='lung_transplant', 
              pattern=[{'LOWER': {'IN': ['transplantation', 'lung']}},
                       {'LOWER': {'IN': ['lung', 'transplantation', 'transplant']}}]),
    TargetRule("PERICARDIAL_WINDOW", category='pericardial_window', 
              pattern=[{'LOWER': {'IN': ['pericardial']}},
                       {'LOWER': {'IN': ['window']}}]),
    TargetRule("CORONARY_ANGIOGRAM", category='CORONARY_ANGIOGRAM', pattern=[{'LOWER': {'IN': ['rhc', 'lhc']}}]),
    TargetRule("CORONARY_ANGIOGRAM", category='CORONARY_ANGIOGRAM', 
              pattern=[{'LOWER': {'IN': ['coronary', 'heart']}},
                       {'LOWER': {'IN': ['angiogram', 'angio', 'cath', 'catheterization']}}]),
     TargetRule("MAZE", category='MAZE', pattern=[{'LOWER': {'IN': ['maze']}}]),        
     TargetRule("lvad", category='lvad', pattern=[{'LOWER': {'IN': ['lvad']}}]),
     TargetRule("lvad", category='lvad', pattern=[{'LOWER': {'IN': ['left ventricular assist device']}}]),
     TargetRule("mvr", category='mvr', pattern=[{'LOWER': {'IN': ['mvr']}}]),
     TargetRule("avr", category='avr', pattern=[{'LOWER': {'IN': ['mvr']}}]),
     TargetRule("tvr", category='tvr', pattern=[{'LOWER': {'IN': ['tvr']}}]),                                           
     TargetRule("mvr", category='mvr', 
              pattern=[{'LOWER': {'IN': ['mitral']}},
                       {'LOWER': {'IN': ['valve', 'valvulopathy']}},
                       {'LOWER': {'IN': ['repair', 'replacement', 'replaced']}}]),
     TargetRule("avr", category='avr', 
              pattern=[{'LOWER': {'IN': ['aortic']}},
                       {'LOWER': {'IN': ['valve', 'valvulopathy']}},
                       {'LOWER': {'IN': ['repair', 'replacement', 'replaced']}}]),
     TargetRule("tvr", category='tvr', 
              pattern=[{'LOWER': {'IN': ['tricuspid']}},
                       {'LOWER': {'IN': ['valve', 'valvulopathy']}},
                       {'LOWER': {'IN': ['repair', 'replacement', 'replaced']}}]),
     TargetRule("cabg", "cabg"),
     TargetRule("cab", "cabg"),
     TargetRule("cabg x1", "cabg",
              pattern=[{"lower": {"in": ["cabg", "cab"]}},
                       {"lower": {"in": ["x1", "x2", "x3", "x4"]}}]),
     TargetRule("coronary artery bypass graft", "cabg"),
     TargetRule("coronary artery bypass", "cabg"),
     TargetRule("coronary artery bypass grafting", "cabg"),
     TargetRule("impella", "impella"),
     TargetRule("abdominal aortic aneurysm", "disease"),
     TargetRule("aaa", "aaa"),
     TargetRule("triple a", "aaa"), 
     TargetRule("aortic aneurysm", "aaa"),
     TargetRule("oht", "oht"),
     TargetRule("aortic dissection", "aortic_dissection",
              pattern=[{'LOWER': {'IN': ['aortic', 'ascending', 'aorta']}},
                       {'LOWER': {'IN': ['dissection', 'tear']}}]),
     TargetRule("oht", "oht",
              pattern=[{'LOWER': {'IN': ['orthotopic', 'heart', 'cardiac']}},
                       {'LOWER': {'IN': ['transplant', 'transplantation']}}]),
     TargetRule("craniotomy", "craniotomy"),
     TargetRule("craniectomy", "craniectomy"),
     TargetRule("kyphoplasty", "kyphoplasty"),
     TargetRule("ivor-lewis esophagectomy", "ivorlewis"),
     TargetRule("ivor lewis esophagectomy", "ivorlewis"),
     TargetRule("ivor lewis", "ivorlewis"), 



    
]

target_matcher.add(target_rules)

context = nlp.get_pipe("medspacy_context")
context_rules = [
    ConTextRule(literal='history', category='HISTORICAL', pattern=[{'LOWER': {'IN': ['Past Medical History:', 'pmhx', 'pmh', 'history', 'hx', 'hist', 'ho']}}], direction='FORWARD'),
    ConTextRule(literal='h/o', category='HISTORICAL', pattern=None, direction='FORWARD'),
    ConTextRule(literal='past history', category='HISTORICAL', pattern=[{'LOWER': 'past'}, {'LOWER': 'medical', 'OP': '?'}, {'LOWER': 'history'}], direction='FORWARD'), 
    ConTextRule(literal='diagnosed in ', category='HISTORICAL', pattern=[{'LOWER': 'diagnosed'}, {'LOWER': 'in'}, {'LOWER': {'REGEX': '^[\\d]{4}$'}}], direction='BIDIRECTIONAL'),
    ConTextRule(literal='diagnosed in ', category='HISTORICAL', pattern=[{'LOWER': 'diagnosed'}, {'LOWER': 'in'}, {'LOWER': {'REGEX': '^[\\d]{4}$'}}], direction='BIDIRECTIONAL'),
    ConTextRule(literal='diagnosed in ', category='HISTORICAL', pattern=[{'LOWER': 'diagnosed'}, {'LOWER': 'in'}, {'LOWER': {'REGEX': '^[\\d]{4}$'}}], direction='BIDIRECTIONAL'),
    ConTextRule(literal='diagnosed in ', category='HISTORICAL', pattern=[{'LOWER': 'diagnosed'}, {'LOWER': 'in'}, {'LOWER': {'REGEX': '^[\\d]{4}$'}}], direction='BIDIRECTIONAL'),
    ConTextRule(literal='diagnosed in ', category='HISTORICAL', pattern=[{'LOWER': 'diagnosed'}, {'LOWER': 'in'}, {'LOWER': {'REGEX': '^[\\d]{4}$'}}], direction='BIDIRECTIONAL'),
    ConTextRule(literal='diagnosed in ', category='HISTORICAL', pattern=[{'LOWER': 'diagnosed'}, {'LOWER': 'in'}, {'LOWER': {'REGEX': '^[\\d]{4}$'}}], direction='BIDIRECTIONAL'),
    ConTextRule(literal='diagnosed in ', category='HISTORICAL', pattern=[{'LOWER': 'diagnosed'}, {'LOWER': 'in'}, {'LOWER': {'REGEX': '^[\\d]{4}$'}}], direction='BIDIRECTIONAL'),
    ConTextRule(literal='diagnosed in ', category='HISTORICAL', pattern=[{'LOWER': 'diagnosed'}, {'LOWER': 'in'}, {'LOWER': {'REGEX': '^[\\d]{4}$'}}], direction='BIDIRECTIONAL'),
    ConTextRule(literal='surgeon', category='surgeon', pattern=[{'LOWER': {'IN': ['surgeon', 'surgeon(s)']}}], max_targets = 1, direction='FORWARD'),
    ConTextRule(literal='surgeon', category='surgeon', pattern=None, max_targets = 1, direction='FORWARD'),
    ConTextRule(literal='procedure', category='procedure', pattern=[{'LOWER': {'IN': ['procedure', 'procedure:','procedure(s)', 'procedures', 'procedures:']}}], max_targets = 1, direction='FORWARD'),
]   
context.add(context_rules)

sectionizer = nlp.add_pipe("medspacy_sectionizer")

section_patterns = [
    SectionRule("Brief Hospital Course:", "hospital_course",  "HISTORY_OF_PRESENT_ILLNESS" ),
    SectionRule(literal="PAST MEDICAL HISTORY:", category="past_medical_history", pattern=None, on_match=None, parents=[], parent_required=False),
    SectionRule(literal="Past Medical History:", category="past_medical_history", pattern=None, on_match=None, parents=[], parent_required=False),
    SectionRule(literal="Current Medical Problems:", category="past_medical_history", pattern=None, on_match=None, parents=[], parent_required=False),
    SectionRule(literal="HISTORY:", category="past_medical_history", pattern=None, on_match=None, parents=[], parent_required=False),
    SectionRule(literal="PAST HISTORY", category="past_medical_history", pattern=None, on_match=None, parents=[], parent_required=False),
    SectionRule(literal="CLINICAL HISTORY:", category="past_medical_history", pattern=None, on_match=None, parents=[], parent_required=False),
    SectionRule(literal="History of Chronic Illness:", category="past_medical_history", pattern=None, on_match=None, parents=[], parent_required=False),
    SectionRule(literal="MHx:", category="past_medical_history", pattern=None, on_match=None, parents=[], parent_required=False),
    SectionRule(literal="PAST HISTORY:", category="past_medical_history", pattern=None, on_match=None, parents=[], parent_required=False),
    SectionRule(literal="Past med history:", category="past_medical_history", pattern=None, on_match=None, parents=[], parent_required=False),
    SectionRule(literal="PAST MEDICAL Hx:", category="past_medical_history", pattern=None, on_match=None, parents=[], parent_required=False),
    SectionRule(literal="PAST SURGICAL HISTORY:", category="past_medical_history", pattern=None, on_match=None, parents=[], parent_required=False),
    SectionRule(literal="PMH:", category="past_medical_history", pattern=None, on_match=None, parents=[], parent_required=False),
    SectionRule(literal="PMHx:", category="past_medical_history", pattern=None, on_match=None, parents=[], parent_required=False),
    SectionRule(literal="UNDERLYING MEDICAL CONDITION:", category="past_medical_history", pattern=None, on_match=None, parents=[], parent_required=False),
    SectionRule(literal="Family History:", category="family_history", pattern=None, on_match=None, parents=[], parent_required=False),
    SectionRule(literal="FAMILY HISTORY:", category="family_history", pattern=None, on_match=None, parents=[], parent_required=False),
    SectionRule(literal="Fam Hx:", category="family_history", pattern=None, on_match=None, parents=[], parent_required=False),
    SectionRule(literal="Family Hx:", category="family_history", pattern=None, on_match=None, parents=[], parent_required=False),
    SectionRule(literal="Fam/Hx:", category="family_history", pattern=None, on_match=None, parents=[], parent_required=False),
    SectionRule(literal="Fam_Hx:", category="family_history", pattern=None, on_match=None, parents=[], parent_required=False),
    SectionRule(literal="Assessment and Plan", category="observation_and_plan", pattern=None, on_match=None, parents=[], parent_required=False),

]

sectionizer.add(section_patterns)

postprocessor = nlp.add_pipe("medspacy_postprocessor", config={"debug": True})

postprocess_rules = [
    # Instantiate our rule
    PostprocessingRule(
        # Pass in a list of patterns
        patterns=[
            # The pattern will check if the entitie's section is "patient_instructions"
            PostprocessingPattern(condition=lambda ent: ent._.section_category, success_value="past_medical_history"),
        ],
        # If all patterns are True, this entity will be removed.
        action=postprocessing_functions.set_historical,
        description="Remove any entities from the instructions section."
    ),
    
]

postprocessor.add(postprocess_rules)

#screen grabber pipeline
warnings.filterwarnings('ignore')
nlp2 = medspacy.load(disable={"medspacy_context"})
nlp2.pipe_names
preprocessor2 = Preprocessor(nlp2.tokenizer)
nlp2.tokenizer = preprocessor2

preprocess_rules2 = [
    PreprocessingRule(
        r"\[\*\*[\d]{1,4}-[\d]{1,2}(-[\d]{1,2})?\*\*\]",
        repl="01-01-2010",
        desc="Replace MIMIC date brackets with a generic date."
    ),
    
    PreprocessingRule(
        r"\[\*\*[\d]{4}\*\*\]",
        repl="2010",
        desc="Replace MIMIC year brackets with a generic year."
    ),
    
    PreprocessingRule(
        r"dx'd", 
        repl="Diagnosed", 
        desc="Replace abbreviation"
    ),
    
    PreprocessingRule(
        r"tx'd", 
        repl="Treated", 
        desc="Replace abbreviation"
    ),
    
        PreprocessingRule(
        r"\[\*\*[^\]]+\]", 
        desc="Remove all other bracketed placeholder text from MIMIC"
    ), lambda x: x.lower(),

        PreprocessingRule(
        r"~",
        repl=" ",
        desc="remove tilda"
    ),
]

#preprocessor2.add(preprocess_rules2)
target_matcher2 = nlp2.get_pipe("medspacy_target_matcher")

target_rules2 = [
        TargetRule("", "number",
               pattern=[{"LIKE_NUM": True}]),
        TargetRule("male", "male", pattern=[{'LOWER': {'IN': ['man', 'y/o/m', 'yom']}}]),
        TargetRule("male", "male", 
                   pattern=[{'LOWER': {'IN': ['year', 'y', 'y/']}},
                            {'LOWER': {'IN': ['old', 'o', '/o']}},
                            {'LOWER': {'IN': ['man', 'male', 'm']}}]),
        TargetRule("female", "female", pattern=[{'LOWER': {'IN': ['woman', 'y/o/w', 'yow', 'female']}}]),
        TargetRule("female", "female", 
                   pattern=[{'LOWER': {'IN': ['year', 'y', 'y/']}},
                            {'LOWER': {'IN': ['old', 'o', '/o']}},
                            {'LOWER': {'IN': ['woman', 'female', 'w']}}]),
        TargetRule("male", "male", 
                   pattern=[{'LOWER': {'IN': ['year', 'yo', 'y/o', 'year', 'y.o.']}},
                            {'LOWER': {'IN': ['old', 'man', 'male', 'm']}}]) ,
        TargetRule("female", "female", 
                   pattern=[{'LOWER': {'IN': ['year', 'yo', 'y/o', 'year', 'y.o.']}},
                            {'LOWER': {'IN': ['old', 'woman', 'female', 'f', 'w']}}]) ,

]

target_matcher2.add(target_rules2)
context2 = nlp2.add_pipe("medspacy_context", config={"rules": None})

context_rules2 = [
    ConTextRule(literal='Na', category='NA', pattern=[{'LOWER': {'IN': ['na:', 'na', 'sodium', 'sodio', 'sod', 'sodium:', 'sodio:', 'sod:']}}], max_targets = 1, direction='FORWARD'),
    ConTextRule(literal='K', category='K', pattern=[{'LOWER': {'IN': ['k+:', 'potassium', 'potassio', 'pot', 'k+', 'k', 'k:', 'potassium:', 'potassio:', 'pot:']}}], max_targets = 1, direction='FORWARD'),
    ConTextRule(literal='kul', category='kul', pattern=[{'LOWER': {'IN': ['k/ul:', 'k/ul:', 'ul']}}], max_targets = 1, direction='FORWARD'),
    ConTextRule(literal='Systolic EF', category='sysef', pattern=[{'LOWER': {'IN': ['systolic', 'sys']}},
                                                              {'LOWER': {'IN': ['ef', 'ef:']}}], max_targets = 1, direction='FORWARD'),
    ConTextRule(literal='Creatinine', category='CREATININE', pattern=[{'LOWER': {'IN': ['cr:', 'creatinine:', 'creatinine', 'creat', 'cr', 'creat:']}}], max_targets = 1, direction='FORWARD'),
    ConTextRule(literal='protein', category='PROTEIN', pattern=[{'LOWER': {'IN': ['prot', 'protein', 'prot:', 'protein:']}}], max_targets = 1, direction='FORWARD'),
    ConTextRule(literal='Chloride', category='CHLORIDE', pattern=[{'LOWER': {'IN': ['cl:', 'cl', 'chloride', 'chloride:']}}], max_targets = 1, direction='FORWARD'),
    ConTextRule(literal='CO2', category='CO2', pattern=[{'LOWER': {'IN': ['c02:', 'c02','co2', 'hc03:', 'bicarbonate', 'bicarb', 'co2:', 'hc03', 'bicarbonate:', 'bicarb:']}}], max_targets = 1, direction='FORWARD'),
    ConTextRule(literal='CO2', category='CO2', pattern=[{'LOWER': {'IN': ['carbon']}},
                                                        {'LOWER': {'IN': ['dioxide']}}], max_targets = 1, direction='FORWARD'),ConTextRule(literal='BUN', category='BUN', pattern=[{'LOWER': {'IN': ['bun:', 'bun', 'urea:', 'urea']}}], max_targets = 1, direction='FORWARD'),
    ConTextRule(literal='BUN', category='BUN', pattern=[{'LOWER': {'IN': ['blood']}},
                                                        {'LOWER': {'IN': ['urea']}},
                                                        {'LOWER': {'IN': ['nitrogen', 'nitrogen:']}}], max_targets = 1, direction='FORWARD'),
    ConTextRule(literal='Calcium', category='CALCIUM', pattern=[{'LOWER': {'IN': ['ca:', 'ca+', 'calcium', 'ca', 'ca+:', 'calcium:']}}], max_targets = 1, direction='FORWARD'),
    ConTextRule(literal='Glucose', category='GLUCOSE', pattern=[{'LOWER': {'IN': ['glucose:', 'glucose', 'fs', 'gluc', 'fs:', 'gluc:']}}], max_targets = 1, direction='FORWARD'),
    ConTextRule(literal='Glucose', category='GLUCOSE', pattern=[{'LOWER': {'IN': ['finger']}},
                                                                {'LOWER': {'IN': ['glucose:', 'glucose', 'stick', 'stick:']}}],max_targets = 1, direction='FORWARD'),
    ConTextRule(literal='Hemoglobin', category='HEMOGLOBIN', pattern=[{'LOWER': {'IN': ['hemoglobin:', 'hemoglobin', 'hgb', 'hgb:']}}], max_targets = 1, direction='FORWARD'),
    ConTextRule(literal='WBC', category='WBC', pattern=[{'LOWER': {'IN': ['wbc:', 'wbc', 'white cell', 'white blood cell', 'white cells', 'white blood cells', 'white cell:', 'white blood cell:', 'white cells:', 'white blood cells:']}}], max_targets = 1, direction='FORWARD'),
    ConTextRule(literal='Platelet', category='Platelet', pattern=[{'LOWER': {'IN': ['platelet:', 'plat', 'platelet', 'platelets', 'plt', 'platelets:', 'plt:']}}], max_targets = 1, direction='FORWARD'),
    ConTextRule(literal='Albumin', category='ALBUMIN', pattern=[{'LOWER': {'IN': ['albumin:', 'albumin', 'alb', 'alb:']}}], max_targets = 1, direction='FORWARD'),
    ConTextRule(literal='Bilirubin', category='BILIRUBIN', pattern=[{'LOWER': {'IN': ['bilirubin:', 'bilirubin', 'bili', 'bili:']}}], max_targets = 1, direction='FORWARD'),
    ConTextRule(literal='AST', category='AST', pattern=[{'LOWER': {'IN': ['ast:', 'ast', 'aspartate aminotransferase', 'aspartate aminotransferase:']}}], max_targets = 1, direction='FORWARD'),
    ConTextRule(literal='ALT', category='ALT', pattern=[{'LOWER': {'IN': ['alt:', 'alt', 'alanine transaminase', 'alanine transaminase:']}}], max_targets = 1, direction='FORWARD'),
    ConTextRule(literal='Alk Phos', category='ALK PHOS', 
                pattern=[{'LOWER': {'IN': ['ALK', 'alk', 'alkaline', 'alkphos', 'alkphos:']}},
                         {'LOWER': {'IN': ['PHOS', 'phos', 'phosphatase', 'phosphatase:', 'phos:']}}], max_targets = 1, direction='FORWARD'),
    ConTextRule(literal='Alk Phos', category='ALK PHOS', pattern=[{'LOWER': {'IN': ['alkphos', 'alkphos:']}}], max_targets = 1, direction='FORWARD'),
    ConTextRule(literal='Temperature', category='TEMPERATURE', pattern=[{'LOWER': {'IN': ['temp:', 'temp', 'temperature', 'temperature:']}}], max_targets = 1, direction='FORWARD'),
    ConTextRule(literal='Heary Rate', category='HEART RATE', pattern=[{'LOWER': {'IN': ['pulse:', 'pulse', 'hr', 'hr:']}}], max_targets = 1, direction='FORWARD'),
    ConTextRule(literal='Heart Rate', category='HEART RATE', 
                pattern=[{'LOWER': {'IN': ['heart', 'pulse']}},
                         {'LOWER': {'IN': ['rate:', 'rate']}}], max_targets = 1, direction='FORWARD'),
    ConTextRule(literal='Systolic BP', category='SYSTOLIC', pattern=[{'LOWER': {'IN': ['systolic:', 'systolic', 'sbp', 'sbp:']}}], max_targets = 1, direction='FORWARD'),
    ConTextRule(literal='Systolic BP', category='SYSTOLIC', 
                pattern=[{'LOWER': {'IN': ['systolic']}},
                         {'LOWER': {'IN': ['blood', 'pressure']}}], max_targets = 1, direction='FORWARD'),
    ConTextRule(literal='Systolic BP', category='SYSTOLIC', 
                pattern=[{'LOWER': {'IN': ['systolic']}},
                         {'LOWER': {'IN': ['blood']}},
                         {'LOWER': {'IN': ['pressure', 'pressure:']}}], max_targets = 1, direction='FORWARD'),
    ConTextRule(literal='Diastolic BP', category='DIASTOLIC', pattern=[{'LOWER': {'IN': ['diastolic:', 'diastolic', 'dbp', 'dbp:']}}], max_targets = 1, direction='FORWARD'),
    ConTextRule(literal='Diastolic BP', category='DIASTOLIC', 
                pattern=[{'LOWER': {'IN': ['diastolic']}},
                         {'LOWER': {'IN': ['pressure', 'pressure:']}}]  , max_targets = 1, direction='FORWARD'),
    ConTextRule(literal='Diastolic BP', category='DIASTOLIC', 
                pattern=[{'LOWER': {'IN': ['diastolic']}},
                         {'LOWER': {'IN': ['blood']}},
                         {'LOWER': {'IN': ['pressure', 'pressure:']}}]  , max_targets = 1, direction='FORWARD'),
    ConTextRule(literal='Mean BP', category='MAP', pattern=[{'LOWER': {'IN': ['map:', 'map']}}], max_targets = 1, direction='FORWARD'),
    ConTextRule(literal='Mean BP', category='MAP', pattern=[{'LOWER': {'IN': ['mean']}},
                                                            {'LOWER': {'IN': ['pressure', 'pressure:']}}], max_targets = 1, direction='FORWARD'),
    ConTextRule(literal='Mean BP', category='MAP', pattern=[{'LOWER': {'IN': ['mean']}},
                                                            {'LOWER': {'IN': ['arterial', 'artery']}},
                                                            {'LOWER': {'IN': ['pressure', 'pressure:']}}], max_targets = 1, direction='FORWARD'),
    ConTextRule(literal='Blood Pressure', category='SYSTOLIC', pattern=[{'LOWER': {'IN': ['bp:', 'bp']}}], max_targets = 1, direction='FORWARD'),
    ConTextRule(literal='Blood Pressure', category='SYSTOLIC', pattern=[{'LOWER': {'IN': ['blood', 'arterial']}},
                                                                  {'LOWER': {'IN': ['pressure', 'pressure:', 'line', 'line:']}}], max_targets = 1, direction='FORWARD'),
    ConTextRule(literal='Respiratory Rate', category='RESPIRATORY RATE', pattern=[{'LOWER': {'IN': ['rr', 'rr:', 'resp', 'resp:']}}], max_targets = 1, direction='FORWARD'),
    ConTextRule(literal='Respiratory Rate', category='RESPIRATORY RATE', pattern=[{'LOWER': {'IN': ['respiratory', 'resp']}},
                                                                                  {'LOWER': {'IN': ['rate:', 'rate']}}], max_targets = 1, direction='FORWARD'),    
    ConTextRule(literal='O2 Sat', category='O2 SAT', pattern=[{'LOWER': {'IN': ['spo2', 'spo2:']}}], max_targets = 1, direction='FORWARD'),
    ConTextRule(literal='O2 Sat', category='O2 SAT', pattern=[{'LOWER': {'IN': ['o2', 'oxygen']}},
                                                              {'LOWER': {'IN': ['sat:', 'sat', 'saturation']}}], max_targets = 1, direction='FORWARD'),
    ConTextRule(literal='aPTT', category='PTT', pattern=[{'LOWER': {'IN': ['aptt', 'ptt', 'aptt:', 'ptt:']}}], max_targets = 1, direction='FORWARD'),
    ConTextRule(literal='aPTT', category='PTT', pattern=[{'LOWER': {'IN': ['partial', 'activated']}},
                                                         {'LOWER': {'IN': ['thromboplastin']}},
                                                         {'LOWER': {'IN': ['time', 'time:']}}], max_targets = 1, direction='FORWARD'),
    ConTextRule(literal='PT', category='PT', pattern=[{'LOWER': {'IN': ['pt', 'pt:']}}], max_targets = 1, direction='FORWARD'),
    ConTextRule(literal='PT', category='PT', pattern=[{'LOWER': {'IN': ['prothrombin']}},
                                                      {'LOWER': {'IN': ['time', 'time:']}}],max_targets = 1, direction='FORWARD'),
    ConTextRule(literal='INR', category='INR', pattern=[{'LOWER': {'IN': ['inr', 'inr:']}}], max_targets = 1, direction='FORWARD'),
    ConTextRule(literal='INR', category='INR', pattern=[{'LOWER': {'IN': ['international']}},
                                                        {'LOWER': {'IN': ['normal', 'normalized']}},
                                                        {'LOWER': {'IN': ['ratio', 'ratio:']}}],max_targets = 1, direction='FORWARD'),
    ConTextRule(literal='AGE', category='AGE', pattern=[{'LOWER': {'IN': ['yow', 'y/o/w', 'yom']}}], max_targets = 1, direction='BACKWARD'),
    ConTextRule(literal='AGE', category='AGE', pattern=[{'LOWER': {'IN': ['year', 'y', 'y/', 'yo']}},
                                                        {'LOWER': {'IN': ['old', 'o', '/o', ""]}},
                                                        {'LOWER': {'IN': ['man', 'male', 'woman', 'female', 'm', 'f', 'w']}}],max_targets = 1, direction='BACKWARD'),
    ConTextRule(literal='AGE', category='AGE', pattern=[{'LOWER': {'IN': ['year', 'yo', 'y/o', 'year']}},
                                                        {'LOWER': {'IN': ['old', 'man', 'male', 'woman', 'female', 'm', 'f', 'w']}}],max_targets = 1, direction='BACKWARD'),              

]   
context2.add(context_rules2)

#medical workup
Dx5Wk1 = "Pt presents with evidence of acute coronary syndrome. Ddx includes type 1 versus type 2.  Suspect NSTEMI."
Dx5Wk2 = "Pt presents with evidence of unstable angina."
Dx5Wk3 = "Pt presents with evidence of acute coronary syndrome and cardiac arrest."
Dx5Wk4 = "Pt presents with evidence of ischemia in setting of bleeding.  Will need to maximize O2 delivery and minimize myocardial O2 demand without compromising hemodynamics."
Dx5Wk5 = "Pt presents with evidence of ischemia in setting of Coagulopathy.  Will need to maximize O2 delivery and minimize myocardial O2 demand without compromising hemodynamics."
Dx6Wk1 = "Pt with evidence of AAA on imaging."
Dx6Wk2 = "Pt with AAA and tachycardia.  Will need to reduce shear stress on flap."
Dx6Wk3 = "Pt presents with evidence of AAA and CHF concerning for possible aortic valvulopathy."
Dx6Wk4 = "Pt with AAA and hypertension, will need to control HR and BP."
Dx6Wk5 = "Pt with AAA and evidence of hypotension and malperfusion.  Unclear if cardiogenic versus hypovolemic."
Dx7Wk1 = "Pt with evidence of Tachycardia unclear if sinus vs SVT.  Will obtain EKG."
Dx7Wk2 = "Pt with tachycardia in setting of CHF.  If SVT will need to control chemically vs cardioversion."
Dx7Wk3 = "Pt noted to be both hypertensive and tachycardic.  Will need to control BP and HR."
Dx7Wk4 = "Pt noted to be both hypertensive and tachycardic in setting of Respiratory failure.  Will need to place on ventilatory support relieve anxiety or pain and then consider drugs to reduce HR and BP."
Dx7Wk5 = "Pt noted to have hypotension and tachycardia.  Will need to evaluate whether sinus or SVT."
Dx7Wk6 = "Pt noted to have tachycardia in setting of obstructive lung disease and neb use."
Dx7Wk7 = "Pt with tachycardia in setting of PE"
Dx7Wk8 = "The pt has afib with RVR"
Dx7Wk9 = "The pt is noted to have unstable afib with RVR and is hypotensive"
Dx8Wk1 = "Pt with bradycardia will need EKG to determine if heart block versus sinus brady vs junctional"
Dx8Wk2 = "Pt with evidence of bradycardia and heart failure.  Will need to increase rate to improve cardiac output and perfusion."
Dx8Wk3 = "Pt with hypertension and bradycardia, this may be reflexive versus concurrent."
Dx8Wk4 = "The pt has hypotension and bradycardia.  Etiology unclear but the differential includes ischemia, electrophysiologic disease, drug related."
Dx9Wk1 = "The pt has evidence of CHF.   Will need echo to determine whether HFpEF, HFrEF."
Dx9Wk2 = "The pt has HTN emergency with resultant CHF exacerbation.  Will need to preload and afterload reduce as well diurese patient."
Dx9Wk3 = "The pt has decompensated heart failure and evidence of cardiogenic shock.  Will need to improve cardiac output."
Dx9Wk4 = "The pt presents with cardiac arrest and evidence of cardiogenic shock."
Dx9Wk5 = "The pt has evidence of CHF with pulmonary edema and severe respiratory failure.  Will need need improve preload afterload and assist with respiration."
Dx9Wk6 = "Pt noted to have evidence of CHF in setting of liver failure.  Unclear etiology but ddx includes hepatic hydrothorax, venous congestion, aspiration."
Dx9Wk7 = "PT presents with pulmonary edema and evidence of endocarditis with possible valvular regurgitation."
Dx9Wk8 = "Pt presents with SAH and now has pulmonary edema likely from sympathetic surge."
Dx9Wk9 = "Pt with evidence of CHF as well as AKI which could represent a cardiorenal syndrome."
Dx10Wk1 = "Pt presents with HTN urgency/emergency."
Dx10Wk2 = "Pt has HTN urgency in setting of respiratory failure."
Dx10Wk3 = "PT with hypertension in setting of ICH.  Will need to reduce systolic blood pressure <160."
Dx10Wk4 = "PT with hypertension in setting of subdural hemorrhage.  Will need to reduce systolic blood pressure <160"
Dx10Wk5 = "PT with hypertension in setting of subarachnoid hemorrhage.  Will need to reduce systolic blood pressure <140 until source of bleed resolved."
Dx10Wk6 = "Pt with known space occupying lesion as well as HTN and bradycardia.  Will need to evaluate for possible cushing reflex."
Dx10Wk7 = "Pt with HTN in setting of renal dysfunction.  Will need to evaluate volume status and if overloaded diurese or consider RRT."
Dx11Wk1 = "Pt in shock the subtype of which is unclear.  The differential includes cardiogenic, distributive, hypovolemic, obstructive shock."
Dx11Wk2 = "Pt in shock the subtype of which is unclear.  The differential includes cardiogenic, distributive, hypovolemic, obstructive shock.  Because pt has evidence of PNA, this  may be pneumonia with septic shock."
Dx11Wk3 = "Pt in shock the subtype of which is unclear.  The differential includes cardiogenic, distributive, hypovolemic, obstructive shock.  Because pt has evidence of pneumothorax, this may be obstructive."
Dx11Wk4 = "Pt in shock the subtype of which is unclear.  The differential includes cardiogenic, distributive, hypovolemic, obstructive shock.  Because pt has evidence PE this may be massive PE and obstructive shock."
Dx11Wk5 = "Pt is hypotensive in setting of liver failure.  The differential includes cardiogenic, distributive, hypovolemic, obstructive shock.  Will evaluate for bleed and infection."
Dx11Wk6 = "PT has evidence of hypotension in setting of active GI bleed.  Will need to volume resuscitate and stop source of bleed."
Dx11Wk7 = "Pt in shock the subtype of which is unclear.  The differential includes cardiogenic, distributive, hypovolemic, obstructive shock.  Because pt has evidence of colitis will need to monitor for toxic megacolon."
Dx11Wk8 = "Pt presents with evidence of hypotensions in setting of bacteremia and so shock state likely septic shock."
Dx11Wk9 = "Pt in shock the subtype of which is unclear.  The differential includes cardiogenic, distributive, hypovolemic, obstructive shock.  The pt has evidence of infection and so this is possibly septic shock."
Dx12Wk1 = "Pt presents after cardiac arrest and achievement of ROSC. Unclear if PEA vs Vtach/Vfib."
Dx12Wk2 = "Pt presents after cardiac arrest and achievement of ROSC. Unclear if PEA vs Vtach/Vfib.  Pt now in cardiogenic shock as well.  Will need to begin pressor and inotrope."
Dx12Wk3 = "The pt has suffered cardiac arrest in setting of PE.  The pt may need emergent thrombolysis vs ECMO"
Dx12Wk4 = "Pt noted to have PEA arrest with bradycardia."
Dx13Wk1 = "Etiology of airway obstruction unclear, ddx includes stenosis, scarring, tumor, edema."
Dx14Wk1 = "Pt noted to have history/exam which is suggestive of COPD."
Dx14Wk2 = "Pt with evidence of ischemia and pulmonary outflow obstruction unclear if cardiac wheezing versus COPD."
Dx14Wk3 = "The pt has evidence of COPD and tachycardia likely from neb use vs acidemia."
Dx14Wk4 = "The pt has COPD and pneumonia."
Dx14Wk5 = "The pt has a history of COPD and has radiographic evidence of PTX.  It is unclear if this is related to bleb or true PTX."
Dx14Wk6 = "The pt is having increased work of breathing in setting of COPD likely exacerbation"
Dx14Wk7 = "The pt is having severely increased work of breathing in setting of COPD exacerbation requiring intubation"
Dx15Wk1 = "Pt noted to have consolidation on CXR with evidence of SIRS consistent with pneumonia"
Dx15Wk2 = "Pt with evidence of PE and PNA on radiology.  Unclear if aspiration vs pneumonia vs pulmonary infarct"
Dx15Wk3 = "PT with pneumonia in setting of liver disease, will treat and monitor for mental status changes and aspiration"
Dx15Wk4 = "The patient has pneumonia in setting of bacteremia which could be septic emboli"
Dx15Wk5 = "The patient has pneumonia in setting of endocarditis and bacteremia which could be septic emboli"
Dx16Wk1 = "Pt with evidence of pneumothorax on CXR."
Dx17Wk1 = "Pt with evidence of respiratory failure."
Dx17Wk2 = "Pt with respiratory failure and hypotension etiology unclear however ddx includes acidemia, sedation related, autopeep"
Dx17Wk3 = "The pt present with respiratory failure in setting of obstructive lung disease"
Dx17Wk4 = "The pt has evidence of acute respiratory failure with hypoxia"
Dx17Wk5 = "The patient has respiratory failure in setting of a pneumothorax.  Will need to perform needle decompression or tube thoracostomy emergently"
Dx17Wk6 = "Pt with evidence of PE and hypoxia"  
Dx17Wk7 = "The pt has respiratory failure in setting of liver disease.  Unclear etiology but ddx includes aspiration, hepatic encephalopathy, pneumonia, pulmonary edema, hepatopulmonary syndrome or portopulmonary syndrome"
Dx17Wk8 = "The patient has respiratory failure in setting of neurologic dysfunction.  Will need to protect airway"
Dx18Wk1 = "Pt with evidence of PE."
Dx18Wk2 = "Pt noted to have PE however also exhibits bleeding precluding ability to place on anticoagulation or tPA"
Dx18Wk3 = "Pt noted to have PE however also at increased risk of bleeding which precludes ability to place on anticoagulation or tPA"
Dx19Wk1 = "Pt with evidence of liver failure from likely hepatitis vs tylenol overdose vs etoh vs autoimmune."
Dx19Wk2 = "Pt with liver failure and lower extremity cellulitis likely from elevated portal vein pressures"
Dx20Wk1 = "Pt has evidence of GIB etiology of which is unclear."
Dx20Wk2 = "The pt presents with GI bleed in setting of liver dysfunction.  DDx includes variceal bleed vs PUD vs colon related."
Dx20Wk3 = "Pt has evidence of GIB as well as ACS.  Will need to transfuse however cannot give antiplatelet or heparin yet"
Dx21Wk1 = "Pt has evidence of pancreatitis etiology unclear ddx includes idiopathic vs alcohol vs gallstone vs drug related vs hypertriglyceridemia."
Dx22Wk1 = "Pt with evidence of colitits, ddx includes ischemic, infectious, inflammatory."  
Dx22Wk2 = "Pt presents with evidence of Cdiff colitis."
Dx23Wk1 = "Pt with evidence of gastroenteritis, unclear if viral vs bacterial."
Dx24Wk1 = "Pt has evidence of SBO"
Dx25Wk1 = "Pt with positive blood cultures. Source of infection is unclear at present."
Dx25Wk2 = "Pt with positive blood cultures. Source of infection is unclear at present however pt also with pneumonia and so may be pulmonary"
Dx25Wk3 = "Pt with positive blood cultures. Source of infection is unclear at present however pt with evidence of liver failure and so may be related to gut translocation versus sbp versus pneumonia"
Dx25Wk4 = "Pt with positive blood cultures. Source of infection is unclear at present however pt with GI sepsis and so that may be the source"
Dx25Wk5 = "Pt with positive blood cultures. Source of infection is unclear however patient has evidence of CNS infection.  Will need to monitor for pneumococcus versus neisseria versus listeria"
Dx25Wk6 = "Pt with positive blood cultures and evidence of vegetation on echo suggesting possible endocarditis"
Dx25Wk7 = "Pt with positive blood cultures. Source of infection is unclear at present however patient has UTI with sepsis and so that may be the source"
Dx25Wk8 = "Pt with positive blood cultures. Source of infection is unclear at present however patient has cellulitis with sepsis and so that may be the source.  Will need to check for necrotizing fasciitis"
Dx26Wk1 = "Pt with signs and symptoms of meningitis."  
Dx26Wk2 = "The patient has evidence of both meningitis and endocarditis which may represent seeding from bacteremia versus a component of Austrian's triad"
Dx27Wk1 = "Pt with evidence of endocarditis"
Dx28Wk1 = "Pt noted to have ICH on CT scan, etiology of which unclear ddx includes amyloid vs AVM vs trauma vs spontaneous"
Dx28Wk2 = "Pt noted to have ICH on CT scan, etiology of which unclear ddx includes amyloid vs AVM vs trauma vs spontaneous.  The pt also noted to be hypertensive and so will need to control."
Dx28Wk3 = "Pt noted to have evidence of ICH and SDH on CT scan, etiology of which unclear ddx includes amyloid vs AVM vs trauma vs spontaneous.  The pt also noted to be hypertensive and so will need to control"
Dx28Wk4 = "Pt noted to have evidence of ICH and SAH on CT scan, etiology of which unclear ddx includes amyloid vs AVM vs trauma vs spontaneous.  The pt also noted to be hypertensive and so will need to control"
Dx28Wk5 = "Pt noted to have ICH on CT scan complicated by seizures.  The etiology of the bleed is unclear, ddx includes amyloid vs AVM vs trauma vs spontaneous.  The will need to control seizures"
Dx28Wk6 = "Pt noted to have ICH on CT scan in setting of possible brain tumor.  Tumor likely led to bleeding.  Will need to control BP, reverse coagulopathy monitor airway."
Dx29Wk1 = "Pt noted to have SDH on CT scan, etiology of which unclear ddx includes trauma vs spontaneous"
Dx29Wk2 = "Pt noted to have SDH on CT scan, etiology of which unclear ddx includes trauma vs spontaneous.  PT with hypertension and so will need to monitor for cushings reflex as well as control pressure."
Dx29Wk3 = "Pt noted to have SDH on CT scan, etiology of which unclear ddx includes trauma vs spontaneous now complicated by seizures."
Dx29Wk4 = "Pt noted to have SDH on CT scan, etiology of which unclear ddx includes trauma vs spontaneous.  PT also noted to have evidence of brain tumors which could be reponsible for bleeding."
Dx29Wk5 = "Pt noted to have SDH on CT scan, etiology of which unclear ddx includes trauma vs spontaneous.  PT with hypertension and so will need to monitor for cushings reflex as well as control pressure.  Pt with coagulopathy which will need to be corrected."
Dx30Wk1 = "Pt found to have SAH on CT scan.  Etiology unclear however ddx includes aneurysmal versus trauma."
Dx30Wk2 = "Pt found to have SAH on CT scan.  Etiology unclear however ddx includes aneurysmal versus trauma.  Pt course being complicated by seizures"
Dx31Wk1 = "Pt with evidence of seizures.  Cause of seizures is unclear ddx includes med noncompliance, toxic metabolic disturbance, brain tumor, ingestion, fatigue, CVA."
Dx32Wk1 = "Pt with evidence of brain tumor on imaging"
Dx33Wk1 = "Pt with evidence of UTI with sepsis"
Dx34Wk1 = "Pt noted to have signs of soft tissue infection"
Dx34Wk2 = "Pt noted to have signs of soft tissue infection. Pt noted to have signs of soft tissue infection in setting of delirium, will need to evaluate for possible necrotizing fasciitis. "
Dx35Wk1 = "Pt found to have infectious colitis on cdiff assay."
Dx36Wk1 = "Pt with history of DM."
Dx37Wk1 = "Pt presents with AKI, etiology of which is unclear.  DDx includes prerenal vs post renal vs intrinsic."
Dx38Wk1 = "Pt noted to have elevated FS and AG consistent with DKA."
Dx39Wk1 = "Pt noted to be anemic, etiology of which is unclear ddx includes macro vs micro vs normocytic." 
Dx40Wk1 = "Pt with low glucose level"
Dx41Wk1 = "Pt noted to have elevated sodium for unclear reasons.  DDx includes dietary vs diabetes insipidus vs diuretic related"
Dx42Wk1 = "Etiology of hyponatremia unclear.  Ddx includes hypovolemic vs euvolemic vs hypervolemic"
Dx43Wk1 = "Pt with hyperkalemia etiology unclear ddx includes renal failure vs acidemia vs drug related vs rhabdo vs tumor lysis"
Dx44Wk1 = "Pt with hypokalemia"
Dx45Wk1 = "Pt noted to have coagulopathy"
Dx45Wk2 = "Pt noted to have coagulopathy and bleeding"
Dx46Wk1 = "Pt HD stable monitor vitals"
Dx47Wk1 = "Promote IS use, keep o2 sat >92%"
Dx48Wk1 = "Advance diet when ok with primary" 
Dx49Wk1 = "Monitor for AKI and replete lytes"
Dx50Wk1 = "Monitor for delirium"
Dx51Wk1 = "Monitor for bleeding and blood clots.  Transfuse for hgb < 7 and platelets <10"
Dx52Wk1 = "Maintain FS 150-180"
Dx53Wk1 = "Monitor for infection"
Dx57Wk1 = "The pt is noted to be on ventilator for respiratory support and airway protection"
Dx57Wk2 = "The pt is now intubated in the setting of CHF exacerbation.  Will need positive pressure to reduce afterload"
Dx57Wk3 = "The pt is intubated after episode for cardiac arrest and ROSC"
Dx57Wk4 = "The pt is intubated in setting of COPD, PNA and respiratory failure"
Dx57Wk5 = "Will need lung protective ventilation, minimize O2, permissive hypercapnia, TV 6cc/kg, high PEEP."
Dx57Wk6 = "The pt has evidence of COPD exacerbation and respiratory failure in the setting of PNA"
Dx58Wk1 = "The pt is found to have afib unclear if paroxysmal or persistent."
Dx58Wk2 = "The pt is found to have afib unclear if paroxysmal or persistent in setting of bleeding risk."
Dx59Wk1 = "The pt is found to have an anion gap acidosis of unclear cause.  The differential is elevated levels of methanol, urea, keto acidosis, starvation ketosis, lactic acidosis, ethanol, isoniazid, salicylates or possible paraldehyde"
Dx59Wk2 = "The pt is found to have DKA.  The possible causes include med noncompliance, infection, new diagnosis, worsening insulin production"
Dx59Wk3 = "The pt is noted to have a negative anion gap.  DDx includes possible hypoalbuminemia vs paraproteinemia including MGUS, multiple myeloma vs high cation state such as Li+ Ca++, Mg++ vs lab error"
Dx60Wk1 = "The pt has history of ESRD"
Dx61Wk1 = "The pt has history of CKD"
Dx62Wk1 = "The pt has a history of HTN"
Dx62Wk2 = "The pt has a history of HTN however the BP is low at present"
Dx63Wk1 = "The pt has history of asthma"
Dx63Wk2 = "The Pt noted to have asthma exacerbation and increased work of breathing likely"
Dx64Wk1 = "The pt has a EtOH use and now with tremor and signs of alcohol withdrawal"
Dx64Wk2 = "The pt has a EtOH use and now with signs of severe withdrawal requiring intubation."
Dx65Wk1 = "The pt has a history of CAD."
Dx65Wk2 = "The pt has a history of CAD in setting of bleeding risk"
Dx66Wk1 = "The pt has a history of OSA."
Dx67Wk1 = "The pt has a history of hypothyroid."
Dx68Wk1 = "The pt has a history of hyperthyroidism."
Dx69Wk1 = "The pt has evidence of ischemic stroke.  Neuro deficit as is noted"
Dx69Wk2 = "The pt has evidence of ischemic stroke.  Neuro deficit as is noted.  The pt is noted to have coagulopathy."
Dx69Wk3 = "The pt has evidence of ischemic stroke.  Neuro deficit as is noted.  The pt is noted to also have bleeding."
Dx69Wk4 = "The pt has evidence of ischemic stroke. Neuro deficit as is noted. The pt is noted to also have brain tumor."
Dx70Wk1 = "The pt has evidence of delirium."
Dx71Wk1 = "The pt has a history of dyslipidemia"
Dx72Wk1 = "The pt has a history of lupus."
Dx72Wk2 = "The pt has history of lupus now with HTN emergency, will need to monitor for possible lupus flare."
Dx72Wk3 = "The pt has hx of respiratory failure and lupus, will need to monitor for pneumonitis vs serositis."
Dx72Wk4 = "The pt has lupus and now encephalitis. Will need to monitor of lupus cerebritis."
Dx72Wk5 = "The pt has hx of lupus now with AKI, will need to evaluate for lupus nephritis."
Dx72Wk6 = "The pt has lupus and now seizures. Will need to monitor of lupus cerebritis."
Dx73Wk1 = "The pt has a hx of sclerderma and signs of scleroderma"
Dx74Wk1 = "The pt has a history of pericardial effusion unclear etiology ddx includes infectious, rheumatologic, malignant or idiopathic."
Dx74Wk2 = "The pt has pericardial effusion in setting of AAA. Will need to monitor for extension of dissection."
Dx74Wk3 = "The pt has pericardial effusion in setting of shock state. The pt is high risk of having pericardial tamponade."
Dx74Wk4 = "The pt has a history of pericardial effusion unclear etiology ddx includes infectious, rheumatologic, malignant or idiopathic. Pt with renal failure which might explain pericardial effusion."
Dx74Wk5 = "The pt has a history of pericardial effusion unclear etiology ddx includes infectious, rheumatologic, malignant or idiopathic. Pt with lupus which might explain pericardial effusion."
Dx75Wk1 = "The pt has evidence of cholecystitis. The cause is unclear but ddx includes stones vs tumor vs stricture"
Dx76Wk1 = "The pt is showings signs of adrenal insufficiency.  It is unclear if primary or secondary"
Dx77Wk1 = "The pt has signs and symptoms of hypercortisolemia.  It is unclear whether primary or secondary"
Dx78Wk1 = "The pt is noted to have diffuse non-cardiac consolidation with P/F <300 consistent with ARDS"
Dx79Wk1 = "The pt has recent head trauma"
Dx80Wk1 = "The pt has burns will need to evaluate thickness fraction of BSA affected."
Dx81Wk1 = "The pt has a history of peripheral vascular disease."
Dx82Wk1 = "The pt has a evidence of brain mass."
Dx83Wk1 = "The pt has a history of GERD"
hist2=  "The pt has a history of MI."
hist3=  "The pt has a history of AAA."
hist6   ="The pt has a history of CHF."
hist11= "The pt has a history of pneumonia in the past."
hist12= "The pt has a history of COPD."
hist13= "The pt has had pneumothorax in the past."
hist15= "The pt has a hx of cirrhosis."
hist16= "The pt has had GIB in the past."
hist17= "The pt has had pancreatitis in the past."
hist18= "The pt has a history of colitis."
hist21= "The pt has a history of endocarditis."
hist22= "The pt has had previous UTI."
hist23= "The pt has a history of being vented"
hist24= "The pt has a history of afib."
hist30= "The pt has a history of DM."
hist33= "The pt has a history of PE."
hist34= "The pt has a history of ICH."
hist35= "The pt has a history of SDH."
hist36= "The pt has a history of SAH."
hist37= "The pt has a history of Seizures."
hist41= "The pt has history of asthma"
hist43= "The pt has had etoh withdrawal in the past"
hist48= "The pt has a history of CVA"
hist52= "The pt has a history of lupus"
hist54  ="The pt has a history of scleroderma"
hist55= "The pt has a history of pericardial effusion"
hist58= "The pt has a history of cellulitis"
hist59= "The pt has a history of TBI"
hist61= "The pt has a history of PAD"
hist62= "The pt has a history of brain tumor"
hist74= "The pt has chronic pleural effusions"
hist75= "The pt has a history of trach"

#history library
Hemicolectomy = "hemicolectomy"
Colon_cancer = ['colon ca', 'colon cancer']
ACS = ['acs' ,'stemi', "mi", "nstemi", "stemi", "heart attack", "myocardial infarction"]
AAA = ["abdominal aortic aneurysm", "triple a", "aaa", "aortic aneurysm", "abd aorta enlarged", "enlarged aorta"]
Tachycardia = ["tachycardia", "tachycardia", "tachycardic", "svt", "afib with rvr", "vtach", "ventricular tachycardia"]
Bradycardia = ["bradycardia", "heart block", "wenckebach"]
CHF = ["chf", "hfref", "hfpef", "cardiomyopathy", "congestive heart failure", "heart failure", "cardiac failure", "low ejection fraction", "low ef", "congestive heart"]
HTN_Emergency = ["malignant hypertension", "htn emergency", "htn crisis", "htn crisis" "hypertensive urgency", "hypertensive emergency", "hypertensive crisis"]
HypotensionShock = ["hypotension", "hypotension", "hypotensive", "shock", "malperfusion", "hypoperfuse", "hypoperfusion"]
Cardiac_arrest =["cardiac arrest", "cardiopulmonary arrest", "cardiac death"]
Airway_Obstruction = [ "airway obstruction", "tracheal stenosis", "airway stenosis", "obstruction", "laryngeal edema", "laryngeal mass", "vocal chord edema", "vocal chord swelling"]
COPD = ["copd", "copd", "chronic obstructive pulmonary disease", "chronic obstructive lung disease"]
Pneumonia = ["pna", "pneumonia", "cap", "hcap", "vap", "lung infection", "pulmonary infection", "lung infiltrate", "pulmonary infiltrate", "pulmonary consolidation"]
Pneumothorax = ["collapsed lung", "air around lung", "ptx", "ptx", "pneumothorax", "pneumothoraces"]
Respiratory_Failure = ["respiratory failure", "respiratory arrest", "respiratory failure","respiratory arrest","respiratory distress","resp failure","resp arrest", "resp distress", "pulmonary distress"]
PE = ["pe", "pulmonary embolus", "pulmonary embolism", "pulmonary clots", "saddle pe", "saddle clot", "saddle embolus"]
ALF = ["alcoholic liver cirrhosis", "liver failure", "liver cirrhosis", "hepatitis", "cirrhosis", "ascites", "cirrhotic", "varices", "hepatitis", "transaminitis"]
GI_Bleed = ["gi bleed", "gastrointestinal bleed", "gi bleed", "gi bleeding", "variceal bleed", "gastric bleed", "varcieal oozing", "intestinal bleeding", "gib", "hematchezia", "melena", "vomiting blood", "bright red blood per rectum", "brbpr"]
Pancreatitis = ["inflamed pancreas", "elevated lipase", "pancreatic inflammation", "pancreatitis"]
Colitis = ["colitis", "cdiff", "bowel infection", "gut inflammation", "bowel inflammation", "inflamed bowel"]
Gastroenteritis = ["gastroenteritis", "gastroenteritis", "enteritis"]
Bacteremia = ["blood infection", "positive blood culture", "gram negatives in blood", "+ blood culture", "bacteremia", "septicemia"]
meningitis_encephalitis = ["meningitis", "nuchal rigidity", "cns infection", "encephalitis"]
Endocarditis = ["endocarditis", "vegetation", "septic emboli"]
ICH = ["ich", "ivh", "iph", "intracranial hemorrhage", "intracranial hemorrhaging", "intracranial bleed", "intracranial hematoma", "brain hemorrhage", "intracerebral bleed", "brain bleed", "cerebral hemorrhage", "intraventricular hemorrhage"]
SDH = ["sdh", "subdural hemorrhage", "subdural hemorrhaging", "subdural bleed", "subdural hematoma"]
SAH = ["sah", "subarachnoid hemorrhage", "subarachnoid hemorrhaging", "subarachnoid bleed", "subarachnoid hematoma"]
Seizures = ["seizure", "epilepsy", "convulsion", "pseudoseizure", "seizures", "convulsions", "pseudoseizures"]
UTI = ["urinary tract infection", "positive urine", "uti", "uti", "dysuria"]
Cellulitis = ["cellulitis" "skin infection", "skin inflammation", "nec fasc", "necrotizing fasciitis", "cellulitic"]
DM = ["diabetes mellitus", "DMT2", "dm", "dm", "diabetes", "iddm", "dm1", "dm2", "iddm1"]
AKI = ["aki", "kidney injury", "acute kidney injury", "acute renal injury", "acute kidney failure"]
DKA = ["diabetic ketoacidosis", "diabetic ketosis", "ketoacidosis", "ketoacidotic", "ketosis", "dka"]
Afib = ["atrial fib", "tachycardia", "afib", "a fib", "a-fib", "atrial fibrillation"]
ESRD = ["esrd", "end-stage renal disease", "end stage kidney disease"]
CKD = ["ckd", "chronic kidney disease", "chronic renal disease", "chronic renal dysfunction" "kidney disease"]
HTN = ["hypertension", "htn", "hipertension", "hypertensive", "high blood pressure", "elevated blood pressure"] 
Asthma = ["asthma", "asthmatic"]
EtOH_WD = ["alcohol withdrawal", "alcohol hallucinosis", "etoh wd", "etoh withdrawal",  "alcoholic", "delirium tremens"]
CAD = ["coronary artery disease", "coronary disease", "coronary dz", "heart disease", "coronary vessel disease"]
OSA = ["osa", "obstructive sleep apnea", "sleep apnea"]
Hypothyroid = ["hypothyroidism", "hypothyroid", "myxedema", "hypoactive thyroid", "myxedema coma"]
Hyperthyroid = ["hyperactive thyroid", "hyperthyroidism", "hyperthyroidism", "hyperthyroid", "graves"]
CVA = ["cva", "stroke", "ischemic stroke", "mca", "pca", "aca", "cerebrovascular accident"]
Delirium = ["delirium", "confusion", "sundowning", "sundown", "sundowning", "confused", "delirious", "encephalopathy", "encephalopathic"]
DLD = ["hyperlipidemia", "hld", "dld", "dyslipidemia", "dyslipidemic", "hyperlipidemic", "hypertriglyceridemia", "high cortisol"]
Lupus = ["lupus", "sle", "systemic lupus erythematosus", "lupus erythematosus", "systemic lupus"]
Scleroderma = ["systemic sclerosis", "scleroderma", "crest", "sclerodactyly"]
Pericardial_Effusion = ["hemopericardium", "pneumopericardium", "pericardial effusion", "pericardial effusion", "pericardial tamponade", "pericardial fluid"]
Cholecystitis = ["cholecystitis", "gallbladder inflammation", "necrotic gallbladder", "gallbladder edema"]
AI = ["adrenal insufficiency", "ai", "low cortisol", "cushings disease", "hypocortisolemia", "hypoaldosteronism", "hypoaldosterone"]
Cushing = ["cushings", "hypercortisolemia", "hyperaldosteronism", "hyperaldosterone", "cushingoid", "cushing"]
ARDS = ["ards", "acute respiratory distress syndrome", "respiratory distress syndrome"]
TBI = ["traumatic brain injury", "brain injury", "brain trauma", "tbi"]
Burns = ["burn", "burns", "thermal injury"]
PAD = ["pvd", "pad", "peripheral artery disease", "peripheral vascular disease", "peripheral arterial disease"]
Brain_Tumor = ["gbm", "glioblastoma", "astrocytoma", "brain tumor", "brain mass", "glioblastoma", "cerebral met", "brain met", "brain metastasis"]
GERD = ["gerd", "gastroesophageal reflux disease", "gastroesophageal reflux", "gastric reflux", "reflux disease"]     
pleural_effusion = ["hemothorax", "pleural effusion", "pleural fluid", "lung edema", "complicated effusion", "loculated effusion"]
vented = ["intubated", "vented" "ventilator use"]
trach = ["trach", "tracheostomy", "trached"]
extremity_angiogram = ["extremity balloon angioplasty", "extremity_angiogram", 'rue angiogram', 'rle angiogram', 'lle angiogram', 'lue angiogram', "extremity balloon angiogram", "extremity_angio", 'rue angio', 'rle angio', 'lle angio', 'lue angio']
lung_transplant = ["transplantation lung", "lung_transplant", "transplantation lung", "transplant lung"]
pericardial_window = ['pericardial window']
coronary_angiogram = ['coronary angiogram', 'rhc', 'lhc', 'coronary angiogram', 'heart angiogram', 'coronary angio', 'heart angio', 'coronary cath', 'heart cath', 'coronary catheterization', 'heart catheterization']
maze = ["maze", "mini maze"]
lvad = ['lvad', 'left ventricular assist device']
mvr = ['mvr', 'mitral valve repair', 'mitral valve replacement', 'mitral valve replaced', 'mitral valvulopathy repair', 'mitral valvulopathy replacement', 'mitral valvulopathy replaced']
avr = ['avr', 'aortic valve repair', 'aortic valve replacement', 'aortic valve replaced', 'aortic valvulopathy repair', 'aortic valvulopathy replacement', 'aortic valvulopathy replaced']
tvr = ['tvr', 'tricuspid valve repair', 'tricuspid valve replacement', 'tricuspid valve replaced', 'tricuspid valvulopathy repair', 'tricuspid valvulopathy replacement', 'tricuspid valvulopathy replaced']
cabg = ["cabg", "cab", "cabg x1", "cabg x2", "cabg x3", "cabg x4", "coronary artery bypass graft", "coronary artery bypass grafting", "coronary artery bypass"]
impella = "impella"
aaa = ["abdominal aortic aneurysm", "aaa", "triple a", "aortic aneurysm"] 
oht = ["oht", "orthotopic heart transplant", "heart transplant", "cardiac transplantation"]
aortic_dissection = ["aortic dissection", 'type 1 dissection', 'type 2 dissection'] 
craniotomy = ["craniotomy", "craniectomy"]
kyphoplasty = "kyphoplasty"
ivorlewis = ["ivor-lewis esophagectomy", "ivor lewis esophagectomy", "ivor lewis", "ivorlewis", "ivor-lewis"]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.2/110.2 kB 5.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.4/67.4 kB 5.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.1/71.1 kB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.0/69.0 kB 7.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 235.9/235.9 kB 9.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 575.5/575.5 kB 34.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.7/311.7 kB 16.9 MB/s eta 0:00:00
ERROR: Operation cancelled by user
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/pip/_internal/cli/base_command.py", line 160, in exc_logging_wrapper
    status = run_func(*args)
  File "/usr/local/lib/python3.10/dist-packages/pip/_internal/cli/req_command.py", line 241, in w

In [ ]:
#@markdown Enter Fields
Note = " 69 M\xA0with extensive CAD, CABG, and multiple stents presents to the emergency room for chest pain\xA0after PCI this morning by Dr. Brown. \xA0 Patient was diagnosed with CAD in 2005.\xA0\xA0In 2005, he underwent four-vessel bypass by Dr. Vanhooser.\xA0\xA0Since, patient has had multiple coronary stents. \xA0 Today, patient was admitted likely for coronary angiogram.\xA0\xA0Patient received\xA0a 3.5x12 DES to SVG to distal RCA. \xA0Patient denied any chest pain after the procedure.\xA0\xA0At 4:00\xA0PM, he\xA0was charged to home.\xA0\xA0At home, approximately 4:30 PM,\xA0he began to experience severe substernal chest pain rating down his left arm and epigastric area.\xA0\xA010/10.\xA0\xA0According his wife, patient was hunched over in severe pain. \xA0He was also short of breath. \xA0 Patient returned to the emergency room.\xA0\xA0In the ER, BP was low. \xA0No NTG was given. \xA0Patient was given IV fentanyl.\xA0\xA0At the time of evaluation, patient's chest pain is 3/10. \xA0His shortness of breath improved. \xA0 Cath film reviewed: Occluded proximal RCA. \xA0Occluded distal RCA. \xA0Patent 1/4 bypasses. \xA0DES to distal SVG-RCA which supplies retrograde to mid RCA. \xA0No flow to PDA or PL. \xA0 \xA0 EKG 1514: Afib. No ST elevation in V1. \xA0 EKG: 1716: Afib. 1-2 mm ST elevation V1. \xA01 mm V2. \xA0 Echo: 5/1/2023 1. \xA0Nondilated left ventricle with mild concentric left ventricular hypertrophy. \xA0Systolic function is mildly decreased with an ejection fraction of 40 to 45%. \xA0There is mild global hypokinesia 2. \xA0Normal right ventricular size and function 3. \xA0Moderate biatrial dilatation 4. \xA0Mild to moderate mitral regurgitation 5. \xA0No evidence of elevated pulmonary artery pressures 6. \xA0No pericardial disease identified Objective data  Visit Vitals BP\t(!) 86/76 Pulse\t(!) 104 Temp\t36.8 \xB0C (98.3 \xB0F) (Temporal ) Resp\t21 Ht\t182.9 cm (6' 0.01\") Wt\t96.3 kg (212 lb 4.9 oz) SpO2\t97% BMI\t28.79 kg/m\xB2    Intake/Output Summary (Last 24 hours) at 5/2/2023 0128 Last data filed at 5/1/2023 2100 Gross per 24 hour Intake\t\u2014 Output\t300 ml Net\t-300 ml    Lab Results Lab\tValue\tDate/Time \tSODIUM\t139\t05/01/2023 1736 \tSODIUM\t140\t07/23/2020 0915 \tPOTASSIUM\t4.5\t05/01/2023 1736 \tPOTASSIUM\t4.5\t07/23/2020 0915 \tINR\t1.1\t05/01/2023 1933 \tPTT\t23.9\t05/01/2023 0609  " #@param {type:"string"}
Op_Note = 'ccc' #@param {type:"string"}
Na = None #@param {type:"number"}
K = None #@param {type:"number"}
AG = None #@param {type:"number"}
Cr = None #@param {type:"number"}
Hgb = None #@param {type:"number"}
Plt = None #@param {type:"number"}
INR = None #@param {type:"number"}
aPTT = None #@param {type:"number"}
SBP  = None #@param {type:"number"}
Intubated = 0 #@param {type:"integer"}
Narrative = 'n' #@param {type:"string"}

#screen grabber values
doc2 = nlp2(Note)


warnings.filterwarnings('ignore')
zp = pd.DataFrame(doc2._.context_graph.edges)

warnings.filterwarnings('ignore')
if len(zp) > 0:
  zp.columns=['column1', 'column2']
  zp['column1'] = zp['column1'].astype(str)
  zp['column2'] = zp['column2'].astype(str)
  NaSodium = zp.loc[zp['column2'].str.contains("NA")]
  NaSodium['column1'] = NaSodium['column1'].astype(float)
  NaSodium = list(NaSodium['column1'])
  KPotassium = zp.loc[zp['column2'].str.contains("K")]
  KPotassium['column1'] = KPotassium['column1'].astype(float)
  KPotassium = list(KPotassium['column1'])
  CrCreatinine = zp.loc[zp['column2'].str.contains("CREATININE")]
  CrCreatinine['column1'] = CrCreatinine['column1'].astype(float)
  CrCreatinine = list(CrCreatinine['column1'])
  BiliBilirubin = zp.loc[zp['column2'].str.contains("BILIRUBIN")]
  BiliBilirubin['column1'] = BiliBilirubin['column1'].astype(float)
  BiliBilirubin = list(BiliBilirubin['column1'])
  ClChloride = zp.loc[zp['column2'].str.contains("CHLORIDE")]
  ClChloride['column1'] = ClChloride['column1'].astype(float)
  ClChloride = list(ClChloride['column1'])
  CO2co2 = zp.loc[zp['column2'].str.contains("CO2")]
  CO2co2['column1'] = CO2co2['column1'].astype(float)
  CO2co2 = list(CO2co2['column1'])
  GlGlucose = zp.loc[zp['column2'].str.contains("GLUCOSE")]
  GlGlucose['column1'] = GlGlucose['column1'].astype(float)
  GlGlucose = list(GlGlucose['column1'])
  AlbAlbumin = zp.loc[zp['column2'].str.contains("ALBUMIN")]
  AlbAlbumin['column1'] = AlbAlbumin['column1'].astype(float)
  AlbAlbumin = list(AlbAlbumin['column1'])
  PrProtein = zp.loc[zp['column2'].str.contains("PROTEIN")]
  PrProtein['column1'] = PrProtein['column1'].astype(float)
  PrProtein = list(PrProtein['column1'])
  AstAST = zp.loc[zp['column2'].str.contains("AST")]
  AstAST['column1'] = AstAST['column1'].astype(float)
  AstAST = list(AstAST['column1'])
  AltALT = zp.loc[zp['column2'].str.contains("ALT")]
  AltALT['column1'] = AltALT['column1'].astype(float)
  AltALT = list(AltALT['column1'])
  wbcWBC = zp.loc[zp['column2'].str.contains("WBC")]
  wbcWBC['column1'] = wbcWBC['column1'].astype(float)
  wbcWBC = list(wbcWBC['column1'])
  hgbHGB = zp.loc[zp['column2'].str.contains("HEMOGLOBIN]")]
  hgbHGB['column1'] = hgbHGB['column1'].astype(float)
  hgbHGB = list(hgbHGB['column1'])
  pltPLT = zp.loc[zp['column2'].str.contains("PLATELET")]
  pltPLT['column1'] = pltPLT['column1'].astype(float)
  pltPLT = list(pltPLT['column1'])
  TTemp = zp.loc[zp['column2'].str.contains("TEMPERATURE")]
  TTemp['column1'] = TTemp['column1'].astype(float)
  TTemp = list(TTemp['column1'])
  bpBP = zp.loc[zp['column2'].str.contains("BP")]
  bpBP['column1'] = bpBP['column1'].astype(float)
  bpBP = list(bpBP['column1'])
  sbpSBP = zp.loc[zp['column2'].str.contains("SYSTOLIC")]
  sbpSBP['column1'] = sbpSBP['column1'].astype(float)
  sbpSBP = list(sbpSBP['column1'])
  dbpDBP = zp.loc[zp['column2'].str.contains("DIASTOLIC")]
  dbpDBP['column1'] = dbpDBP['column1'].astype(float)
  dbpDBP = list(dbpDBP['column1'])
  mapMAP = zp.loc[zp['column2'].str.contains("MAP")]
  mapMAP['column1'] = mapMAP['column1'].astype(float)
  mapMAP = list(mapMAP['column1'])
  HRHeartrate = zp.loc[zp['column2'].str.contains("HEART RATE")]
  HRHeartrate['column1'] = HRHeartrate['column1'].astype(float)
  HRHeartrate = list(HRHeartrate['column1'])
  O2satSPO2 = zp.loc[zp['column2'].str.contains("O2 SAT")]
  O2satSPO2['column1'] = O2satSPO2['column1'].astype(float)
  O2satSPO2 = list(O2satSPO2['column1'])
  RRResp = zp.loc[zp['column2'].str.contains("RESPIRATORY RATE")]
  RRResp['column1'] = RRResp['column1'].astype(float)
  RRResp = list(RRResp['column1'])
  aPTTptt = zp.loc[zp['column2'].str.contains("PTT")]
  aPTTptt['column1'] = aPTTptt['column1'].astype(float)
  aPTTptt = list(aPTTptt['column1'])
  PTpt = zp.loc[zp['column2'].str.contains("PT")]
  PTpt['column1'] = PTpt['column1'].astype(float)
  PTpt = list(PTpt['column1'])
  inrINR = zp.loc[zp['column2'].str.contains("INR")]
  inrINR['column1'] = inrINR['column1'].astype(float)
  inrINR = list(inrINR['column1'])
  sysef = zp.loc[zp['column2'].str.contains("SYSEF")]
  sysef['column1'] = sysef['column1'].astype(float)
  sysef = list(sysef['column1'])
  kuL = zp.loc[zp['column2'].str.contains("KUL")]
  kuL['column1'] = kuL['column1'].astype(float)
  kuL = list(kuL['column1'])
  KPotassium = [elem for elem in KPotassium if elem not in kuL]
  sbpSBP = [elem for elem in sbpSBP if elem not in sysef]

  #harmonizing variable sources: this filters impossible variables and allows input of variables automatically and manually without error
  NaSodiummod = [num for num in NaSodium if num > 99 and num < 200]
  KPotassiummod = [num for num in KPotassium if num > 1.5 and num < 10]
  CrCreatininemod = [num for num in CrCreatinine if num > .2 and num < 20]
  BiliBilirubinmod = [num for num in BiliBilirubin if num > .2 and num < 50]
  ClChloridemod = [num for num in ClChloride if num > 65 and num < 130]
  CO2co2mod = [num for num in CO2co2 if num > 6 and num < 50]
  GlGlucosemod = [num for num in GlGlucose if num > 6 and num < 2000]
  AlbAlbuminmod = [num for num in AlbAlbumin if num > .6 and num < 6]
  PrProteinmod = [num for num in PrProtein if num > 1 and num < 11]
  AstASTmod = [num for num in AstAST if num > 6 and num < 2500]
  AltALTmod = [num for num in AltALT if num > 6 and num < 2500]
  wbcWBCmod = [num for num in wbcWBC if num > 0 and num < 200]
  hgbHGBmod = [num for num in hgbHGB if num > 0 and num < 20]
  pltPLTmod = [num for num in pltPLT if num >= 1 and num < 1000]
  TTempmod = [num for num in TTemp if num > 32 and num < 108]
  bpBPmod = [num for num in bpBP if num > 32 and num < 300]
  sbpSBPmod = [num for num in sbpSBP if num > 32 and num < 300]
  dbpDBPmod = [num for num in dbpDBP if num > 32 and num < 300]
  mapMAPmod = [num for num in mapMAP if num > 32 and num < 180]
  HRHeartratemod = [num for num in HRHeartrate if num > 12 and num < 280]
  O2satSPO2mod = [num for num in O2satSPO2 if num > 22 and num < 100]
  RRRespmod = [num for num in RRResp if num > 3 and num < 44]
  aPTTpttmod = [num for num in aPTTptt if num > 8 and num < 300]
  PTptmod = [num for num in PTpt if num > 5 and num < 45]
  inrINRmod = [num for num in inrINR if num > .5 and num < 18]
  def get_number_farthest_from_stdev(num):
    if len(num) > 0:
      mean = statistics.mean(num)
      stdev = statistics.stdev(num)
      diffs = [abs(num - mean) for num in num]
      farthest_index = diffs.index(max(diffs))
      return num[farthest_index]
    else:
      num = num      
  if len(NaSodiummod) > 1:
    NaSodiumvec = get_number_farthest_from_stdev(NaSodiummod)
  elif len(NaSodiummod) == 1:
    NaSodiumvec = float(NaSodiummod[0])
  elif len(NaSodiummod) == 0:
    NaSodiumvec = 140
  if len(KPotassiummod) > 1:
    KPotassiumvec = get_number_farthest_from_stdev(KPotassiummod)
  else:
    if len(KPotassiummod) == 1:
      KPotassiumvec = float(KPotassiummod[0])
    else:
      KPotassiumvec = 4
  if len(CrCreatininemod) > 1:
    CrCreatininevec = get_number_farthest_from_stdev(CrCreatininemod)
  else:
    if len(CrCreatininemod) == 1:
      CrCreatininevec = float(CrCreatininemod[0])
    else:
      CrCreatininevec = 1
  if len(ClChloridemod) > 1:
    ClChloridevec = get_number_farthest_from_stdev(ClChloridemod)
  else:
    if len(ClChloridemod) == 1:
      ClChloridevec = float(ClChloridemod[0])
    else:
      ClChloridevec = None
  if len(CO2co2mod) > 1:
    CO2co2vec = get_number_farthest_from_stdev(CO2co2mod)
  else:
    if len(CO2co2mod) == 1:
      CO2co2vec = float(CO2co2mod[0])
    else:
      CO2co2vec = None
  if len(GlGlucosemod) > 1:
    GlGlucosevec = get_number_farthest_from_stdev(GlGlucosemod)
  else:
    if len(GlGlucosemod) == 1:
      GlGlucosevec = float(GlGlucosemod[0])
    else:
      GlGlucosevec = None
  if len(BiliBilirubinmod) > 1:
    BiliBilirubinvec = get_number_farthest_from_stdev(BiliBilirubinmod)
  else:
    if len(BiliBilirubinmod) == 1:
      BiliBilirubinvec = float(BiliBilirubinmod[0])
    else:
      BiliBilirubinvec = None
  if len(AlbAlbuminmod) > 1:
    AlbAlbuminvec = get_number_farthest_from_stdev(AlbAlbuminmod)
  else:
    if len(AlbAlbuminmod) == 1:
      AlbAlbuminvec = float(AlbAlbuminmod[0])
    else:
      AlbAlbuminvec = None
  if len(PrProteinmod) > 1:
    PrProteinvec = get_number_farthest_from_stdev(PrProteinmod)
  else:
    if len(PrProteinmod) == 1:
      PrProteinvec = float(PrProteinmod[0])
    else:
      PrProteinvec = None
  if len(AstASTmod) > 1:
    AstASTvec = get_number_farthest_from_stdev(AstASTmod)
  else:
    if len(AstASTmod) == 1:
      AstASTvec = float(AstASTmod[0])
    else:
      AstASTvec = None
  if len(AltALTmod) > 1:
    AltALTvec = get_number_farthest_from_stdev(AltALTmod)
  else:
    if len(AltALTmod) == 1:
      AltALTvec = float(AltALTmod[0])
    else:
      AltALTvec = None
  if len(wbcWBCmod) > 1:
    wbcWBCvec = get_number_farthest_from_stdev(wbcWBCmod)
  else:
    if len(wbcWBCmod) == 1:
      wbcWBCvec = float(wbcWBCmod[0])
    else:
      wbcWBCvec = None
  if len(hgbHGBmod) > 1:
    hgbHGBvec = get_number_farthest_from_stdev(hgbHGBmod)
  else:
    if len(hgbHGBmod) == 1:
      hgbHGBvec = float(hgbHGBmod[0])
    else:
      hgbHGBvec = 12
  if len(pltPLTmod) > 1:
    pltPLTvec = get_number_farthest_from_stdev(pltPLTmod)
  else:
    if len(pltPLTmod) == 1:
      pltPLTvec = float(pltPLTmod[0])
    else:
      pltPLTvec = 300
  if len(TTempmod) > 1:
    TTempvec = get_number_farthest_from_stdev(TTempmod)
  else:
    if len(TTempmod) == 1:
      TTempvec = float(TTempmod[0])
    else:
      TTempvec = None
  if len(bpBPmod) > 1:
    bpBPvec = get_number_farthest_from_stdev(bpBPmod)
  else:
    if len(bpBPmod) == 1:
      bpBPvec = float(bpBPmod[0])
    else:
      bpBPvec = None
  if len(sbpSBPmod) > 1:
    sbpSBPvec = get_number_farthest_from_stdev(sbpSBPmod)
  else:
    if len(sbpSBPmod) == 1:
      sbpSBPvec = float(sbpSBPmod[0])
    else:
      sbpSBPvec = 120
  if len(dbpDBPmod) > 1:
    dbpDBPvec = get_number_farthest_from_stdev(dbpDBPmod)
  else:
    if len(dbpDBPmod) == 1:
      dbpDBPvec = float(dbpDBPmod[0])
    else:
      dbpDBPvec = None
  if len(mapMAPmod) > 1:
    mapMAPvec = get_number_farthest_from_stdev(mapMAPmod)
  else:
    if len(mapMAPmod) == 1:
      mapMAPvec = float(mapMAPmod[0])
    else:
      mapMAPvec = None
  if len(HRHeartratemod) > 1:
    HRHeartratevec = get_number_farthest_from_stdev(HRHeartratemod)
  else:
    if len(HRHeartratemod) == 1:
      HRHeartratevec = float(HRHeartratemod[0])
    else:
      HRHeartratevec = None
  if len(RRRespmod) > 1:
    RRRespvec = get_number_farthest_from_stdev(RRRespmod)
  else:
    if len(RRRespmod) == 1:
      RRRespvec = float(RRRespmod[0])
    else:
      RRRespvec = None
  if len(O2satSPO2mod) > 1:
    O2satSPO2vec = get_number_farthest_from_stdev(O2satSPO2mod)
  else:
    if len(O2satSPO2mod) == 1:
      O2satSPO2vec = float(O2satSPO2mod[0])
    else:
      O2satSPO2vec = None
  if (len(NaSodiummod) > .1 and len(ClChloridemod) > .1 and len(CO2co2mod) > .1):
    agAG = (NaSodiumvec-ClChloridevec-CO2co2vec)
  else:
    agAG = 10
  if len(aPTTpttmod) > 1:
    aPTTpttvec = get_number_farthest_from_stdev(aPTTpttmod)
  else:
    if len(aPTTpttmod) == 1:
      aPTTpttvec = float(aPTTpttmod[0])
    else:
      aPTTpttvec = 30
  if len(PTptmod) > 1:
    PTptvec = get_number_farthest_from_stdev(PTptmod)
  else:
    if len(PTptmod) == 1:
      PTptvec = float(PTptmod[0])
    else:
      PTptvec = 15
  if len(inrINRmod) > 1:
    inrINRvec = get_number_farthest_from_stdev(inrINRmod)
  else:
    if len(inrINRmod) == 1:
      inrINRvec = float(inrINRmod[0])
    else:
      inrINRvec = 1
else:
  zp="" 

#medical treatments
if Narrative == "yes" or Narrative == "Yes" or Narrative == "y" or Narrative == "Y":
  Dx5Tx1 = "Start ASA, Plavix, anticoagulation, statin, consider BB, consult Cardiology"
  Dx5Tx2 = "Will give aspirin and trend troponins"
  Dx5Tx3 = "Start ASA, Plavix, anticoagulation, statin, will need to rate control, consult Cardiology"
  Dx5Tx4 = "Start ASA, Plavix, anticoagulation, statin, maintain HR at acceptable level to maximize perfusion, consult Cardiology"
  Dx5Tx5 = "Start ASA, Plavix, anticoagulation, statin, will need afterload reduction and preload reduction, consult Cardiology"
  Dx5Tx6 = "Start ASA, Plavix, anticoagulation, statin, will start nitroglycerin drip and consult Cardiology"
  Dx5Tx7 = "Start ASA, Plavix, anticoagulation, statin, will need to consult Cardiology and may consider balloon pump for cardiogenic shock component"
  Dx5Tx8 = "Start ASA, Plavix, anticoagulation, statin, will need to consult Cardiology and may consider cath lab for PCI and or balloon pump"
  Dx5Tx9 = "Will maintain MAP >65, cannot give blood thinners and will consider use of nitro drip or esmolol for rapid titration"
  Dx6Tx1 = "Start esmolol and monitor BP.  Will call surgery."
  Dx6Tx2 = "Will need ventilatory support, immediately discuss with cardiac surgery"
  Dx6Tx3 = "Will start esmolol or labetalol and monitor BP and HR.  Will call vascular surgery"
  Dx6Tx4 = "Will need to get type and screen, consider immediate blood transfusion, start pressors and emergently contact surgery"
  Dx7Tx1 = "Place tele monitoring, consider amiodarone and if unstable will need to cardiovert"
  Dx7Tx2 = "Obtain EKG, consider adenosine or betablocker if HD stable.  Will need to cardiovert if unstable"
  Dx7Tx3 = "Can start labetalol and consider anxiolysis or pain control"
  Dx7Tx4 = "If SVT will need to electrically cardiovert.  Will give IV fluid bolus and may need to start vaso pressors"
  Dx7Tx5 = "Will monitor after nebs given"
  Dx7Tx6 = "Will need to monitor for worsening hemodynamic collapse"
  Dx7Tx7 = "Will need to slow rate.  Will give bolus beta blocker, calcium channel or digoxin.  If no responding will need to start drip"
  Dx7Tx8 = "Will need to slow down rate, can improve filling with IVF and if no response need amiodarone.  If pt remains unstable with hypotension will need to electrically cardiovert"
  Dx8Tx1 = "Pt bradycardic, will need to monitor for hemodynamic instability.  Keep atropine close by consider pacing if HD unstable, hold rate slowing agents"
  Dx8Tx2 = "Will need to give atropine and if no improvement consider isoproterenol or dopamine"
  Dx8Tx3 = "Will need to pace patient.  Will start with transthoracic pacing and will need tranvenous pacer placed.  Send lyme, TSH, troponin and 12 lead"
  Dx9Tx1 = "Lasix, beta blocker, ace inhibitor if can tolerate"
  Dx9Tx2 = "Will start nitroglycerine drip, give lasix, consider labetalol, send troponin, EKG"
  Dx9Tx3 = "Will start levophed drip and add epi or dobutamine if cardiac output is not improved.  Will need troponin, EKG, lactate level, monitor urine and mental status"
  Dx9Tx4 = "Will start nitroglycerine drip, give lasix, consider labetalol, send troponin, EKG.  Pt will also need positive pressure ventilation to unload LV"
  Dx9Tx5 = "Will need to diurese patient and start lasix aldactone and obtain echo"
  Dx9Tx6 = "Will need to diurese patient, continue antibiotics and consult CV surgery"
  Dx9Tx7 = "Will need to target BP <140, trend troponins, give lasix"
  Dx9Tx8 = "Will start diuresis, place foley and trend creatinine"
  Dx10Tx1 = "Will need to reduce MAP by no more than 25% in the first 24h.  Start cardene gtt"
  Dx10Tx2 = "Will need to improve work of breathing and evaluate for improvement in blood pressure.  When breathing is improved, can start labetalol push or nicardipine drip"
  Dx10Tx3 = "Start labetalol and cardene drip to achieve SBP <160"
  Dx10Tx4 = "Start labetalol and cardene drip to achieve SBP <140 until bleeding focus coiled"
  Dx10Tx5 = "Will need to reduce MAP by no more than 25% in the first 24h. Monitor mental status, evaluate for herniation, start cardene and labetalol PRN"
  Dx10Tx6 = "Will need to reduce MAP by no more than 25% in the first 24h. Will give cardene gtt or labetalol and consider addition of hydralazine if no effect"
  Dx11Tx1 = "Start levophed and bolus crystalloid.  Obtain echo to differentiate shock type."
  Dx11Tx2 = "Start antibiotics and bolus crystalloid up to 30 cc/kg and if no response start levophed and titrate to MAP >65.  Trend lactates and obtain echo to differentiate shock type"
  Dx11Tx3 = "Will bolus crystalloid up to 30 cc/kg and if no response start levophed and titrate to MAP >65.  Will place chest tube"
  Dx11Tx4 = "Will start levophed and maintain MAP >65.  If not improvement, will need to add epinephrine"
  Dx11Tx5 = "Check troponin, CBC, basic, draw cultures, start vanc zosyn and may need IV fluid bolus or levophed to maintain MAP >60"
  Dx11Tx6 = "Will get 2 large bore IV, send type and screen, get 4:4:1 RBC/FFP/plt transfusion on call, check cbc, check PTT, INR and basic.  Will also need to call GI"
  Dx12Tx1 = "Will CT head and if no bleed will need targeted temperature management"
  Dx12Tx2 = "Start epi drip and titrate to MAP of 65.  If persistently bradycardic may need to consider pacing"
  Dx12Tx3 = "Will need emergent tPA and discuss with cardiac surgery crashing onto ECMO"
  Dx13Tx1 = "Will monitor airway and intubated if needed, start decadron 10 and contact ENT and Pulm"
  Dx13Tx2 = "Will start levo and then epi or dopamine if not effect.  Pt may need transthoracic pacing and will need tranvenous pacer placed.  Send lyme, TSH, troponin and 12 lead"
  Dx14Tx1 = "Begin nebs, steroids, azithromycin.  The patient may need bipap support if hypercapneic."
  Dx14Tx2 = "Will treat ischemic disease and consider bipap steroids and nebulizers"
  Dx14Tx3 = "Begin nebs, steroids, pt may need bipap support, start ceftriaxone and azithro"
  Dx14Tx4 = "Will continue nebs, azithro and lower steroids.  Pt may need to have pigtail placed if pneumothorax expands"
  Dx14Tx5 = "Will start duonebs, steroids, azithro and consider BIPAP as needed."
  Dx14Tx6 = "Will start duonebs, steroids, azithromycin and wean ventilator as tolerated."
  Dx15Tx1 = "Will start ceftriaxone azithro for CAP or vancomycin and zosyn for HCAP, send blood cxs, sputum cxs, check covid-19, respiratory viral pance, legionella Ag and chlamydia spp ag." 
  Dx15Tx2 = "Will send repeat blood cultures daily until cleared and start vancomycin and zosyn"
  Dx16Tx1 = "Will need tube decompression if unresolved with oxygen"
  Dx17Tx1 = "Pt will need BIPAP or need to be intubated if no improvement in parameters."
  Dx17Tx2 = "Will need to check ABG, start BIPAP or place on ventilator, monitor for autopeep."
  Dx17Tx3 = "Will check ABG, start steroids, duonebs, attempt trial of BIPAP however the patient may need to be intubated."
  Dx17Tx4 = "The pt will need to be ventilated, will start ceftriaxone azithro, send blood cxs, sputum cxs, check covid-19, respiratory viral pance, legionella Ag and chlamydia spp ag."
  Dx17Tx5 = "Place chest tube and will need to place patient on the ventilator."
  Dx17Tx6 = "Will start anticoagulation emergently and may consider tPA vs thrombectomy vs ECMO.  Pt may need to be intubated but will keep PEEP lower so as to not worsen RV dysfunction."
  Dx17Tx7 = "Will place patient on nebulizers, lasix, obtain chest xray, and use BIPAP or place on ventilator."
  Dx17Tx8 = "Place on ventilator and monitor ABG and CO2 levels to reduce risk of increased ICP."
  Dx18Tx1 = "Will start anticoagulation, get trops and echo to evaluate for RV strain"
  Dx18Tx2 = "Will need to place IVC filter as patient is not a candidate for anticoagulation."
  Dx19Tx1 = "monitor for encephalopathy, hypotension infection and GIB"
  Dx19Tx2 = "Will start lasix and aldactone as well as vanco.  Monitor renal function"
  Dx20Tx1 = "Will get large bore IV, type and screen, transfuse Hgb > 7, plt > 50, INR <1.7.  Will call GI and start IV protonix for UGIB and octreotide if cirrhotic.  DDAVP for azotemia"
  Dx20Tx2 = "Will get large bore IV, type and screen, transfuse Hgb > 7, plt > 50, INR <1.7.  Will call GI and start IV protonix and octreotide.  DDAVP for azotemia"
  Dx21Tx1 = "Will start fluids, monitor FS levels, check TG, NPO, consider imaging of abdomen and if >30% necrosis then start carbapenem."
  Dx22Tx1 = "Will start Cipro/flagyl and send stool studies."
  Dx22Tx2 = "Will start IV flagyl and PO vancomycin.  If severe, can also add cefepime."
  Dx23Tx1 = "Will send stool studies."
  Dx24Tx1 = "Will place NGT, start bowel rest/NPO, contact surgery and start IVF."
  Dx25Tx1 = "Will send repeat blood cultures daily until cleared and start vancomycin and zosyn."
  Dx25Tx2 = "Will send repeat blood cultures daily until cleared and start vancomycin, zosyn, and flagyl."
  Dx25Tx3 = "Will send repeat blood cultures daily until cleared and start vancomycin, ceftriaxone and ampicillin.  Will get prelim cultures to decide on use of steroids."
  Dx25Tx4 = "Will send repeat blood cultures daily until cleared and start vancomycin, clindamycin and zosyn.  Will get CT scan of affected area and consider surgery consult."
  Dx26Tx1 = "Will follow up LP data and start vanc/ceftriaxone.  If concern for meningococcus, will start steroids.  Ampicllin for high risk of listeria."
  Dx27Tx1 = "Will need to start vanco, possible TEE then discuss case with ID and CT surgery."
  Dx28Tx1 = "Will monitor mental status and intubate if unable to protect airway.  Will need to discuss case with neurosurgery and consider sedation, hypertonic or mannitol for elevated ICPs."
  Dx28Tx2 = "Will monitor mental status and intubate if unable to protect airway.  Will need to discuss case with neurosurgery and consider sedation, hypertonic or mannitol for elevated ICPs.  Keep SBP <160"
  Dx28Tx3 = "Will monitor mental status and intubate if unable to protect airway.  Will need to discuss case with neurosurgery and consider sedation, hypertonic or mannitol for elevated ICPs.  Start keppra."
  Dx28Tx4 = "Will monitor mental status and intubate if unable to protect airway.  Will need to discuss case with neurosurgery and consider sedation, hypertonic or mannitol for elevated ICPs.  Reverse any coagulopathy and start decadron for tumor."
  Dx28Tx5 = "WIll need to keep SBP <160 in setting of ICH/SDH, order cardene and PRN labetalol"
  Dx29Tx1 = "Will monitor mental status and intubate if unable to protect airway.  Will need to discuss case with neurosurgery and consider sedation, hypertonic or mannitol for elevated ICPs."
  Dx29Tx2 = "Will monitor mental status and intubate if unable to protect airway.  Will need to discuss case with neurosurgery and consider sedation, hypertonic or mannitol for elevated ICPs.  Keep SBP <160.  Pt with seizures will need keppra and benzos for breakthrough seizures."
  Dx29Tx3 = "Will monitor mental status and intubate if unable to protect airway.  Will need to discuss case with neurosurgery and consider sedation, hypertonic or mannitol for elevated ICPs.  Keppra Prophylaxis. Keep SBP <160.  Will need to discuss decadron, keep HOB 45 degrees."
  Dx30Tx1 = "Will discuss with neurosurgery need for ICP monitor.  Neuro checks, after DSA and securing potential aneurysm will allow permissive htn to reduce risk of vasospasm.  Start nimodipine."
  Dx30Tx2 = "Will discuss with neurosurgery need for ICP monitor.  Neuro checks, after DSA and securing potential aneurysm will allow permissive htn to reduce risk of vasospasm.  Start nimodipine,keppra and benzo for breakthrough seizing."
  Dx31Tx1 = "Start seizure precautions, benzos for breakthrough seizure and fosphenytoin for status epilepticus.  Start EEG and discuss with neuro need for keppra."
  Dx32Tx1 = "Pt with evidence of brain tumor on imaging"
  Dx33Tx1 = "Will start ceftriaxone"
  Dx34Tx1 = "Will need to start gram positive coverage"
  Dx34Tx2 = "Will start vanco and consider surgical eval or imaging.  If noted to have necrotizing fasciitis, will add clinda to reduce exotoxin"
  Dx35Tx1 = "Will start PO Vanco or IV Flagyl"
  Dx36Tx1 = "Will stat insulin sliding scale and add long acting insulin as needed to keep serum glucose 150-180"
  Dx37Tx1 = "Will give fluids, renal ultrasound, renally adjust meds, replete electrolytes and check urinalysis"
  Dx38Tx1 = "Will start insulin drip in order to close AG and then transition back to long acting insulin."
  Dx39Tx1 = "Will need to send TSH, reticulocytes, bilirubin, iron level, TIBC, iron saturation, look for shistocytes, haptoglobin, SPEP, UPEP. Keep Hgb >7 and plt >10"
  Dx40Tx1 = "Will give d50 and if persistent, start d10 drip."
  Dx41Tx1 = "Will give freewater, correct 6-8 meq / 24h"
  Dx42Tx1 = "Will send TSH, Cortisol, urine sodium, and urine osmolality.  Will need hypertonic if symptomatic or with seizures.  Correct 6-8 meq per 24h. Consult renal."
  Dx42Tx2 = "PT noted to have hyponatremia. Monitor for AMS and seizures.  Give Hypertonic if patient begins having seizures."
  Dx43Tx1 = "Will check EKG, if changes will give calcium, B2 agonist, insulin/D50.  May consider lokelma, kayexalate and/or dialysis if not resolving"
  Dx44Tx1 = "Will give potassium replacement, monitor for arrhythmia"
  Dx45Tx1 = "Will monitor for bleeding"
  Dx45Tx2 = "Will need to replete to INR <1.6, Plt >50, or aPTT <50.  If bleeding persists send repeat INR, CBC, aPTT, fibrinogen and TEG/ROTEM."
  Dx46Tx1 = "Pt HD stable monitor vitals"
  Dx47Tx1 = "Promote IS use, keep o2 sat >92%"
  Dx48Tx1 = "Advance diet when ok with primary"
  Dx48Tx2 = "Will keep patient NPO for now"
  Dx49Tx1 = "Monitor for AKI and replete lytes"
  Dx50Tx1 = "Monitor for delirium"
  Dx51Tx1 = "Monitor for bleeding and blood clots. Keep Hgb >7 and plt >10"
  Dx52Tx1 = "Maintain FS 150-180"
  Dx53Tx1 = "Monitor for infection"
  Dx54Tx1 = "The pt will need to be monitored closely for DVT as there are contraindications for both chemical and mechanical DVT ppx."
  Dx54Tx2 = "The pt will need to be monitored closely for DVT as there are contraindications for both chemical and mechanical DVT ppx.  PPI or H2 for GI ppx"
  Dx54Tx3 = "SCD for DVT PPx.  PPI or H2 for GI ppx"
  Dx54Tx4 = "SCD for DVT"
  Dx54Tx5 = "Chemical DVT PPx.  PPI or H2 for GI ppx"
  Dx54Tx6 = "Chemical DVT PPx"
  Dx57Tx1 = "WIll continue sedation and perform daily sedation vacations"
  Dx57Tx2 = "Will continue vent support and wean as tolerated, will also need diuresis to improve A-a and ventilattion"
  Dx57Tx3 = "Will need to increase prolong expiratory phase as much as possible, monitor for autopeep"
  Dx57Tx4 = "Will continue vent support and wean as tolerated, will need to monitor for ARDS also need diuresis to improve A-a and ventilation"
  Dx57Tx5 = "Will continue vent support and wean as tolerated, start q6 duonebs, solumedrol as well as ceftriaxone azithro for CAP or vancomycin and zosyn for HCAP."
  Dx57Tx6 = "Will start duonebs, steroids, consider BIPAP.  Solumedrol as well as ceftriaxone azithro for CAP or vancomycin and zosyn for HCAP, send blood cxs, sputum cxs, check covid    -19, respiratory viral pance, legionella Ag and chlamydia spp ag."
  Dx58Tx1 = "Will rate control patient < 110 and start anticoagulation unless there is a contraindication."
  Dx58Tx2 = "Will rate control patient < 110 however will need to hold anticoagulation."
  Dx59Tx1 = "Will check levels of possible toxic agents, check urine for ketones, get history on PO intake, send lactate level and monitor for improvement"
  Dx59Tx2 = "Will start DKA insulin protocol, give fluids, and transition after gap closed to long activen and sliding scale"
  Dx59Tx3 = "Will repeat basic and if persistent, send SPEP UPEP"
  Dx60Tx1 = "Will need to consult renal and discuss need for dialysis.  Will need to renally adjust medications"
  Dx61Tx1 = "Will monitor for fluid overload, avoid nephrotoxic agents, renally adjust meds, and consult renal"
  Dx62Tx1 = "If HD stable, can continue home BP meds"
  Dx62Tx2 = "Will need to hold BP medications until BP improves"
  Dx62Tx3 = "Will need PRN labetalol or hydralazine to keep SBP <160"
  Dx62Tx4 = "HTN in setting of CVA will allow Bp 180/105 post tpa.  PRN labetalol and hydralazine"
  Dx63Tx1 = "Would continue nebs as needed, check peak flows, steroids if work of breathing does not improve"
  Dx63Tx2 = "Will start continuous nebs, steroids, consider duonebs and magnesium if no improvement.  Will need peak flow monitoring"
  Dx64Tx1 = "Will need to monitor for seizures, AMS and delirium.  Pt will need benzo therapy and if no improvement consider adding precedex or barbiturate"
  Dx64Tx2 = "Will need to monitor for seizures, AMS and delirium.  Pt will need benzo therapy and sedate with propofol and if needed precedex."
  Dx64Tx3 = "Will need to monitor for DTs and sympathetic surge"
  Dx65Tx1 = "Will start ASA and statin.  If pt has coronary stent will need plavix."
  Dx65Tx2 = "Will monitor for signs of ischemia but will need to hold antiplatelets."
  Dx66Tx1 = "If home CPAP will need to continue at night.  Monitor closely for sedatives to reduce risk of obstruction and hypoxia."
  Dx67Tx1 = "Will need to repeat TSH and T4, continue synthroid"
  Dx68Tx1 = "Will need to repeat TSH and T4, continue methimazole/propylthiouricil"
  Dx69Tx1 = "Keep BP <180/105 after tPA and <220/120 if no tPA, repeat CT or MRI, pt will need echo eval source of embolus, send lipid profile, HbA1c, TFT, ANA, RF, CRP, ESR, B12, Folate, UDS, Syphilis Ab, HIV Ab. Consider ASA/plavix when safe from neuro standpoint, cont lipitor" 
  Dx69Tx2 = "Keep BP  <220/120 unable to give tPA given coagulopathy, repeat CT or MRI, pt will need echo eval source of embolus, send lipid profile, HbA1c, TFT, ANA, RF, CRP, ESR, B12, Folate, UDS, Syphilis Ab, HIV Ab.  Cont statin"
  Dx70Tx1 = "Will need to reorient patient, reduce deliriogenic drugs, open windows, promote day-night circadian ryhthm, encourage familiar faces, treat pain and if no improvement consider antipsychotic or precedex."
  Dx70Tx2 = "Will need to reorient patient, reduce deliriogenic drugs, open windows, promote day-night circadian ryhthm, encourage familiar faces, treat pain and if no improvement consider antipsychotic or precedex. Pt also with cirrhosis send ammonia level"
  Dx71Tx1 = "Cont statin and lipid lowering agents"
  Dx72Tx1 = "Will continue home regimen"
  Dx72Tx2 = "Will send C3, C4, anti-smith, CRP, anti-DS, and consult rheumatology."
  Dx72Tx3 = "Will send C3, C4, anti-smith, CRP, anti-DS, MRI brain and consult rheumatology as well as neurology."
  Dx72Tx4 = "Will send C3, C4, anti-smith, CRP, anti-DS, urine microscopy and consult rheumatology as well as nephrology."
  Dx73Tx1 = "Will continue home regimen and consult rheumatology"
  Dx74Tx1 = "Will need to consult cardiology and evaluate for tamponade, send ESR, CRP, ANA, cultures, echo to start"
  Dx74Tx2 = "Will need to consult CV surgery, get echo, monitor hemodynamics."
  Dx75Tx1 = "Will send cultures, start zosyn, call surgery and may consider HIDA"
  Dx76Tx1 = "Will need to send morning cortisol, TSH, consider ACTH stim test.  Begin hydrocortisone and fludrocortisone"
  Dx77Tx1 = "Will need to send morning cortisol, TSH, consider dexamethasone suppression test."
  Dx78Tx1 = "Will need lung protective ventilation, minimize O2, permissive hypercapnia, TV 6cc/kg, high PEEP.  If severe hypoxia/ARDS will need to consider paralytic, proning or diuretic if BP tolerates"
  Dx79Tx1 = "Will need to monitor CPP >60 if bolt placed, start keppra prophylaxis, neurochecks Q1, repeat CT if change in mental status"
  Dx80Tx1 = "Will need transfer to burn center, place silvadene for wounds and cover affected areas"
  Dx81Tx1 = "Monitor for distal pulseness and persistent limb pain.  Cont ASA, statin."
  Dx82Tx1 = "Monitor neuro exam, call neurosurg, consider starting decadron and keppra."
  Dx83Tx1 = "Cont PPI or H2 Blocker"
  histtx12= "Will continue nebs and inhaled steroids"
  histtx15= "Will need to monitor for GIB, check ammonia levels, if volume overloaded start diuretics"
  histtx37= "Continue home AED regimen"
  histtx41= "Continue home inhalers."
  histtx75= "Cont vent support with trach and wean as tolerated"
else:
  Dx5Tx1 = """
    -ASA
    -Plavix
    -anticoagulation
    -statin
    -consider BB
    -consult Cardiology"""
  Dx5Tx2 = """
    -Will give aspirin
    -trend troponins"""
  Dx5Tx3 = """
    -Start ASA, Plavix, anticoagulation, statin
    -Will need to rate control, consult Cardiology"""
  Dx5Tx4 = """
    -Start ASA, Plavix, anticoagulation, statin
    -maintain HR at acceptable level to maximize perfusion
    -consult Cardiology"""
  Dx5Tx5 = """
    -Start ASA, Plavix, anticoagulation, statin
    -will need afterload reduction and preload reduction
    -consult Cardiology"""
  Dx5Tx6 = """
    -Start ASA, Plavix, anticoagulation, statin
    -will start nitroglycerin drip
    -consult Cardiology"""
  Dx5Tx7 = """
    -Start ASA, Plavix, anticoagulation, statin
    -will need to consult Cardiology
    -consider balloon pump for cardiogenic shock component"""
  Dx5Tx8 = """
    -Start ASA, Plavix, anticoagulation, statin
    -will need to consult Cardiology 
    -consider cath lab for PCI and or balloon pump"""
  Dx5Tx9 = """Cannot give blood thinners
    -Will maintain MAP >65
    -consider use of nitro drip or esmolol for rapid titration"""
  Dx6Tx1 = """
    -Start esmolol and monitor BP
    -Call surgery."""
  Dx6Tx2 = """
    -Will need ventilatory support
    -immediately discuss with cardiac surgery"""
  Dx6Tx3 = """
    -Will start esmolol or labetalol
    -monitor BP and HR
    -Will call vascular surgery"""
  Dx6Tx4 = """
    -Will need to get type and screen
    -consider immediate blood transfusion
    -start pressors
    -emergently contact surgery"""
  Dx7Tx1 = """
    -Place tele monitoring
    -consider amiodarone
    -if unstable will need to cardiovert"""
  Dx7Tx2 = """
    -Obtain EKG
    -consider adenosine or betablocker if HD stable
    -Will need to cardiovert if unstable"""
  Dx7Tx3 = """
    -Can start labetalol 
    -consider anxiolysis or pain control"""
  Dx7Tx4 = """
    -If SVT will need to electrically cardiovert
    -Will give IV fluid bolus and may need to start vasopressors"""
  Dx7Tx5 = """
    -Will monitor after nebs given"""
  Dx7Tx6 = """
    -Will need to monitor for worsening hemodynamic collapse"""
  Dx7Tx7 = """
    -Will need to slow rate
    -Will give bolus beta blocker, calcium channel or digoxin
    -If no responding will need to start drip"""
  Dx7Tx8 = """
    -Will need to slow down rate
    -can improve filling with IVF and if no response need amiodarone
    -If pt remains unstable with hypotension will need to electrically cardiovert"""
  Dx8Tx1 = """
    -Pt bradycardic, will need to monitor for hemodynamic instability
    -Keep atropine close by consider pacing if HD unstable
    -Hold rate slowing agents"""
  Dx8Tx2 = """
    -Will need to give atropine
    -if no improvement consider isoproterenol or dopamine"""
  Dx8Tx3 = """
    -Will need to pace patient
    -Will start with transthoracic pacing 
    -Will need tranvenous pacer placed
    -Send lyme, TSH, troponin and 12 lead"""
  Dx9Tx1 = """
    -Lasix, beta blocker, ace inhibitor if can tolerate"""
  Dx9Tx2 = """
    -Will start nitroglycerine drip
    -give lasix, consider labetalol
    -send troponin, EKG"""
  Dx9Tx3 = """
    -Will start levophed drip and add epi or dobutamine if cardiac output is not improved
    -Will need troponin, EKG, lactate level, monitor urine and mental status"""
  Dx9Tx4 = """
    -Will start nitroglycerine drip
    -give lasix
    -consider labetalol
    -send troponin, EKG
    -Pt will also need positive pressure ventilation to unload LV"""
  Dx9Tx5 = """
    -Will need to diurese patient
    -start lasix aldactone
    -obtain echo"""
  Dx9Tx6 = """
    -Will need to diurese patient
    -continue antibiotics
    -consult CV surgery"""
  Dx9Tx7 = """
    -Will need to target BP <140
    -trend troponins
    -give lasix"""
  Dx9Tx8 = """
    -Will start diuresis
    -place foley
    -trend creatinine"""
  Dx10Tx1 = """
    -Start cardene gtt
    -reduce MAP by no more than 25% in the first 24h"""
  Dx10Tx2 = """
    -Will need to improve work of breathing and evaluate for improvement in blood pressure
    -When breathing is improved, can start labetalol push or nicardipine drip"""
  Dx10Tx3 = """
    -Start labetalol and cardene drip to achieve SBP <160"""
  Dx10Tx4 = """
    -Start labetalol and cardene drip to achieve SBP <140 until bleeding focus coiled"""
  Dx10Tx5 = """
    -start cardene and labetalol PRN
    -Will need to reduce MAP by no more than 25% in the first 24h
    -Monitor mental status, evaluate for herniation"""
  Dx10Tx6 = """
    -Will need to reduce MAP by no more than 25% in the first 24h
    -Will give cardene gtt or labetalol and consider addition of hydralazine if no effect"""
  Dx11Tx1 = """
    -Start levophed and bolus crystalloid
    -Obtain echo to differentiate shock type."""
  Dx11Tx2 = """
    -Start antibiotics and bolus crystalloid up to 30 cc/kg
    -if no response start levophed and titrate to MAP >65
    -Trend lactates and obtain echo to differentiate shock type"""
  Dx11Tx3 = """
    -Will bolus crystalloid up to 30 cc/kg
    -if no response start levophed and titrate to MAP >65
    -Will place chest tube"""
  Dx11Tx4 = """
    -Will start levophed and maintain MAP >65
    -If not improvement, will need to add epinephrine"""
  Dx11Tx5 = """
    -Check troponin, CBC, basic, draw cultures
    -start vanc zosyn
    -may need IV fluid bolus or levophed to maintain MAP >60"""
  Dx11Tx6 = """
    -Will get 2 large bore IV
    -send type and screen
    -get 4:4:1 RBC/FFP/plt transfusion on call
    -check cbc, check PTT, INR and basic
    -Will also need to call GI"""
  Dx12Tx1 = """
    -Will CT head and if no bleed will need targeted temperature management"""
  Dx12Tx2 = """
    -Start epi drip and titrate to MAP of 65
    -If persistently bradycardic may need to consider pacing"""
  Dx12Tx3 = """
    -Will need emergent tPA
    -discuss with cardiac surgery crashing onto ECMO"""
  Dx13Tx1 = """
    -Will monitor airway and intubated if needed, start decadron 10 and contact ENT and Pulm"""
  Dx13Tx2 = """
    -Will start levo and then epi or dopamine
    -if not effect.  Pt may need transthoracic pacing and will need tranvenous pacer placed
    -Send lyme, TSH, troponin and 12 lead"""
  Dx14Tx1 = """
    -Begin nebs, steroids, azithromycin
    -The patient may need bipap support if hypercapneic."""
  Dx14Tx2 = """
    -Will treat ischemic disease
    -consider bipap steroids and nebulizers"""
  Dx14Tx3 = """
    -Begin nebs, steroids
    -pt may need bipap support
    -start ceftriaxone and azithro"""
  Dx14Tx4 = """
    -Will continue nebs, azithro and lower steroids
    -Pt may need to have pigtail placed if pneumothorax expands"""
  Dx14Tx5 = """
    -Will start duonebs, steroids, azithro
    -consider BIPAP as needed."""
  Dx14Tx6 = """
    -Will start duonebs, steroids, azithromycin
    -wean ventilator as tolerated."""
  Dx15Tx1 = """
    -Will start ceftriaxone azithro for CAP or vancomycin and zosyn for HCAP
    -send blood cxs, sputum cxs
    -check covid-19, respiratory viral pance, legionella Ag and chlamydia spp ag.""" 
  Dx15Tx2 = """
    -Will send repeat blood cultures daily until cleared
    -start vancomycin and zosyn"""
  Dx16Tx1 = """
    -Will need tube decompression if unresolved with oxygen"""
  Dx17Tx1 = """
    -Pt will need BIPAP or need to be intubated if no improvement in parameters."""
  Dx17Tx2 = """
    -Will need to check ABG
    -start BIPAP or place on ventilator
    -monitor for autopeep."""
  Dx17Tx3 = """
    -Will check ABG
    -start steroids, duonebs
    -attempt trial of BIPAP however the patient may need to be intubated."""
  Dx17Tx4 = """
    -The pt will need to be ventilated
    -will start ceftriaxone azithro
    -send blood cxs, sputum cxs
    -check covid-19, respiratory viral pance, legionella Ag and chlamydia spp ag."""
  Dx17Tx5 = """
    -Place chest tube
    -place patient on the ventilator."""
  Dx17Tx6 = """
    -Will start anticoagulation emergently
    -consider tPA vs thrombectomy vs ECMO.
    -Pt may need to be intubated but will keep PEEP lower so as to not worsen RV dysfunction."""
  Dx17Tx7 = """
    -Will place patient on nebulizers, lasix
    -obtain chest xray
    -use BIPAP or place on ventilator."""
  Dx17Tx8 = """
    -Place on ventilator
    -monitor ABG and CO2 levels to reduce risk of increased ICP."""
  Dx18Tx1 = """
    -Will start anticoagulation
    -get trops
    -echo to evaluate for RV strain"""
  Dx18Tx2 = """
    -Will need to place IVC filter as patient is not a candidate for anticoagulation."""
  Dx19Tx1 = """
    -monitor for encephalopathy, hypotension infection and GIB"""
  Dx19Tx2 = """
    -Will start lasix and aldactone
    -start vanco
    -Monitor renal function"""
  Dx20Tx1 = """
    -Will get large bore IV
    -type and screen
    -transfuse Hgb > 7, plt > 50, INR <1.7
    -Will call GI and start IV protonix for UGIB and octreotide if cirrhotic.  DDAVP for azotemia"""
  Dx20Tx2 = """
    -Will get large bore IV
    -type and screen
    -transfuse Hgb > 7, plt > 50, INR <1.7
    -Will call GI
    -start IV protonix and octreotide
    -DDAVP for azotemia"""
  Dx21Tx1 = """
    -Will start fluids
    -monitor FS levels
    -check TG
    -NPO
    -consider imaging of abdomen and if >30% necrosis then start carbapenem."""
  Dx22Tx1 = """
    -Will start Cipro/flagyl
    -send stool studies."""
  Dx22Tx2 = """
    -Will start IV flagyl and PO vancomycin. If severe, can also add cefepime."""
  Dx23Tx1 = """
    -Will send stool studies."""
  Dx24Tx1 = """
    -Will place NGT
    -start bowel rest/NPO
    -contact surgery
    -start IVF."""
  Dx25Tx1 = """
    -Will send repeat blood cultures daily until cleared
    -start vancomycin and zosyn."""
  Dx25Tx2 = """
    -Will send repeat blood cultures daily until cleared
    -start vancomycin, zosyn, and flagyl."""
  Dx25Tx3 = """
    -Will send repeat blood cultures daily until cleared
    -start vancomycin, ceftriaxone and ampicillin
    -Will get prelim cultures to decide on use of steroids."""
  Dx25Tx4 = """
    -Will send repeat blood cultures daily until cleared
    -start vancomycin, clindamycin and zosyn
    -Will get CT scan of affected area
    -consider surgery consult."""
  Dx26Tx1 = """
    -Will follow up LP data and start vanc/ceftriaxone
    -If concern for meningococcus, will start steroids
    -Ampicllin for high risk of listeria."""
  Dx27Tx1 = """
    -Will need to start vanco
    -possible TEE
    -discuss case with ID and CT surgery."""
  Dx28Tx1 = """
    -Will monitor mental status and intubate if unable to protect airway
    -Will need to discuss case with neurosurgery and consider sedation
    -hypertonic or mannitol for elevated ICPs."""
  Dx28Tx2 = """
    -Will monitor mental status and intubate if unable to protect airway
    -Will need to discuss case with neurosurgery
    -consider sedation, hypertonic or mannitol for elevated ICPs
    -Keep SBP <160"""
  Dx28Tx3 = """
    -Will monitor mental status and intubate if unable to protect airway
    -Will need to discuss case with neurosurgery
    -consider sedation, hypertonic or mannitol for elevated ICPs
    -Start keppra."""
  Dx28Tx4 = """
    -Will monitor mental status and intubate if unable to protect airway
    -Will need to discuss case with neurosurgery 
    -consider sedation, hypertonic or mannitol for elevated ICPs
    -Reverse any coagulopathy
    -start decadron for tumor."""
  Dx28Tx5 = """
    -WIll need to keep SBP <160 in setting of ICH/SDH
    -order cardene and PRN labetalol"""
  Dx29Tx1 = """
    -Will monitor mental status and intubate if unable to protect airway
    -Will need to discuss case with neurosurgery
    -consider sedation, hypertonic or mannitol for elevated ICPs."""
  Dx29Tx2 = """
    -Will monitor mental status and intubate if unable to protect airway
    -Will need to discuss case with neurosurgery
    -consider sedation, hypertonic or mannitol for elevated ICPs
    -Keep SBP <160 
    -Pt with seizures will need keppra and benzos for breakthrough seizures."""
  Dx29Tx3 = """
    -Will monitor mental status and intubate if unable to protect airway.  Will need to discuss case with neurosurgery and consider sedation, hypertonic or mannitol for elevated ICPs.  Keppra Prophylaxis. Keep SBP <160.  Will need to discuss decadron, keep HOB 45 degrees."""
  Dx30Tx1 = """
    -Will discuss with neurosurgery need for ICP monitor
    -Neuro checks
    -After DSA and securing potential aneurysm, allow permissive htn to reduce risk of vasospasm
    -Start nimodipine."""
  Dx30Tx2 = """
    -Will discuss with neurosurgery need for ICP monitor
    -Neuro checks
    -After DSA and securing potential aneurysm, will allow permissive htn to reduce risk of vasospasm
    -Start nimodipine
    -keppra and benzo for breakthrough seizing."""
  Dx31Tx1 = """
    -Start seizure precautions
    -benzos for breakthrough seizure and fosphenytoin for status epilepticus
    -Start EEG
    -discuss with neuro need for keppra."""
  Dx32Tx1 = """
    -Pt with evidence of brain tumor on imaging"""
  Dx33Tx1 = """
    -Will start ceftriaxone"""
  Dx34Tx1 = """
    -Will need to start gram positive coverage"""
  Dx34Tx2 = """
    -Will start vanco
    -consider surgical eval or imaging
    -If noted to have necrotizing fasciitis, will add clinda to reduce exotoxin"""
  Dx35Tx1 = """
    -Will start PO vanco or IV Flagyl."""
  Dx36Tx1 = """
    -Will stat insulin sliding scale to keep serum glucose 150-180
    -add long acting insulin as needed"""
  Dx37Tx1 = """
    -Will give fluids
    -renal ultrasound
    -renally adjust meds
    -replete electrolytes
    -check urinalysis"""
  Dx38Tx1 = """
    -Will start insulin drip
    -Close AG x2 and then transition back to long acting insulin."""
  Dx39Tx1 = """
    -Will need to send TSH, reticulocytes, bilirubin, iron level, TIBC, iron saturation, haptoglobin, SPEP, UPEP.
    -Smear for shistocytes 
    -Keep Hgb >7 and plt >10"""
  Dx40Tx1 = """
    -Will give d50 and if hypoglycemia persistent, start d10 drip."""
  Dx41Tx1 = """
    -Will give freewater, correct 6-8 meq / 24h"""
  Dx42Tx1 = """
    -Will send TSH, Cortisol, urine sodium, and urine osmolality
    -Will need hypertonic if symptomatic or with seizures
    -Correct 6-8 meq per 24h
    -Consult renal."""
  Dx42Tx2 = """
    -PT noted to have hyponatremia
    -Monitor for AMS and seizures
    -Give Hypertonic saline if patient begins having seizures."""
  Dx43Tx1 = """
    -Will check EKG
    -If ekg changes, will give calcium, B2 agonist, insulin/D50
    -Consider lokelma, kayexalate
    -dialysis if not resolving"""
  Dx44Tx1 = """
    -Will give potassium replacement
    -monitor for arrhythmia"""
  Dx45Tx1 = """
    -Will monitor for bleeding"""
  Dx45Tx2 = """
    -Will need to replete to INR <1.6, Plt >50, or aPTT <50
    -If bleeding persists send repeat INR, CBC, aPTT, fibrinogen and TEG/ROTEM."""
  Dx46Tx1 = """
    -Pt HD stable monitor vitals"""
  Dx47Tx1 = """
    -Promote IS use
    -keep o2 sat >92%"""
  Dx48Tx1 = """
    -Advance diet when ok with primary"""
  Dx48Tx2 = """
    -Will keep patient NPO for now"""
  Dx49Tx1 = """
    -Monitor for AKI
    -replete lytes"""
  Dx50Tx1 = """
    -Monitor for delirium"""
  Dx51Tx1 = """
    -Monitor for bleeding and blood clots
    -Keep Hgb >7 and plt >10"""
  Dx52Tx1 = """
    -Maintain FS 150-180"""
  Dx53Tx1 = """
    -Monitor for infection"""
  Dx54Tx1 = """
    -The pt will need to be monitored closely for DVT as there are contraindications for both chemical and mechanical DVT ppx."""
  Dx54Tx2 = """
    -The pt will need to be monitored closely for DVT as there are contraindications for both chemical and mechanical DVT ppx
    -PPI or H2 for GI ppx"""
  Dx54Tx3 = """
    -SCD for DVT PPx
    -PPI or H2 for GI ppx"""
  Dx54Tx4 = """
    -SCD for DVT"""
  Dx54Tx5 = """
    -Chemical DVT PPx
    -PPI or H2 for GI ppx"""
  Dx54Tx6 = """
    -Chemical DVT PPx"""
  Dx57Tx1 = """
    -WIll continue sedation
    -Perform daily sedation vacations"""
  Dx57Tx2 = """
    -Will continue vent support and wean as tolerated
    -Will also need diuresis to improve A-a and ventilation"""
  Dx57Tx3 = """
    -Will need to prolong expiratory phase as much as possible, monitor for autopeep"""
  Dx57Tx4 = """
    -Will continue vent support and wean as tolerated
    -Monitor for ARDS
    -Need diuresis to improve A-a and ventilation"""
  Dx57Tx5 = """
    -Will continue vent support and wean as tolerated
    -start q6 duonebs
    -solumedrol 
    -ceftriaxone azithro for CAP or vancomycin and zosyn for HCAP."""
  Dx57Tx6 = """
    -Will start duonebs, steroids
    -Solumedrol as well as ceftriaxone azithro for CAP or vancomycin and zosyn for HCAP
    -send blood cxs, sputum cxs, check covid-19, respiratory viral pance, legionella Ag and chlamydia spp ag.
    -consider BIPAP"""
  Dx58Tx1 = """
    -Will rate control patient < 110
    -Start anticoagulation unless there is a contraindication."""
  Dx58Tx2 = """
    -Will rate control patient < 110
    -Hold anticoagulation."""
  Dx59Tx1 = """
    -Will check levels of possible toxic agents, check urine for ketones, send lactate level 
    -get history on PO intake
    -monitor for improvement"""
  Dx59Tx2 = """
    -Will start DKA insulin protocol
    -give fluids
    -transition after gap closed to long acting and sliding scale"""
  Dx59Tx3 = """
    -Will repeat basic and if persistent, send SPEP UPEP"""
  Dx60Tx1 = """
    -Will need to consult renal and discuss need for dialysis
    -Renally adjust medications"""
  Dx61Tx1 = """
    -Will monitor for fluid overload
    -avoid nephrotoxic agents, renally adjust meds
    -consult renal"""
  Dx62Tx1 = """
    -If HD stable, can continue home BP meds"""
  Dx62Tx2 = """
    -Will need to hold BP medications until BP improves"""
  Dx62Tx3 = """
    -Will need PRN labetalol or hydralazine to keep SBP <160"""
  Dx62Tx4 = """
    -HTN in setting of CVA will allow Bp 180/105 post tpa
    -PRN labetalol and hydralazine"""
  Dx63Tx1 = """
    -Would continue nebs as needed
    -Check peak flows
    -Administer steroids if work of breathing does not improve"""
  Dx63Tx2 = """
    -Will start continuous nebs, steroids, consider duonebs and magnesium if no improvement.  Will need peak flow monitoring"""
  Dx64Tx1 = """
    -Will need to monitor for seizures, AMS and delirium
    -Pt will need benzo therapy and if no improvement consider adding precedex or barbiturate"""
  Dx64Tx2 = """
    -Will need to monitor for seizures, AMS and delirium
    -Pt will need benzo therapy and sedate with propofol
    -If still withdrawaling, add precedex."""
  Dx64Tx3 = """
    -Will need to monitor for DTs and sympathetic surge"""
  Dx65Tx1 = """
    -Will start ASA and statin
    -If pt has coronary stent will need plavix."""
  Dx65Tx2 = """
    -Will monitor for signs of ischemia
    -Hold antiplatelets."""
  Dx66Tx1 = """
    -If home CPAP will need to continue at night
    -Monitor closely for sedatives to reduce risk of obstruction and hypoxia."""
  Dx67Tx1 = """
    -Will need to repeat TSH and T4
    -continue synthroid"""
  Dx68Tx1 = """
    -Will need to repeat TSH and T4
    -Continue methimazole/propylthiouricil"""
  Dx69Tx1 = """
    -Keep BP <180/105 after tPA and <220/120 if no tPA
    -Repeat CT or MRI
    -Pt will need echo eval source of embolus
    -Send lipid profile, HbA1c, TFT, ANA, RF, CRP, ESR, B12, Folate, UDS, Syphilis Ab, HIV Ab
    -Consider ASA/plavix when safe from neuro standpoint
    -Cont lipitor""" 
  Dx69Tx2 = """
    -Keep BP  <220/120 unable to give tPA given coagulopathy
    -Repeat CT or MRI
    -Pt will need echo eval source of embolus
    -Send lipid profile, HbA1c, TFT, ANA, RF, CRP, ESR, B12, Folate, UDS, Syphilis Ab, HIV Ab
    -Cont statin"""
  Dx70Tx1 = """
    -Will need to reorient patient
    -Reduce deliriogenic drugs, open windows, promote day-night circadian ryhthm, encourage familiar faces, treat pain
    -If no improvement, consider antipsychotic or precedex."""
  Dx70Tx2 = """
    -Will need to reorient patient
    -Reduce deliriogenic drugs, open windows, promote day-night circadian ryhthm, encourage familiar faces, treat pain
    -If no improvement, consider antipsychotic or precedex.
    -Pt with cirrhosis, send ammonia level"""
  Dx71Tx1 = """
    -Cont statin and lipid lowering agents"""
  Dx72Tx1 = """
    -Will continue home regimen"""
  Dx72Tx2 = """
    -Will send C3, C4, anti-smith, CRP, anti-DS
    -Consult rheumatology."""
  Dx72Tx3 = """
    -Will send C3, C4, anti-smith, CRP, anti-DS
    -MRI brain
    -Consult rheumatology as well as neurology."""
  Dx72Tx4 = """
    -Will send C3, C4, anti-smith, CRP, anti-DS, urine microscopy
    -Consult rheumatology as well as nephrology."""
  Dx73Tx1 = """
    -Will continue home regimen
    -consult rheumatology"""
  Dx74Tx1 = """
    -Will need to consult cardiology and evaluate for tamponade
    -Send ESR, CRP, ANA, cultures
    -Obtain echo"""
  Dx74Tx2 = """
    -Will need to consult CV surgery
    -Get echo, monitor hemodynamics."""
  Dx75Tx1 = """
    -Will send cultures
    -Start zosyn
    -Call surgery
    -Consider HIDA"""
  Dx76Tx1 = """
    -Will need to send morning cortisol, TSH, consider ACTH stim test
    -Begin hydrocortisone and fludrocortisone"""
  Dx77Tx1 = """
    -Will need to send morning cortisol, TSH
    -Consider dexamethasone suppression test."""
  Dx78Tx1 = """
    -Will need lung protective ventilation, minimize O2, permissive hypercapnia, TV 6cc/kg, high PEEP
    -If severe hypoxia/ARDS will need to consider paralytic, proning or diuretic if BP tolerates"""
  Dx79Tx1 = """
    -Will need to monitor CPP >60 if bolt placed
    -Start keppra prophylaxis
    -Neurochecks Q1
    -Repeat CT if change in mental status"""
  Dx80Tx1 = """
    -Will need transfer to burn center
    -Place silvadene for wounds and cover affected areas"""
  Dx81Tx1 = """
    -Monitor for distal pulseness and persistent limb pain
    -Cont ASA, statin."""
  Dx82Tx1 = """
    -Monitor neuro exam, call neurosurg
    -Consider starting decadron and keppra."""
  Dx83Tx1 = """
    -Cont PPI or H2 Blocker"""
  histtx12= """
  -continue nebs
  -inhaled steroids"""
  histtx15= """
  -Monitor for GIB
  -Check ammonia levels
  -If volume overloaded, start diuretics"""
  histtx37= """
  -Continue home AED regimen"""
  histtx41= """
  -Continue home inhalers."""
  histtx75= """
  -Cont vent support on trach
  -Wean as tolerated"""

#Op A/P

Op1 = """#1 Neuro: Multimodal analgesia.
 
#2 CV: The pt is a HD stable at present s/p peripheral angioplasty.  Cont stable dose heparin, papaverine, asa, neurovascular checks q1. 
 
#3 Pulm: Promote IS use, keep O2 to keep SaO2 >92%.   
 
#4 GI: Advance diet when cleared by surgeon

#5 Renal: Monitor for renal failure, replete lytes
 
#6 ID: monitor for infection, cefazolin ppx
 
#7 Endo: keep FS 150-180
 
#8 Heme: Tx plats >10k hgb >7, check CBC
 
#9 PPx: HSQ and PPI """

Op2 = """#1 Neuro: Monitor for delirium.
 
#2 CV: The pt has cardiogenic shock now s/p impella placement.  COntinue inotropes and pressors as needed.
 
#3 Pulm: Promote IS use, keep O2 to keep SaO2 >92%.   
 
#4 GI: Advance diet when ok with Cardiology

#5 Renal: Monitor for renal failure, replete lytes
 
#6 ID: monitor for infection
 
#7 Endo: keep FS 150-180
 
#8 Heme: Tx plats >10k hgb >7, check CBC
 
#9 PPx: HSQ and PPI"""

Op3 = """#1 Neuro: Brain tumor cerebral edema. Maintain SBP < 160mmHg to prevent neurological deterioration. Neuro checks per protocol.  Repeat CT.  Cont decadron, continue keppra, prn analgesia, zofran prn, vanc/cefepime. PRN ativan

#2 CV: HTN in setting of ICH.  Will keep SBP <160 with cardene.  
 
#3 Pulm: Pt intubated wean as tolerated.
     -Pulm Toilet
     -Supplemental O2 to keep SaO2 >92%. 
 
#4 GI: miralax and docusate, advance diet as tolerated when nsgy ok
 
#5 Renal: Monitor for renal failure, replete lytes
 
#6 ID: monitor for infection
 
#7 Endo: keep FS 150-180
 
#8 Heme: Tx plats >100k hgb >7
 
#9 PPx: SCD and PPI"""

Op4 = """#1 Neuro: Monitor for delirium.
 
#2 CV: The pt presnets with concern for ACS.  Pt now s/p catheterization and PCI placed.  Continue ASA, plavix, statin, start metoprolol when HD stable.
 
#3 Pulm: Promote IS use, keep O2 to keep SaO2 >92%.   
 
#4 GI: Advance diet when ok with Cardiology

#5 Renal: Monitor for renal failure and CIN, replete lytes
 
#6 ID: monitor for infection
 
#7 Endo: keep FS 150-180
 
#8 Heme: Tx plats >10k hgb >7, check CBC
 
#9 PPx: HSQ and PPI"""

Op5 = """#1 Neuro: Multimodal analgesia.  Wean sedation as tolerated.
 
#2 CV: The pt Presents for CABGx POD 0.  Swan in place, titrate pressors and inotropes to maximize cardiac function.  
 
#3 Pulm: Pt remains intubated postop, will attempt to wean and consider extubation if patient meets parameters. CT in place, monitor for output and repeat CXR.
 
#4 GI: NPO for now

#5 Renal: Monitor for renal failure, replete lytes, follow nephrocheck
 
#6 ID: monitor for infection, cefazolin ppx
 
#7 Endo: keep FS 150-180
 
#8 Heme: Tx plats >10k hgb >7, check CBC
 
#9 PPx: SCD and PPI """

Op6 = """#1 Neuro: Multimodal analgesia.  Wean sedation as tolerated.
 
#2 CV: The pt Presents for AVR POD 0.  Swan in place, titrate pressors and inotropes to maximize cardiac function.  
 
#3 Pulm: Pt remains intubated postop, will attempt to wean and consider extubation if patient meets parameters. 
 
#4 GI: NPO for now

#5 Renal: Monitor for renal failure, replete lytes, follow nephrocheck
 
#6 ID: monitor for infection, cefazolin ppx
 
#7 Endo: keep FS 150-180
 
#8 Heme: Tx plats >10k hgb >7, check CBC
 
#9 PPx: SCD and PPI """

Op7 = """#1 Neuro: Multimodal analgesia.  Monitor neurochecks for LE weakness.
 
#2 CV: The pt is a HD stable at present s/p EVAR.  Cont blood pressure management, monitor LE motor function, maintain BP.
 
#3 Pulm: Promote IS use, keep O2 to keep SaO2 >92%.   
 
#4 GI: Advance diet when cleared by surgeon

#5 Renal: Monitor for renal failure, replete lytes
 
#6 ID: monitor for infection, cefazolin ppx
 
#7 Endo: keep FS 150-180
 
#8 Heme: Tx plats >10k hgb >7, check CBC
 
#9 PPx: HSQ and PPI """

Op8 = """#1 Neuro: Multimodal analgesia.  Wean sedation as tolerated.
 
#2 CV: The pt Presents for pericardial window POD 0.  Monitor drain for output and hemodynamic stability
 
#3 Pulm: Promote IS use, keep O2 to keep SaO2 >92%.
 
#4 GI: NPO for now

#5 Renal: Monitor for renal failure 
 
#6 ID: monitor for infection, cefazolin ppx
 
#7 Endo: keep FS 150-180
 
#8 Heme: Tx plats >10k hgb >7, check CBC
 
#9 PPx: SCD and PPI """

Op9 = """#1 Neuro: Multimodal analgesia.  Wean sedation as tolerated.
 
#2 CV: The pt Presents for MVR POD 0.  Swan in place, titrate pressors and inotropes to maximize cardiac function.  
 
#3 Pulm: Pt remains intubated postop, will attempt to wean and consider extubation if patient meets parameters. 
 
#4 GI: NPO for now

#5 Renal: Monitor for renal failure, replete lytes, follow nephrocheck
 
#6 ID: monitor for infection, cefazolin ppx
 
#7 Endo: keep FS 150-180
 
#8 Heme: Tx plats >10k hgb >7, check CBC
 
#9 PPx: SCD and PPI """

#data input display
if len(zp) > 0:
  if Na == None:
    Na = NaSodiumvec
  elif (NaSodiumvec == None and Na == None):
    Na = 140

  if K == None:
    K = KPotassiumvec
  elif (KPotassiumvec == None and K == None):
    K = 4

  if AG == None:
    AG = agAG
  elif (agAG == None and AG == None):
    AG = 10

  if Cr == None:
    Cr = CrCreatininevec
  elif (CrCreatininevec == None and Cr == None):
    Cr = 1
  
  if Hgb == None:
    Hgb = hgbHGBvec
  elif (hgbHGBvec == None and Hgb == None):
    Hgb = 12

  if Plt == None:
    Plt = pltPLTvec
  elif (pltPLTvec == None and Plt == None):
    Plt = 300

  if SBP == None:
    SBP = sbpSBPvec
  elif (sbpSBPvec == None and SBP == None):
    SBP = 120

  if aPTT == None:
    aPTT = aPTTpttvec
  elif (aPTTpttvec == None and aPTT == None):
    aPTT = 30

  if INR == None:
    INR = inrINRvec
  elif (inrINRvec == None and INR == None):
    INR = 1
else:
  if (Na == None):
    Na = 140

  if K == None:
    K = 4

  if AG == None:
    AG = 10

  if Cr == None:
    Cr = 1
  
  if Hgb == None:
    Hgb = 12

  if Plt == None:
    Plt = 300

  if SBP == None:
    SBP = 120

  if aPTT == None:
    aPTT = 30

  if INR == None:
    INR = 1


data = pd.DataFrame(data={'Number': [0], 'Note': [Note], 'Op Note': [Op_Note], 'Na': [Na], 'K': [K], 'AG': [AG], 'Cr': [Cr], 'Hgb': [Hgb]})
supportv = pd.DataFrame(data={'SBP': [SBP], 'Intubated': [Intubated], 'Plt': [Plt], 'INR': [INR], 'aPTT': [aPTT]})
cv = CountVectorizer(decode_error='replace', vocabulary=read_dat('HPNLP2featureMT15.pkl'))
data_cv = cv.fit_transform(data['Note'])   
cv1 = CountVectorizer(decode_error='replace', vocabulary=read_dat('Preop2featureMT15.pkl'))
data_cv1 = cv1.fit_transform(data['Op Note']) 
data_dtm = pd.DataFrame(data_cv.toarray(), columns=cv.get_feature_names())
data_dtm1 = pd.merge(data['Number'], data_dtm, right_index=True, left_index=True)
Z = ['Na', 'K', 'AG', 'Cr', 'Hgb'] 
Q = pd.DataFrame(data_cv1.toarray(), columns=cv1.get_feature_names())
X = pd.merge(data_dtm, data[Z], right_index=True, left_index=True)
XXX = pd.merge(data, supportv, right_index=True, left_index=True)

#dx calculator
from sklearn.model_selection import cross_val_score
from sklearn.metrics import make_scorer
from xgboost import plot_importance
#Class predictions
y_predxgbAbdominal_Pain = xgbAbdominal_Pain.predict_proba(X)[:,1]
y_predxgbACS = xgbACS.predict_proba(X)[:,1]
y_predxgbAAA = xgbAAA.predict_proba(X)[:,1]
y_predxgbTachycardia = xgbTachycardia.predict_proba(X)[:,1]
y_predxgbBradycardia = xgbBradycardia.predict_proba(X)[:,1]
y_predxgbCHF = xgbCHF.predict_proba(X)[:,1]
y_predxgbHTN_Emergency = xgbHTN_Emergency.predict_proba(X)[:,1]
y_predxgbHypotensionShock = xgbHypotensionShock.predict_proba(X)[:,1]
y_predxgbCardiac_arrest = xgbCardiac_arrest.predict_proba(X)[:,1]
y_predxgbAirway_Obstruction = xgbAirway_Obstruction.predict_proba(X)[:,1]
y_predxgbCOPD = xgbCOPD.predict_proba(X)[:,1]
y_predxgbPneumonia = xgbPneumonia.predict_proba(X)[:,1]
y_predxgbPneumothorax = xgbPneumothorax.predict_proba(X)[:,1]
y_predxgbRespiratory_Failure = xgbRespiratory_Failure.predict_proba(X)[:,1]
y_predxgbALF = xgbALF.predict_proba(X)[:,1]
y_predxgbGI_Bleed = xgbGI_Bleed.predict_proba(X)[:,1]
y_predxgbPancreatitis = xgbPancreatitis.predict_proba(X)[:,1]
y_predxgbColitis = xgbColitis.predict_proba(X)[:,1]
y_predxgbGastroenteritis = xgbGastroenteritis.predict_proba(X)[:,1]
y_predxgbBacteremia = xgbBacteremia.predict_proba(X)[:,1]
y_predxgbmeningitis_encephalitis = xgbmeningitis_encephalitis.predict_proba(X)[:,1]
y_predxgbEndocarditis = xgbEndocarditis.predict_proba(X)[:,1]
y_predxgbUTI = xgbUTI.predict_proba(X)[:,1]
y_predxgbafib = xgbafib.predict_proba(X)[:,1]
y_predxgbvented = xgbvented.predict_proba(X)[:,1]
y_predxgbDM = xgbDM.predict_proba(X)[:,1]
y_predxgbDKA = xgbDKA.predict_proba(X)[:,1]
y_predxgbICH = xgbICH.predict_proba(X)[:,1]
y_predxgbSDH = xgbSDH.predict_proba(X)[:,1]
y_predxgbSAH = xgbSAH.predict_proba(X)[:,1]
y_predxgbSeizure = xgbSeizure.predict_proba(X)[:,1]
y_predxgbPE = xgbPE.predict_proba(X)[:,1]
y_predxgbESRD = xgbESRD.predict_proba(X)[:,1]
y_predxgbCKD = xgbCKD.predict_proba(X)[:,1]
y_predxgbAKI = xgbAKI.predict_proba(X)[:,1]
y_predxgbHTN = xgbHTN.predict_proba(X)[:,1]
y_predxgbAsthma = xgbAsthma.predict_proba(X)[:,1]
y_predxgbEtOH_WD = xgbEtOH_WD.predict_proba(X)[:,1]
y_predxgbCAD = xgbCAD.predict_proba(X)[:,1]
y_predxgbOSA = xgbOSA.predict_proba(X)[:,1]
y_predxgbhypothyroid = xgbhypothyroid.predict_proba(X)[:,1]
y_predxgbhyperthyroid = xgbhyperthyroid.predict_proba(X)[:,1]
y_predxgbCVA = xgbCVA.predict_proba(X)[:,1]
y_predxgbDelirium = xgbDelirium.predict_proba(X)[:,1]
y_predxgbDLD = xgbDLD.predict_proba(X)[:,1]
y_predxgbAI = xgbAI.predict_proba(X)[:,1]
y_predxgbLupus = xgbLupus.predict_proba(X)[:,1]
y_predxgbCushing = xgbCushing.predict_proba(X)[:,1]
y_predxgbScleroderma = xgbScleroderma.predict_proba(X)[:,1]
y_predxgbPericardial_Effusion = xgbPericardial_Effusion.predict_proba(X)[:,1]
y_predxgbCholecystitis = xgbCholecystitis.predict_proba(X)[:,1]
y_predxgbARDS = xgbARDS.predict_proba(X)[:,1]
y_predxgbCellulitis = xgbCellulitis.predict_proba(X)[:,1]
y_predxgbTBI = xgbTBI.predict_proba(X)[:,1]
y_predxgbBurn = xgbBurn.predict_proba(X)[:,1]
y_predxgbPAD = xgbPAD.predict_proba(X)[:,1] 
y_predxgbBrain_Tumor = xgbBrain_Tumor.predict_proba(X)[:,1]
y_predxgbGERD = xgbGERD.predict_proba(X)[:,1]
y_predxgbpreopUEAngio = xgbpreopUEAngio.predict_proba(Q)[:,1]
y_predxgbpreopImpella = xgbpreopImpella.predict_proba(Q)[:,1]
y_predxgbpreopCraniotomy = xgbpreopCraniotomy.predict_proba(Q)[:,1]
y_predxgbpreopCath = xgbpreopCath.predict_proba(Q)[:,1]
y_predxgbpreopCABG = xgbpreopCABG.predict_proba(Q)[:,1]
y_predxgbpreopAVR = xgbpreopAVR.predict_proba(Q)[:,1]
y_predxgbpreopAngiogram = xgbpreopAngiogram.predict_proba(Q)[:,1]
y_predxgbpreopAAA = xgbpreopAAA.predict_proba(Q)[:,1]

#medspacy history and negation calculator (negation dormant)

doc = nlp(Note) 
docop = nlp(Op_Note) 

op = pd.DataFrame(docop._.context_graph.edges)
pp = pd.DataFrame(doc._.context_graph.edges)
if len(pp) > 0:
  pp.columns=['column1', 'column2']
  pp['column1'] = pp['column1'].astype(str)
  pp['column2'] = pp['column2'].astype(str)
  selected_rowshist = pp.loc[pp['column2'].str.contains("HISTORICAL")]
  selected_rowshist['column1'] = selected_rowshist['column1'].astype(str)
  hist = list(selected_rowshist['column1'])
  selected_rowsneg = pp.loc[pp['column2'].str.contains("NEGATED_EXISTENCE")]
  selected_rowsneg['column1'] = selected_rowsneg['column1'].astype(str)
  neg = list(selected_rowsneg['column1'])
  #hist

  flag=0
  for i in neg:
      for j in ACS:
          if i==j:
              flag=1
              break
  if flag==1:
      ynegACS = 1
  else:
      ynegACS = 0

  flag=0
  for i in neg:
      for j in AAA:
          if i==j:
              flag=1
              break
  if flag==1:
      ynegAAA = 1
  else:
      ynegAAA = 0
      
  flag=0
  for i in neg:
      for j in Tachycardia:
          if i==j:
              flag=1
              break
  if flag==1:
      ynegTachycardia = 1
  else:
      ynegTachycardia = 0
      
  flag=0
  for i in neg:
      for j in Bradycardia:
          if i==j:
              flag=1
              break
  if flag==1:
      ynegBradycardia = 1
  else:
      ynegBradycardia = 0
      
  flag=0
  for i in neg:
      for j in CHF:
          if i==j:
              flag=1
              break
  if flag==1:
      ynegCHF = 1
  else:
      ynegCHF = 0
      
  flag=0
  for i in neg:
      for j in HTN_Emergency:
          if i==j:
              flag=1
              break
  if flag==1:
      ynegHTN_Emergency = 1
  else:
      ynegHTN_Emergency = 0
      
  flag=0
  for i in neg:
      for j in HypotensionShock:
          if i==j:
              flag=1
              break
  if flag==1:
      ynegHypotensionShock = 1
  else:
      ynegHypotensionShock = 0
      
  flag=0
  for i in neg:
      for j in Cardiac_arrest:
          if i==j:
              flag=1
              break
  if flag==1:
      ynegCardiac_arrest = 1
  else:
      ynegCardiac_arrest = 0

  flag=0
  for i in neg:
      for j in Airway_Obstruction:
          if i==j:
              flag=1
              break
  if flag==1:
      ynegAirway_Obstruction = 1
  else:
      ynegAirway_Obstruction = 0

  flag=0
  for i in neg:
      for j in Pneumonia:
          if i==j:
              flag=1
              break
  if flag==1:
      ynegPneumonia = 1
  else:
      ynegPneumonia = 0

  flag=0
  for i in neg:
      for j in COPD:
          if i==j:
              flag=1
              break
  if flag==1:
      ynegCOPD = 1
  else:
      ynegCOPD = 0

  flag=0
  for i in neg:
      for j in Pneumothorax:
          if i==j:
              flag=1
              break
  if flag==1:
      ynegPneumothorax = 1
  else:
      ynegPneumothorax = 0
      
  flag=0
  for i in neg:
      for j in Respiratory_Failure:
          if i==j:
              flag=1
              break
  if flag==1:
      ynegRespiratory_Failure = 1
  else:
      ynegRespiratory_Failure = 0

  flag=0
  for i in neg:
      for j in GI_Bleed:
          if i==j:
              flag=1
              break
  if flag==1:
      ynegGI_Bleed = 1
  else:
      ynegGI_Bleed = 0
      
  flag=0
  for i in neg:
      for j in ALF:
          if i==j:
              flag=1
              break
  if flag==1:
      ynegALF = 1
  else:
      ynegALF = 0
      
  flag=0
  for i in neg:
      for j in Pancreatitis:
          if i==j:
              flag=1
              break
  if flag==1:
      ynegPancreatitis = 1
  else:
      ynegPancreatitis = 0
      
  flag=0
  for i in neg:
      for j in Colitis:
          if i==j:
              flag=1
              break
  if flag==1:
      ynegColitis = 1
  else:
      ynegColitis = 0
      
  flag=0
  for i in neg:
      for j in Gastroenteritis:
          if i==j:
              flag=1
              break
  if flag==1:
      ynegGastroenteritis = 1
  else:
      ynegGastroenteritis = 0
      
  flag=0
  for i in neg:
      for j in meningitis_encephalitis:
          if i==j:
              flag=1
              break
  if flag==1:
      ynegmeningitis_encephalitis = 1
  else:
      ynegmeningitis_encephalitis = 0
      
  flag=0
  for i in neg:
      for j in Bacteremia:
          if i==j:
              flag=1
              break
  if flag==1:
      ynegBacteremia = 1
  else:
      ynegBacteremia = 0
      
  flag=0
  for i in neg:
      for j in Endocarditis:
          if i==j:
              flag=1
              break
  if flag==1:
      ynegEndocarditis = 1
  else:
      ynegEndocarditis = 0
  
  flag=0
  for i in neg:
      for j in UTI:
          if i==j:
              flag=1
              break
  if flag==1:
      ynegUTI = 1
  else:
      ynegUTI = 0
      
  flag=0
  for i in neg:
      for j in vented:
          if i==j:
              flag=1
              break
  if flag==1:
      ynegvented = 1
  else:
      ynegvented = 0
      
  flag=0
  for i in neg:
      for j in Afib:
          if i==j:
              flag=1
              break
  if flag==1:
      ynegafib = 1
  else:
      ynegafib = 0
      
  flag=0
  for i in neg:
      for j in DM:
          if i==j:
              flag=1
              break
  if flag==1:
      ynegDM = 1
  else:
      ynegDM = 0
      
  flag=0
  for i in neg:
      for j in DKA:
          if i==j:
              flag=1
              break
  if flag==1:
      ynegDKA = 1
  else:
      ynegDKA = 0

  flag=0
  for i in neg:
      for j in PE:
          if i==j:
              flag=1
              break
  if flag==1:
      ynegPE = 1
  else:
      ynegPE = 0
      
  flag=0
  for i in neg:
      for j in ICH:
          if i==j:
              flag=1
              break
  if flag==1:
      ynegICH = 1
  else:
      ynegICH = 0
      
  flag=0
  for i in neg:
      for j in SDH:
          if i==j:
              flag=1
              break
  if flag==1:
      ynegSDH = 1
  else:
      ynegSDH = 0
      
  flag=0
  for i in neg:
      for j in SAH:
          if i==j:
              flag=1
              break
  if flag==1:
      ynegSAH = 1
  else:
      ynegSAH = 0
      
  flag=0
  for i in neg:
      for j in Seizures:
          if i==j:
              flag=1
              break
  if flag==1:
      ynegSeizure = 1
  else:    ynegSeizure = 0

  flag=0
  for i in neg:
      for j in ESRD:
          if i==j:
              flag=1
              break
  if flag==1:
      ynegESRD = 1
  else:
      ynegESRD = 0
      
  flag=0
  for i in neg:
      for j in CKD:
          if i==j:
              flag=1
              break
  if flag==1:
      ynegCKD = 1
  else:
      ynegCKD = 0
      
  flag=0
  for i in neg:
      for j in AKI:
          if i==j:
              flag=1
              break
  if flag==1:
      ynegAKI = 1
  else:
      ynegAKI = 0

  flag=0
  for i in neg:
      for j in HTN:
          if i==j:
              flag=1
              break
  if flag==1:
      ynegHTN = 1
  else:
      ynegHTN = 0

  flag=0
  for i in neg:
      for j in Asthma:
          if i==j:
              flag=1
              break
  if flag==1:
      ynegAsthma = 1
  else:
      ynegAsthma = 0

  flag=0
  for i in neg:
      for j in EtOH_WD:
          if i==j:
              flag=1
              break
  if flag==1:
      ynegEtOH_WD = 1
  else:
      ynegEtOH_WD = 0

  flag=0
  for i in neg:
      for j in CAD:
          if i==j:
              flag=1
              break
  if flag==1:
      ynegCAD = 1
  else:
      ynegCAD = 0

  flag=0
  for i in neg:
      for j in OSA:
          if i==j:
              flag=1
              break
  if flag==1:
      ynegOSA = 1
  else:
      ynegOSA = 0

  flag=0
  for i in neg:
      for j in Hypothyroid:
          if i==j:
              flag=1
              break
  if flag==1:
      yneghypothyroid = 1
  else:
      yneghypothyroid = 0

  flag=0
  for i in neg:
      for j in Hyperthyroid:
          if i==j:
              flag=1
              break
  if flag==1:
      yneghyperthyroid = 1
  else:
      yneghyperthyroid = 0

  flag=0
  for i in neg:
      for j in AI:
          if i==j:
              flag=1
              break
  if flag==1:
      ynegAI = 1
  else:
      ynegAI = 0

  flag=0
  for i in neg:
      for j in Cushing:
          if i==j:
              flag=1
              break
  if flag==1:
      ynegCushing = 1
  else:
      ynegCushing = 0

  flag=0
  for i in neg:
      for j in CVA:
          if i==j:
              flag=1
              break
  if flag==1:
      ynegCVA = 1
  else:
      ynegCVA  = 0

  flag=0
  for i in neg:
      for j in Delirium:
          if i==j:
              flag=1
              break
  if flag==1:
      ynegDelirium = 1
  else:
      ynegDelirium = 0

  flag=0
  for i in neg:
      for j in DLD:
          if i==j:
              flag=1
              break
  if flag==1:
      ynegDLD = 1
  else:
      ynegDLD = 0

  flag=0
  for i in neg:
      for j in Lupus:
          if i==j:
              flag=1
              break
  if flag==1:
      ynegLupus = 1
  else:
      ynegLupus = 0

  flag=0
  for i in neg:
      for j in Scleroderma:
          if i==j:
              flag=1
              break
  if flag==1:
      ynegScleroderma = 1
  else:
      ynegScleroderma = 0

  flag=0
  for i in neg:
      for j in Pericardial_Effusion:
          if i==j:
              flag=1
              break
  if flag==1:
      ynegPericardial_Effusion = 1
  else:
      ynegPericardial_Effusion = 0

  flag=0
  for i in neg:
      for j in Cholecystitis:
          if i==j:
              flag=1
              break
  if flag==1:
      ynegCholecystitis = 1
  else:
      ynegCholecystitis = 0

  flag=0
  for i in neg:
      for j in ARDS:
          if i==j:
              flag=1
              break
  if flag==1:
      ynegARDS = 1
  else:
      ynegARDS = 0
  
  flag=0
  for i in neg:
      for j in Cellulitis:
          if i==j:
              flag=1
              break
  if flag==1:
      ynegCellulitis = 1
  else:
      ynegCellulitis = 0

  flag=0
  for i in neg:
      for j in TBI:
          if i==j:
              flag=1
              break
  if flag==1:
      ynegTBI = 1
  else:
      ynegTBI = 0

  flag=0
  for i in neg:
      for j in Burns:
          if i==j:
              flag=1
              break
  if flag==1:
      ynegBurn = 1
  else:
      ynegBurn = 0

  flag=0
  for i in neg:
      for j in PAD:
          if i==j:
              flag=1
              break
  if flag==1:
      ynegPAD = 1
  else:
      ynegPAD = 0 

  flag=0
  for i in neg:
      for j in Brain_Tumor:
          if i==j:
              flag=1
              break
  if flag==1:
      ynegBrain_Tumor = 1
  else:
      ynegBrain_Tumor = 0

  flag=0
  for i in neg:
      for j in GERD:
          if i==j:
              flag=1
              break
  if flag==1:
      ynegGERD = 1
  else:
      ynegGERD = 0

  flag=0
  for i in neg:
      for j in AAA:
          if i==j:
              flag=1
              break
  if flag==1:
      ynegpreopAAA = 1
  else:
      ynegpreopAAA = 0



  flag=0
  for i in hist:
      for j in ACS:
          if i==j:
              flag=1
              break
  if flag==1:
      yhistACS = 1
  else:
      yhistACS = 0

  flag=0
  for i in hist:
      for j in AAA:
          if i==j:
              flag=1
              break
  if flag==1:
      yhistAAA = 1
  else:
      yhistAAA = 0
      
  flag=0
  for i in hist:
      for j in Tachycardia:
          if i==j:
              flag=1
              break
  if flag==1:
      yhistTachycardia = 1
  else:
      yhistTachycardia = 0
      
  flag=0
  for i in hist:
      for j in Bradycardia:
          if i==j:
              flag=1
              break
  if flag==1:
      yhistBradycardia = 1
  else:
      yhistBradycardia = 0
      
  flag=0
  for i in hist:
      for j in CHF:
          if i==j:
              flag=1
              break
  if flag==1:
      yhistCHF = 1
  else:
      yhistCHF = 0
      
  flag=0
  for i in hist:
      for j in HTN_Emergency:
          if i==j:
              flag=1
              break
  if flag==1:
      yhistHTN_Emergency = 1
  else:
      yhistHTN_Emergency = 0
      
  flag=0
  for i in hist:
      for j in HypotensionShock:
          if i==j:
              flag=1
              break
  if flag==1:
      yhistHypotensionShock = 1
  else:
      yhistHypotensionShock = 0
      
  flag=0
  for i in hist:
      for j in Cardiac_arrest:
          if i==j:
              flag=1
              break
  if flag==1:
      yhistCardiac_arrest = 1
  else:
      yhistCardiac_arrest = 0

  flag=0
  for i in hist:
      for j in Airway_Obstruction:
          if i==j:
              flag=1
              break
  if flag==1:
      yhistAirway_Obstruction = 1
  else:
      yhistAirway_Obstruction = 0

  flag=0
  for i in hist:
      for j in Pneumonia:
          if i==j:
              flag=1
              break
  if flag==1:
      yhistPneumonia = 1
  else:
      yhistPneumonia = 0

  flag=0
  for i in hist:
      for j in COPD:
          if i==j:
              flag=1
              break
  if flag==1:
      yhistCOPD = 1
  else:
      yhistCOPD = 0

  flag=0
  for i in hist:
      for j in Pneumothorax:
          if i==j:
              flag=1
              break
  if flag==1:
      yhistPneumothorax = 1
  else:
      yhistPneumothorax = 0
      
  flag=0
  for i in hist:
      for j in Respiratory_Failure:
          if i==j:
              flag=1
              break
  if flag==1:
      yhistRespiratory_Failure = 1
  else:
      yhistRespiratory_Failure = 0

  flag=0
  for i in hist:
      for j in GI_Bleed:
          if i==j:
              flag=1
              break
  if flag==1:
      yhistGI_Bleed = 1
  else:
      yhistGI_Bleed = 0
      
  flag=0
  for i in hist:
      for j in ALF:
          if i==j:
              flag=1
              break
  if flag==1:
      yhistALF = 1
  else:
      yhistALF = 0
      
  flag=0
  for i in hist:
      for j in Pancreatitis:
          if i==j:
              flag=1
              break
  if flag==1:
      yhistPancreatitis = 1
  else:
      yhistPancreatitis = 0
      
  flag=0
  for i in hist:
      for j in Colitis:
          if i==j:
              flag=1
              break
  if flag==1:
      yhistColitis = 1
  else:
      yhistColitis = 0
      
  flag=0
  for i in hist:
      for j in Gastroenteritis:
          if i==j:
              flag=1
              break
  if flag==1:
      yhistGastroenteritis = 1
  else:
      yhistGastroenteritis = 0
      
  flag=0
  for i in hist:
      for j in meningitis_encephalitis:
          if i==j:
              flag=1
              break
  if flag==1:
      yhistmeningitis_encephalitis = 1
  else:
      yhistmeningitis_encephalitis = 0
      
  flag=0
  for i in hist:
      for j in Bacteremia:
          if i==j:
              flag=1
              break
  if flag==1:
      yhistBacteremia = 1
  else:
      yhistBacteremia = 0
      
  flag=0
  for i in hist:
      for j in Endocarditis:
          if i==j:
              flag=1
              break
  if flag==1:
      yhistEndocarditis = 1
  else:
      yhistEndocarditis = 0
  
  flag=0
  for i in hist:
      for j in UTI:
          if i==j:
              flag=1
              break
  if flag==1:
      yhistUTI = 1
  else:
      yhistUTI = 0
      
  flag=0
  for i in hist:
      for j in vented:
          if i==j:
              flag=1
              break
  if flag==1:
      yhistvented = 1
  else:
      yhistvented = 0
      
  flag=0
  for i in hist:
      for j in Afib:
          if i==j:
              flag=1
              break
  if flag==1:
      yhistafib = 1
  else:
      yhistafib = 0
      
  flag=0
  for i in hist:
      for j in DM:
          if i==j:
              flag=1
              break
  if flag==1:
      yhistDM = 1
  else:
      yhistDM = 0
      
  flag=0
  for i in hist:
      for j in DKA:
          if i==j:
              flag=1
              break
  if flag==1:
      yhistDKA = 1
  else:
      yhistDKA = 0

  flag=0
  for i in hist:
      for j in PE:
          if i==j:
              flag=1
              break
  if flag==1:
      yhistPE = 1
  else:
      yhistPE = 0
      
  flag=0
  for i in hist:
      for j in ICH:
          if i==j:
              flag=1
              break
  if flag==1:
      yhistICH = 1
  else:
      yhistICH = 0
      
  flag=0
  for i in hist:
      for j in SDH:
          if i==j:
              flag=1
              break
  if flag==1:
      yhistSDH = 1
  else:
      yhistSDH = 0
      
  flag=0
  for i in hist:
      for j in SAH:
          if i==j:
              flag=1
              break
  if flag==1:
      yhistSAH = 1
  else:
      yhistSAH = 0
      
  flag=0
  for i in hist:
      for j in Seizures:
          if i==j:
              flag=1
              break
  if flag==1:
      yhistSeizure = 1
  else:    yhistSeizure = 0

  flag=0
  for i in hist:
      for j in ESRD:
          if i==j:
              flag=1
              break
  if flag==1:
      yhistESRD = 1
  else:
      yhistESRD = 0
      
  flag=0
  for i in hist:
      for j in CKD:
          if i==j:
              flag=1
              break
  if flag==1:
      yhistCKD = 1
  else:
      yhistCKD = 0
      
  flag=0
  for i in hist:
      for j in AKI:
          if i==j:
              flag=1
              break
  if flag==1:
      yhistAKI = 1
  else:
      yhistAKI = 0

  flag=0
  for i in hist:
      for j in HTN:
          if i==j:
              flag=1
              break
  if flag==1:
      yhistHTN = 1
  else:
      yhistHTN = 0

  flag=0
  for i in hist:
      for j in Asthma:
          if i==j:
              flag=1
              break
  if flag==1:
      yhistAsthma = 1
  else:
      yhistAsthma = 0

  flag=0
  for i in hist:
      for j in EtOH_WD:
          if i==j:
              flag=1
              break
  if flag==1:
      yhistEtOH_WD = 1
  else:
      yhistEtOH_WD = 0

  flag=0
  for i in hist:
      for j in CAD:
          if i==j:
              flag=1
              break
  if flag==1:
      yhistCAD = 1
  else:
      yhistCAD = 0

  flag=0
  for i in hist:
      for j in OSA:
          if i==j:
              flag=1
              break
  if flag==1:
      yhistOSA = 1
  else:
      yhistOSA = 0

  flag=0
  for i in hist:
      for j in Hypothyroid:
          if i==j:
              flag=1
              break
  if flag==1:
      yhisthypothyroid = 1
  else:
      yhisthypothyroid = 0

  flag=0
  for i in hist:
      for j in Hyperthyroid:
          if i==j:
              flag=1
              break
  if flag==1:
      yhisthyperthyroid = 1
  else:
      yhisthyperthyroid = 0

  flag=0
  for i in hist:
      for j in AI:
          if i==j:
              flag=1
              break
  if flag==1:
      yhistAI = 1
  else:
      yhistAI = 0

  flag=0
  for i in hist:
      for j in Cushing:
          if i==j:
              flag=1
              break
  if flag==1:
      yhistCushing = 1
  else:
      yhistCushing = 0

  flag=0
  for i in hist:
      for j in CVA:
          if i==j:
              flag=1
              break
  if flag==1:
      yhistCVA = 1
  else:
      yhistCVA  = 0

  flag=0
  for i in hist:
      for j in Delirium:
          if i==j:
              flag=1
              break
  if flag==1:
      yhistDelirium = 1
  else:
      yhistDelirium = 0

  flag=0
  for i in hist:
      for j in DLD:
          if i==j:
              flag=1
              break
  if flag==1:
      yhistDLD = 1
  else:
      yhistDLD = 0

  flag=0
  for i in hist:
      for j in Lupus:
          if i==j:
              flag=1
              break
  if flag==1:
      yhistLupus = 1
  else:
      yhistLupus = 0

  flag=0
  for i in hist:
      for j in Scleroderma:
          if i==j:
              flag=1
              break
  if flag==1:
      yhistScleroderma = 1
  else:
      yhistScleroderma = 0

  flag=0
  for i in hist:
      for j in Pericardial_Effusion:
          if i==j:
              flag=1
              break
  if flag==1:
      yhistPericardial_Effusion = 1
  else:
      yhistPericardial_Effusion = 0

  flag=0
  for i in hist:
      for j in Cholecystitis:
          if i==j:
              flag=1
              break
  if flag==1:
      yhistCholecystitis = 1
  else:
      yhistCholecystitis = 0

  flag=0
  for i in hist:
      for j in ARDS:
          if i==j:
              flag=1
              break
  if flag==1:
      yhistARDS = 1
  else:
      yhistARDS = 0
  
  flag=0
  for i in hist:
      for j in Cellulitis:
          if i==j:
              flag=1
              break
  if flag==1:
      yhistCellulitis = 1
  else:
      yhistCellulitis = 0

  flag=0
  for i in hist:
      for j in TBI:
          if i==j:
              flag=1
              break
  if flag==1:
      yhistTBI = 1
  else:
      yhistTBI = 0

  flag=0
  for i in hist:
      for j in Burns:
          if i==j:
              flag=1
              break
  if flag==1:
      yhistBurn = 1
  else:
      yhistBurn = 0

  flag=0
  for i in hist:
      for j in PAD:
          if i==j:
              flag=1
              break
  if flag==1:
      yhistPAD = 1
  else:
      yhistPAD = 0 

  flag=0
  for i in hist:
      for j in Brain_Tumor:
          if i==j:
              flag=1
              break
  if flag==1:
      yhistBrain_Tumor = 1
  else:
      yhistBrain_Tumor = 0

  flag=0
  for i in hist:
      for j in GERD:
          if i==j:
              flag=1
              break
  if flag==1:
      yhistGERD = 1
  else:
      yhistGERD = 0

  flag=0
  for i in hist:
      for j in AAA:
          if i==j:
              flag=1
              break
  if flag==1:
      yhistpreopAAA = 1
  else:
      yhistpreopAAA = 0

  flag=0
  for i in hist:
      for j in trach:
          if i==j:
              flag=1
              break
  if flag==1:
      yhisttrach = 1
  else:
      yhisttrach = 0

  flag=0
  for i in hist:
      for j in pleural_effusion:
          if i==j:
              flag=1
              break
  if flag==1:
      yhistpleural_effusion = 1
  else:
      yhistpleural_effusion = 0
else:
  yhistACS = 0
  yhistAAA = 0
  yhistTachycardia = 0
  yhistBradycardia = 0
  yhistCHF = 0
  yhistHTN_Emergency = 0
  yhistHypotensionShock = 0
  yhistCardiac_arrest = 0
  yhistAirway_Obstruction = 0
  yhistPneumonia = 0
  yhistCOPD = 0
  yhistPneumothorax = 0
  yhistRespiratory_Failure = 0
  yhistGI_Bleed = 0
  yhistALF = 0
  yhistPancreatitis = 0
  yhistColitis = 0
  yhistGastroenteritis = 0
  yhistmeningitis_encephalitis = 0
  yhistBacteremia = 0
  yhistEndocarditis = 0
  yhistUTI = 0
  yhistvented = 0
  yhistafib = 0    
  yhistDM = 0
  yhistDKA = 0
  yhistPE = 0
  yhistICH = 0
  yhistSDH = 0
  yhistSAH = 0
  yhistSeizure = 0
  yhistESRD = 0
  yhistCKD = 0
  yhistAKI = 0
  yhistHTN = 0
  yhistAsthma = 0
  yhistEtOH_WD = 0
  yhistCAD = 0
  yhistOSA = 0
  yhisthypothyroid = 0
  yhisthyperthyroid = 0
  yhistAI = 0
  yhistCushing = 0
  yhistCVA  = 0
  yhistDelirium = 0
  yhistDLD = 0
  yhistLupus = 0
  yhistScleroderma = 0
  yhistPericardial_Effusion = 0
  yhistCholecystitis = 0
  yhistARDS = 0
  yhistCellulitis = 0
  yhistTBI = 0
  yhistBurn = 0
  yhistPAD = 0 
  yhistBrain_Tumor = 0
  yhistGERD = 0
  yhistpreopAAA = 0
  yhisttrach = 0
  yhistpleural_effusion = 0
  ynegACS = 0
  ynegAAA = 0
  ynegTachycardia = 0
  ynegBradycardia = 0
  ynegCHF = 0
  ynegHTN_Emergency = 0
  ynegHypotensionShock = 0
  ynegCardiac_arrest = 0
  ynegAirway_Obstruction = 0
  ynegPneumonia = 0
  ynegCOPD = 0
  ynegPneumothorax = 0
  ynegRespiratory_Failure = 0
  ynegGI_Bleed = 0
  ynegALF = 0
  ynegPancreatitis = 0
  ynegColitis = 0
  ynegGastroenteritis = 0
  ynegmeningitis_encephalitis = 0
  ynegBacteremia = 0
  ynegEndocarditis = 0
  ynegUTI = 0
  ynegvented = 0
  ynegafib = 0    
  ynegDM = 0
  ynegDKA = 0
  ynegPE = 0
  ynegICH = 0
  ynegSDH = 0
  ynegSAH = 0
  ynegSeizure = 0
  ynegESRD = 0
  ynegCKD = 0
  ynegAKI = 0
  ynegHTN = 0
  ynegAsthma = 0
  ynegEtOH_WD = 0
  ynegCAD = 0
  ynegOSA = 0
  yneghypothyroid = 0
  yneghyperthyroid = 0
  ynegAI = 0
  ynegCushing = 0
  ynegCVA  = 0
  ynegDelirium = 0
  ynegDLD = 0
  ynegLupus = 0
  ynegScleroderma = 0
  ynegPericardial_Effusion = 0
  ynegCholecystitis = 0
  ynegARDS = 0
  ynegCellulitis = 0
  ynegTBI = 0
  ynegBurn = 0
  ynegPAD = 0 
  ynegBrain_Tumor = 0
  ynegGERD = 0
  ynegpreopAAA = 0
  ynegtrach = 0
  ynegpleural_effusion = 0      
     
#NLTK Dx calculator
if y_predxgbACS > .4:
    y_predACS = 1
else:
    y_predACS = 0
    
if y_predxgbAAA > .41:
    y_predAAA = 1
else:
    y_predAAA = 0
    
if y_predxgbTachycardia > .4:
    y_predTachycardia = 1
else:
    y_predTachycardia = 0
    
if y_predxgbBradycardia > .4:
    y_predBradycardia = 1
else:
    y_predBradycardia = 0
    
if (y_predxgbCHF > .2 or yhistCHF > .4):
    y_predCHF = 1
else:
    y_predCHF = 0
    
if (y_predxgbHTN_Emergency > .1 or supportv['SBP'][0] >= 195):
    y_predHTN_Emergency = 1
else:
    y_predHTN_Emergency = 0
    
if (y_predxgbHypotensionShock > .4 and supportv['SBP'][0] <= 80) or (supportv['SBP'][0] <= 80):
    y_predHypotensionShock = 1
else:
    y_predHypotensionShock = 0
    
if y_predxgbCardiac_arrest > .4:
    y_predCardiac_arrest = 1
else:
    y_predCardiac_arrest = 0

if y_predxgbAirway_Obstruction > .2:
    y_predAirway_Obstruction = 1
else:
    y_predAirway_Obstruction = 0

if y_predxgbPneumonia > .3:
    y_predPneumonia = 1
else:
    y_predPneumonia = 0

if y_predxgbCOPD > .3:
    y_predCOPD = 1
else:
    y_predCOPD = 0

if y_predxgbPneumothorax > .4:
    y_predPneumothorax = 1
else:
    y_predPneumothorax = 0
    
if (y_predxgbRespiratory_Failure > .4 or y_predxgbvented > .4):
    y_predRespiratory_Failure = 1
else:
    y_predRespiratory_Failure = 0

if y_predxgbGI_Bleed > .4:
    y_predGI_Bleed = 1
else:
    y_predGI_Bleed = 0
    
if y_predxgbALF > .2:
    y_predALF = 1
else:
    y_predALF = 0
    
if y_predxgbPancreatitis > .3:
    y_predPancreatitis = 1
else:
    y_predPancreatitis = 0
    
if y_predxgbColitis > .4:
    y_predColitis = 1
else:
    y_predColitis = 0
    
if y_predxgbGastroenteritis  > .4:
    y_predGastroenteritis = 1
else:
    y_predGastroenteritis = 0
    
if y_predxgbmeningitis_encephalitis > .3:
    y_predmeningitis_encephalitis = 1
else:
    y_predmeningitis_encephalitis = 0
    
if y_predxgbBacteremia > .4:
    y_predBacteremia = 1
else:
    y_predBacteremia = 0
    
if y_predxgbEndocarditis > .4:
    y_predEndocarditis = 1
else:
    y_predEndocarditis = 0
 
if y_predxgbUTI > .2:
    y_predUTI = 1
else:
    y_predUTI = 0
    
if supportv['Intubated'][0] > .4:
    y_predvented = 1
else:
    y_predvented = 0
    
if y_predxgbafib > .4:
    y_predafib = 1
else:
    y_predafib = 0
    
if (y_predxgbDM > .4 or yhistDM >.4):
    y_predDM = 1
else:
    y_predDM = 0
    
if (y_predxgbDKA > .4 and data['AG'][0] > 15):
    y_predDKA = 1
else:
    y_predDKA = 0

if y_predxgbPE > .4:
    y_predPE = 1
else:
    y_predPE = 0
    
if y_predxgbICH > .4:
    y_predICH = 1
else:
    y_predICH = 0
    
if y_predxgbSDH > .4:
    y_predSDH = 1
else:
    y_predSDH = 0
    
if y_predxgbSAH > .4:
    y_predSAH = 1
else:
    y_predSAH = 0
    
if y_predxgbSeizure > .4:
    y_predSeizure = 1
else:
    y_predSeizure = 0

if (y_predxgbESRD > .4 or yhistESRD > .4) and (data['Cr'][0] >= 2):
    y_predESRD = 1
else:
    y_predESRD = 0
    
if (y_predxgbCKD > .4 or yhistCKD > .4) and (data['Cr'][0] >= 1.4):
    y_predCKD = 1
else:
    y_predCKD = 0
    
if (y_predxgbAKI > .4 and data['Cr'][0] >= 1.4):
    y_predAKI = 1
else:
    y_predAKI = 0
    
if y_predxgbAbdominal_Pain > .4:
    y_predAbdominal_Pain = 1
else:
    y_predAbdominal_Pain = 0

if (y_predxgbHTN > .4 or yhistHTN > .4 or supportv['SBP'][0] >= 150):
    y_predHTN = 1
else:
    y_predHTN = 0

if y_predxgbAsthma > .4:
    y_predAsthma = 1
else:
    y_predAsthma = 0

if y_predxgbEtOH_WD > .4:
    y_predEtOH_WD = 1
else:
    y_predEtOH_WD = 0

if (y_predxgbCAD > .4 or yhistCAD > .4 or yhistACS > .4):
    y_predCAD = 1
else:
    y_predCAD = 0

if (y_predxgbOSA > .4 or yhistOSA > .4):
    y_predOSA = 1
else:
    y_predOSA = 0

if (y_predxgbhypothyroid > .4 or yhisthypothyroid > .4):
    y_predhypothyroid = 1
else:
    y_predhypothyroid = 0

if (y_predxgbhyperthyroid > .4 or yhisthyperthyroid > .4):
    y_predhyperthyroid = 1
else:
    y_predhyperthyroid = 0

if (y_predxgbAI > .4):
    y_predAI = 1
else:
    y_predAI = 0

if (y_predxgbCushing > .4 or yhistCushing > .4):
    y_predCushing = 1
else:
    y_predCushing = 0

if y_predxgbCVA > .4:
    y_predCVA = 1
else:
    y_predCVA  = 0

if y_predxgbDelirium > .4:
    y_predDelirium = 1
else:
    y_predDelirium = 0

if (y_predxgbDLD > .4 or yhistDLD > .4):
    y_predDLD = 1
else:
    y_predDLD = 0

if (y_predxgbLupus > .4 or yhistLupus > .4):
    y_predLupus = 1
else:
    y_predLupus = 0

if (y_predxgbScleroderma > .4 or yhistScleroderma >.4):
    y_predScleroderma = 1
else:
    y_predScleroderma = 0

if y_predxgbPericardial_Effusion > .4:
    y_predPericardial_Effusion = 1
else:
    y_predPericardial_Effusion = 0

if y_predxgbCholecystitis > .4:
    y_predCholecystitis = 1
else:
    y_predCholecystitis = 0

if (y_predxgbARDS > .4 and Intubated > .4):
    y_predARDS = 1
else:
    y_predARDS = 0

if (supportv['Plt'][0] < 50 or supportv['INR'][0] > 1.5 or supportv['aPTT'][0] > 50):
    y_predCoagulopathy = 1
else: 
    y_predCoagulopathy = 0

if (y_predxgbCellulitis > .4):
    y_predCellulitis = 1
else:
    y_predCellulitis = 0

if (y_predxgbTBI > .4 or yhistTBI > .4):
    y_predTBI = 1
else:
    y_predTBI = 0

if (y_predxgbBurn > .4):
    y_predBurn = 1
else:
    y_predBurn = 0

if (y_predxgbPAD > .4):
    y_predPAD = 1
else:
    y_predPAD = 0 

if (y_predxgbBrain_Tumor > .4):
    y_predBrain_Tumor = 1
else:
    y_predBrain_Tumor = 0

if (y_predxgbGERD > .4 or yhistGERD > .4):
    y_predGERD = 1
else:
    y_predGERD = 0

if y_predxgbpreopUEAngio > .4:
    y_predpreopUEAngio = 1
else:
    y_predpreopUEAngio = 0

if y_predxgbpreopImpella > .4:
    y_predpreopImpella = 1
else:
    y_predpreopImpella = 0

if y_predxgbpreopCraniotomy > .4:
    y_predpreopCraniotomy = 1
else:
    y_predpreopCraniotomy = 0

if y_predxgbpreopCath > .4:
    y_predpreopCath = 1
else:
    y_predpreopCath = 0

if y_predxgbpreopCABG > .4:
    y_predpreopCABG = 1
else:
    y_predpreopCABG = 0

if y_predxgbpreopAVR > .4:
    y_predpreopAVR = 1
else:
    y_predpreopAVR = 0

if y_predxgbpreopAngiogram > .4:
    y_predpreopAngiogram = 1
else:
    y_predpreopAngiogram = 0

if y_predxgbpreopAAA > .4:
    y_predpreopAAA = 1
else:
    y_predpreopAAA = 0

if (supportv['Plt'][0] < 10 or supportv['INR'][0] > 4 or supportv['aPTT'][0] > 120):
    severecoag = 1
else:
    severecoag = 0

if (supportv['Plt'][0] < 79 or supportv['INR'][0] > 2 or supportv['aPTT'][0] > 50):
    coag = 1
else:
    coag = 0

if (y_predvented >.4 or y_predGERD >.4 or y_predGI_Bleed >.4 or y_predSAH >.4 or y_predSDH >.4 or y_predICH >.4 or y_predRespiratory_Failure >.4 or y_predTBI >.4 or y_predBurn >.4 or y_predCoagulopathy >.4):
    SUP = 1
else:
    SUP = 0

if (y_predSAH >.4 or y_predSDH >.4 or y_predTBI >.4 or y_predGI_Bleed >.4 or y_predICH >.4 or y_predBrain_Tumor >.4 or y_predAAA >.4 or y_predCoagulopathy >.4 or data['Hgb'][0] < 7):
    chemDVT = 0
else:
    chemDVT = 1

if (y_predPAD >.4 or y_predCellulitis >.4 or severecoag > .4):
    mechDVT = 0
else:
    mechDVT = 1

if (y_predSAH >.4 or y_predSDH >.4 or y_predTBI >.4 or y_predGI_Bleed >.4 or y_predICH >.4 or y_predAAA >.4):
    Bleed = 1
else:
    Bleed = 0

if (y_predSAH >.4 or y_predSDH >.4 or y_predTBI >.4 or y_predGI_Bleed >.4 or y_predICH >.4 or y_predAAA >.4 or y_predDKA >.4 or y_predCardiac_arrest >.4):
    NPO = 1
else:
    NPO = 0

#PreOpper
if len(op) > 0:
  op.columns=['column1', 'column2']
  op['column1'] = op['column1'].astype(str)
  op['column2'] = op['column2'].astype(str)
  surgeen = op.loc[op['column2'].str.contains("SURGEON")]
  surgeen['column1'] = surgeen['column1'].astype(str)
  surgeen = list(surgeen['column1'])
  procedure = op.loc[op['column2'].str.contains("PROCEDURE")]
  procedure['column1'] = procedure['column1'].astype(str)
  procedure = list(procedure['column1'])
  #hist
  flag=0
  for i in procedure:
      for j in extremity_angiogram:
          if i==j:
              flag=1
              break
  if flag==1:
      yprocextremity_angiogram = 1
  else:
      yprocextremity_angiogram = 0

  flag=0
  for i in procedure:
      for j in pericardial_window:
          if i==j:
              flag=1
              break
  if flag==1:
      yprocpericardial_window = 1
  else:
      yprocpericardial_window = 0

  flag=0
  for i in procedure:
      for j in lung_transplant:
          if i==j:
              flag=1
              break
  if flag==1:
      yproclung_transplant = 1
  else:
      yproclung_transplant = 0

  flag=0
  for i in procedure:
      for j in coronary_angiogram:
          if i==j:
              flag=1
              break
  if flag==1:
      yproccoronary_angiogram = 1
  else:
      yproccoronary_angiogram = 0

  flag=0
  for i in procedure:
      for j in maze:
          if i==j:
              flag=1
              break
  if flag==1:
      yprocmaze = 1
  else:
      yprocmaze = 0

  flag=0
  for i in procedure:
      for j in lvad:
          if i==j:
              flag=1
              break
  if flag==1:
      yproclvad = 1
  else:
      yproclvad = 0

  flag=0
  for i in procedure:
      for j in mvr:
          if i==j:
              flag=1
              break
  if flag==1:
      yprocmvr = 1
  else:
      yprocmvr = 0

  flag=0
  for i in procedure:
      for j in avr:
          if i==j:
              flag=1
              break
  if flag==1:
      yprocavr = 1
  else:
      yprocavr = 0

  flag=0
  for i in procedure:
      for j in tvr:
          if i==j:
              flag=1
              break
  if flag==1:
      yproctvr = 1
  else:
      yproctvr = 0

  flag=0
  for i in procedure:
      for j in cabg:
          if i==j:
              flag=1
              break
  if flag==1:
      yproccabg = 1
  else:
      yproccabg = 0

  flag=0
  for i in procedure:
      for j in impella:
          if i==j:
              flag=1
              break
  if flag==1:
      yprocimpella = 1
  else:
      yprocimpella = 0

  flag=0
  for i in procedure:
      for j in aaa:
          if i==j:
              flag=1
              break
  if flag==1:
      yprocaaa = 1
  else:
      yprocaaa = 0

  flag=0
  for i in procedure:
      for j in oht:
          if i==j:
              flag=1
              break
  if flag==1:
      yprocoht = 1
  else:
      yprocoht = 0

  flag=0
  for i in procedure:
      for j in aortic_dissection:
          if i==j:
              flag=1
              break
  if flag==1:
      yprocaortic_dissection = 1
  else:
      yprocaortic_dissection = 0

  flag=0
  for i in procedure:
      for j in craniotomy:
          if i==j:
              flag=1
              break
  if flag==1:
      yproccraniotomy = 1
  else:
      yproccraniotomy = 0

  flag=0
  for i in procedure:
      for j in kyphoplasty:
          if i==j:
              flag=1
              break
  if flag==1:
      yprockyphoplasty = 1
  else:
      yprockyphoplasty = 0

  flag=0
  for i in procedure:
      for j in ivorlewis:
          if i==j:
              flag=1
              break
  if flag==1:
      yprocivorlewis = 1
  else:
      yprocivorlewis = 0
else:
  yprocextremity_angiogram = 0
  yprocpericardial_window = 0
  yproclung_transplant = 0
  yproccoronary_angiogram = 0
  yprocmaze = 0
  yproclvad = 0
  yprocmvr = 0
  yprocavr = 0
  yproctvr = 0
  yproccabg = 0
  yprocimpella = 0
  yprocaaa = 0
  yprocoht = 0
  yprocaortic_dissection = 0
  yproccraniotomy = 0
  yprockyphoplasty = 0
  yprocivorlewis = 0


#NoteGen Output
print()  
print()  
print()   
print()   
print('\033[1m' + '*********       Assessment and Plan     *********' + '\033[0m')  
print()  
 
print("#1 Neuro:", end=" ")
if (y_predvented >.4):
    print(Dx57Wk1, Dx57Tx1)

if (y_predCVA >.4 and y_predAAA >.4):
    print(Dx69Wk3, Dx69Tx2)
elif (y_predCVA >.4 and y_predGI_Bleed >.4):
    print(Dx69Wk3, Dx69Tx2)
elif (y_predCVA >.4 and y_predSAH >.4):
    print(Dx69Wk3, Dx69Tx2)
elif (y_predCVA >.4 and y_predICH >.4):
    print(Dx69Wk3, Dx69Tx2)
elif (y_predCVA >.4 and y_predSDH >.4):
    print(Dx69Wk3, Dx69Tx2)
elif (y_predCVA >.4 and y_predBrain_Tumor >.4):
    print(Dx69Wk4, Dx69Tx2)
elif (y_predCVA >.4 and coag >.4):
    print(Dx69Wk2, Dx69Tx2)
elif (y_predCVA >.4):
    print(Dx69Wk1, Dx69Tx1)
elif (yhistCVA >.4):
    print(hist48)

if (y_predDelirium >.4 and y_predALF >.4):
    print(Dx70Wk1, Dx70Tx2)
elif (y_predDelirium >.4):
    print(Dx70Wk1, Dx70Tx1)

if data['Na'][0] <= 128: 
    print(Dx42Tx2)
    
if (y_predEtOH_WD >.4 and y_predvented >.4):
    print(Dx64Wk2, Dx64Tx2)
elif (y_predEtOH_WD >.4):
    print(Dx64Wk1, Dx64Tx1)
elif (yhistEtOH_WD >.4):
    print(hist43)
    
if (y_predmeningitis_encephalitis >.4 and y_predEndocarditis >.4):
    print(Dx26Wk2, Dx26Tx1)
elif (y_predmeningitis_encephalitis >.4 and y_predLupus >.4):
    print(Dx72Wk4, Dx72Tx3)
elif (y_predmeningitis_encephalitis >.4):
    print(Dx26Wk1, Dx26Tx1)
   
if (y_predICH >.4 and y_predHTN_Emergency >.4):
    print(Dx28Wk2, Dx28Tx2)
elif (y_predICH >.4 and y_predSDH >.4):
    print(Dx28Wk3, Dx28Tx2)
elif (y_predICH >.4 and y_predSAH >.4):
    print(Dx28Wk4, Dx28Tx2)
elif (y_predICH >.4): 
    print(Dx28Wk1, Dx28Tx1)
elif (yhistICH >.4): 
    print(hist34)

if (yhistICH >.4 and yhistSeizure >.4):
    print(hist34, hist37, histtx37)
elif (y_predICH >.4 and y_predSeizure >.4):
    print(Dx28Wk5, Dx28Tx3)
elif (y_predSeizure >.4 and y_predLupus >.4):
    print(Dx72Wk6, Dx72Tx3, Dx31Tx1)
elif (y_predSeizure >.4):
    print(Dx31Wk1, Dx31Tx1)
elif (yhistSeizure >.4):
    print(hist37, histtx37)

if (y_predSDH >.4 and y_predHTN_Emergency >.4):
    print(Dx29Wk2, Dx29Tx3)
elif (y_predSDH >.4 and y_predSeizure >.4):
    print(Dx29Wk3, Dx29Tx2)
elif (y_predSDH >.4):
    print(Dx29Wk1, Dx29Tx1)
elif (yhistSDH >.4):
    print(hist35)



if (y_predSAH >.4 and y_predSeizure >.4):
    print(Dx30Wk2, Dx30Tx2)
elif (y_predSAH >.4):
    print(Dx30Wk1, Dx30Tx1)
elif (yhistSAH >.4):
    print(hist36) 

if (y_predTBI >.4):
    print(Dx79Wk1, Dx79Tx1)

if (y_predBrain_Tumor >.4 or yhistBrain_Tumor >.4):
    print(Dx82Wk1, Dx82Tx1)

if (y_predmeningitis_encephalitis <.4 and y_predxgbDelirium <.4 and data['Na'][0] > 128 and y_predvented <.4 and y_predSDH <.4 and y_predSAH <.4 and y_predICH <.4 and y_predSeizure <.4):
    print(Dx50Tx1)

print()
print("#2 CV:", end=" ")
if (yhistACS >.4):
    print(hist2)

if (y_predACS >.4 and y_predCardiac_arrest >.4):
    print(Dx5Wk3, Dx5Tx8)
elif (y_predACS >.4 and y_predGI_Bleed >.4):
    print(Dx5Wk5, Dx5Tx9)
elif (y_predACS >.4 and coag >.4):
    print(Dx5Wk4, Dx5Tx9)
elif (y_predACS >.4):
    print(Dx5Wk1, Dx5Tx1)
elif (y_predCAD >.4 and Bleed >.4):
    print(Dx65Wk2, Dx65Tx2)
elif (y_predCAD >.4 and coag >.4):
    print(Dx65Wk2, Dx65Tx2)
elif (y_predCAD >.4):
    print(Dx65Wk1, Dx65Tx1)   
    

if (y_predAAA >.4 and y_predHypotensionShock >.4):
    print(Dx6Wk5, Dx6Tx4)
elif (y_predAAA >.4 and y_predHTN_Emergency >.4):
    print(Dx6Wk4, Dx6Tx3)
elif (y_predAAA >.4 and y_predCHF >.4):
    print(Dx6Wk3, Dx6Tx2)
elif (y_predAAA >.4 and y_predTachycardia >.4):
    print(Dx6Wk2, Dx6Tx1)
elif (yhistAAA >.4):
    print(hist3)
elif (y_predAAA >.4):
    print(Dx6Wk1, Dx6Tx1)
   
if (y_predTachycardia >.4 and y_predRespiratory_Failure >.4 and y_predHTN_Emergency >.4):
    print(Dx7Wk4, Dx7Tx3)
elif (y_predTachycardia >.4 and y_predHypotensionShock >.4 and y_predafib >.4):
    print(Dx7Wk9, Dx7Tx8)
elif (y_predTachycardia >.4 and y_predHypotensionShock >.4):
    print(Dx7Wk5, Dx7Tx4)
elif (y_predTachycardia >.4 and y_predCHF >.4):
    print(Dx7Wk2, Dx7Tx2)
elif (y_predTachycardia >.4 and y_predHTN_Emergency >.4):
    print(Dx7Wk3, Dx7Tx3)
elif (y_predTachycardia >.4 and y_predCOPD >.4):
    print(Dx7Wk6, Dx7Tx5)
elif (y_predTachycardia >.4 and y_predafib >.4):
    print(Dx7Wk8, Dx7Tx7)
elif (y_predTachycardia >.4 and y_predPE >.4):
    print(Dx7Wk7, Dx7Tx6) 
elif (y_predTachycardia >.4):
    print(Dx7Wk1, Dx7Tx1)
        
if (y_predBradycardia >.4 and y_predHypotensionShock >.4):
    print(Dx8Wk4, Dx8Tx3)
elif (y_predBradycardia >.4 and y_predCHF >.4):
    print(Dx8Wk2, Dx8Tx2)
elif (y_predBradycardia >.4 and y_predHTN_Emergency >.4):
    print(Dx8Wk3, Dx8Tx1)
elif (y_predBradycardia >.4):
    print(Dx8Wk1, Dx8Tx1)

if (y_predEtOH_WD >.4):
    print(Dx64Tx3)
            
if (y_predCHF >.4 and y_predCardiac_arrest >.4):
    print(Dx9Wk4, Dx9Tx3)
elif (y_predCHF >.4 and y_predHypotensionShock >.4):
    print(Dx9Wk3, Dx9Tx3)
elif (y_predCHF >.4 and y_predEndocarditis >.4):
    print(Dx9Wk7, Dx9Tx6)
elif (y_predCHF >.4 and y_predHTN_Emergency >.4):
    print(Dx9Wk2, Dx9Tx2)
elif (y_predCHF >.4 and y_predALF >.4):
    print(Dx9Wk6, Dx9Tx5)
elif (y_predCHF >.4 and y_predAKI >.4):
    print(Dx9Wk9, Dx9Tx8)
elif (y_predCHF >.4 and y_predSAH >.4):
    print(Dx9Wk8, Dx9Tx7)
elif (y_predCHF >.4 and y_predRespiratory_Failure >.4):
    print(Dx9Wk5, Dx9Tx4)
elif (y_predCHF >.4):
    print(Dx9Wk1, Dx9Tx1)  
    
    
if (y_predHTN_Emergency >.4 and y_predRespiratory_Failure >.4):
    print(Dx10Wk2, Dx10Tx2)
elif (y_predHTN_Emergency >.4 and y_predICH >.4):
    print(Dx10Wk3, Dx10Tx3)
elif (y_predHTN_Emergency >.4 and y_predSDH >.4):
    print(Dx10Wk4, Dx10Tx4)
elif (y_predHTN_Emergency >.4 and y_predSAH >.4):
    print(Dx10Wk5, Dx10Tx5)
elif (y_predHTN_Emergency >.4 and y_predAKI >.4):
    print(Dx10Wk7, Dx10Tx6)
elif (y_predHTN_Emergency >.4 and y_predESRD >.4):
    print(Dx10Wk7, Dx10Tx6)
elif (y_predHTN_Emergency >.4):
    print(Dx10Wk1, Dx10Tx1)
if (y_predHTN >.4 and y_predHypotensionShock >.4):
    print(Dx62Wk2, Dx62Tx2)
elif (y_predHTN >.4 and y_predICH >.4):
    print(Dx62Wk1, Dx28Tx5)
elif (y_predICH >.4 or y_predSDH >.4):  
    print(Dx28Tx5)
elif (y_predCVA >.4):
    print(Dx62Tx4)
elif (y_predHTN >.4 and supportv['SBP'][0] >= 150 and supportv['SBP'][0] < 195):
    print(Dx62Wk1, Dx62Tx3) 
elif (y_predHTN >.4):
    print(Dx62Wk1, Dx62Tx1)
    
if (y_predHypotensionShock >.4 and y_predPneumonia >.4):
    print(Dx11Wk2, Dx11Tx2)
elif (y_predHypotensionShock >.4 and y_predPneumothorax >.4):
    print(Dx11Wk3, Dx11Tx3)
elif (y_predHypotensionShock >.4 and y_predPE >.4):
    print(Dx11Wk6, Dx11Tx6)
elif (y_predHypotensionShock >.4 and y_predALF >.4):
    print(Dx11Wk5, Dx11Tx5)
elif (y_predHypotensionShock >.4 and y_predGI_Bleed >.4):
    print(Dx11Wk6, Dx11Tx6)
elif (y_predHypotensionShock >.4 and y_predColitis >.4):
    print(Dx11Wk7, Dx11Tx2)
elif (y_predHypotensionShock >.4 and y_predBacteremia >.4):
    print(Dx11Wk8, Dx11Tx2)
elif (y_predHypotensionShock >.4 and y_predmeningitis_encephalitis >.4):
    print(Dx11Wk9, Dx11Tx2)
elif (y_predHypotensionShock >.4 and y_predEndocarditis >.4):
    print(Dx11Wk8, Dx11Tx2)
elif (y_predHypotensionShock >.4 and y_predUTI >.4):
    print(Dx11Wk9, Dx11Tx2)
elif (y_predHypotensionShock >.4 and y_predColitis >.4):
    print(Dx11Wk9, Dx11Tx2)
elif (y_predHypotensionShock >.4 and y_predICH >.4):
    print(Dx11Wk8, Dx11Tx2) 
elif (y_predHypotensionShock >.4):
    print(Dx11Wk1, Dx11Tx1) 
    
if (y_predCardiac_arrest >.4 and y_predBradycardia >.4):
    print(Dx12Wk2, Dx12Tx2)
elif (y_predCardiac_arrest >.4 and y_predHypotensionShock >.4):
    print(Dx12Wk2, Dx12Tx2)
elif (y_predCardiac_arrest >.4) :
    print(Dx12Wk1, Dx12Tx1)

if (y_predafib >.4 and Bleed >.4):
    print(Dx58Wk2, Dx58Tx2)
elif (y_predafib >.4 and coag >.4):
    print(Dx58Wk2, Dx58Tx2)
elif (y_predafib >.4):
    print(Dx58Wk1, Dx58Tx1)

if (y_predPAD >.4):
    print(Dx81Wk1, Dx81Tx1)
if yhistPAD > .4:
    print(hist61)

if (y_predPericardial_Effusion >.4 and y_predCardiac_arrest >.4):
    print(Dx74Wk3, Dx74Tx2)
elif (y_predPericardial_Effusion >.4 and y_predAAA >.4):
    print(Dx74Wk2, Dx74Tx2)
elif (y_predPericardial_Effusion >.4 and y_predESRD >.4):
    print(Dx74Wk4, Dx74Tx2)
elif (y_predPericardial_Effusion >.4 and y_predLupus >.4):
    print(Dx74Wk5, Dx74Tx2)
elif (y_predPericardial_Effusion >.4):
    print(Dx74Wk1, Dx74Tx1)

if (y_predACS <.4 and y_predAAA <.4 and supportv['SBP'][0] < 150 and y_predCVA <.4 and y_predPericardial_Effusion <.4 and y_predSDH <.4 and y_predCAD <.4 and y_predICH <.4 and y_predTachycardia <.4 and y_predBradycardia <.4 and y_predCHF <.4 and y_predHTN_Emergency <.4 and
    y_predHypotensionShock <.4 and y_predCardiac_arrest <.4 and y_predafib <.4):
    print(Dx46Tx1)
    
print()
print("#3 Pulm:", end=" ")
 

if (y_predARDS >.4):
    print(Dx78Wk1, Dx78Tx1)
elif (y_predvented >.4 and y_predCHF >.4):
    print(Dx57Wk2, Dx57Tx2)
elif (y_predvented >.4 and y_predHTN_Emergency >.4):
    print(Dx57Wk2, Dx57Tx2)
elif (y_predvented >.4 and y_predCardiac_arrest >.4):
    print(Dx57Wk3, Dx57Tx1)
elif (y_predvented >.4):
    print(Dx57Wk1, Dx57Tx1)
         
if (y_predAirway_Obstruction >.4):
    print(Dx13Wk1, Dx13Tx1)

if (y_predvented >.4 or yhisttrach >.4) and (y_predCOPD >.4 and y_predPneumonia >.4 and y_predRespiratory_Failure >.4):
    print(Dx57Wk4, Dx57Tx5)
elif (y_predvented >.4 or yhisttrach >.4) and (y_predCOPD >.4 and y_predPneumonia >.4):
    print(Dx57Wk4, Dx57Tx5)
elif (y_predCOPD >.4 and y_predPneumonia >.4 and y_predRespiratory_Failure >.4):
    print(Dx57Wk6, Dx57Tx6)
elif (y_predvented >.4 or yhisttrach >.4) and (y_predPneumonia >.4):
    print(Dx57Wk5, Dx57Tx4)
elif (y_predRespiratory_Failure >.4 and y_predPneumonia >.4):
    print(Dx17Wk4, Dx17Tx4)
elif (y_predCOPD >.4) and (y_predvented >.4 or yhisttrach >.4):
    print(Dx14Wk7, Dx14Tx6)
elif (y_predCOPD >.4 and y_predRespiratory_Failure >.4):
    print(Dx14Wk6, Dx14Tx5)
elif (y_predCOPD >.4 and y_predPneumonia >.4):
    print(Dx14Wk4, Dx14Tx3) 
elif (y_predCOPD >.4):
    print(Dx14Wk1, Dx14Tx1)
elif (yhistCOPD >.4):
    print(hist12, histtx12)   
elif (y_predPneumonia >.4):
    print(Dx15Wk1)

if yhisttrach >.4:
    print(hist75, histtx75)
    
if (y_predCOPD >.4 and y_predPneumothorax >.4):
    print(Dx14Wk5, Dx14Tx4)
elif (y_predCOPD >.4 and y_predTachycardia >.4):
    print(Dx14Wk3, Dx14Tx1)
elif (y_predCOPD >.4 and y_predACS >.4):
    print(Dx14Wk2, Dx14Tx2)


if (y_predAsthma >.4 and y_predRespiratory_Failure >.4):
    print(Dx63Wk2, Dx63Tx2)
elif (y_predAsthma >.4 and y_predvented >.4):
    print(Dx63Wk2, Dx63Tx2)
elif (y_predAsthma >.4):
    print(Dx63Wk1, Dx63Tx1)
elif (yhistAsthma >.4):
    print(hist41, histtx41)
       
elif (y_predPneumothorax >.4):
    print(Dx16Wk1, Dx16Tx1)
if yhistPneumothorax >.4:
    print(hist13)
          
if (y_predRespiratory_Failure >.4 and y_predHypotensionShock >.4):
    print(Dx17Wk2, Dx17Tx2)
elif (y_predRespiratory_Failure >.4 and y_predPneumothorax >.4):
    print(Dx17Wk5, Dx17Tx5)
elif (y_predRespiratory_Failure >.4 and y_predALF >.4):
    print(Dx17Wk7, Dx17Tx7)
elif (y_predRespiratory_Failure >.4 and y_predmeningitis_encephalitis >.4):
    print(Dx17Wk8, Dx17Tx8)
elif (y_predRespiratory_Failure >.4 and y_predICH >.4):
    print(Dx17Wk8, Dx17Tx8)
elif (y_predRespiratory_Failure >.4 and y_predSDH >.4):
    print(Dx17Wk8, Dx17Tx8)
elif (y_predRespiratory_Failure >.4 and y_predSAH >.4):
    print(Dx17Wk8, Dx17Tx8)
elif (y_predRespiratory_Failure >.4 and y_predSeizure >.4):
    print(Dx17Wk8, Dx17Tx8)
elif (y_predRespiratory_Failure >.4):
    print(Dx17Wk1, Dx17Tx1) 
    
if (y_predPE >.4 and y_predGI_Bleed >.4):
    print(Dx18Wk3, Dx18Tx2)
elif (y_predPE >.4 and y_predICH >.4):
    print(Dx18Wk3, Dx18Tx2)
elif (y_predPE >.4 and y_predSDH >.4):
    print(Dx18Wk3, Dx18Tx2)
elif (y_predPE >.4 and y_predSAH >.4):
    print(Dx18Wk3, Dx18Tx2)
elif (y_predPE >.4 and coag >.4):
    print(Dx18Wk2, Dx18Tx2)
elif (y_predPE >.4):
    print(Dx18Wk1, Dx18Tx1)
elif (yhistPE > .4):
    print(hist33)

if (y_predOSA >.4 and y_predvented <.5):
    print(Dx66Wk1, Dx66Tx1)
    
if (y_predAirway_Obstruction >.4 or y_predPE >.4 or y_predCOPD >.4 or y_predPneumonia >.4 or y_predPneumothorax >.4 or y_predvented >.4 or y_predRespiratory_Failure >.4):
    print()
else: print(Dx47Tx1)

print()    
print("#4 GI:", end=" ")
 
if (y_predALF >.4):
      print(Dx19Wk1, Dx19Tx1)
elif (yhistALF >.4):
      print(hist15, histtx15)
        
if (y_predGI_Bleed >.4 and  y_predALF >.4):
      print(Dx20Wk2, Dx20Tx2)
elif (y_predGI_Bleed >.4 and  y_predACS >.4):
      print(Dx20Wk3, Dx20Tx2)
elif (y_predGI_Bleed >.4):
      print(Dx20Wk1, Dx20Tx1) 
        
if (y_predPancreatitis >.4):
      print(Dx21Wk1, Dx21Tx1)
elif (yhistPancreatitis >.4):
      print(hist17)
        
if (y_predColitis >.4):
      print(Dx22Wk1, Dx22Tx1)
elif (yhistColitis >.4):
      print(hist18)
        
if (y_predGastroenteritis >.4):
      print(Dx23Wk1, Dx23Tx1)

if (y_predGERD >.4):
      print(Dx83Wk1, Dx83Tx1)
        
if (y_predGI_Bleed >.4 or y_predPancreatitis >.4 or y_predALF >.4 or y_predColitis >.4 or y_predGastroenteritis >.4):
      print()
elif (NPO >.4):
      print(Dx48Tx2)
else: print(Dx48Tx1)
    
print()        
print("#5 Renal:", end=" ")
 
if (data['AG'][0] >= 16 and y_predDKA >.4):
    print(Dx59Wk2, Dx59Tx2)
elif (data['AG'][0] >= 16):
    print(Dx59Wk1, Dx59Tx1)
          
if (y_predESRD >.4 and data['Cr'][0] >= 2):
    print(Dx60Wk1, Dx60Tx1)
elif (y_predCKD >.4 and data['Cr'][0] >= 1.4):
    print(Dx61Wk1, Dx61Tx1)
elif (y_predAKI >.4 and data['Cr'][0] >= 1.4):
    print(Dx37Wk1, Dx37Tx1)
          
if data['Na'][0] >= 150: 
    print(Dx41Wk1, Dx41Tx1)
          
if data['Na'][0] <= 130: 
    print(Dx42Wk1, Dx42Tx1)
          
if data['K'][0] >= 5.8: 
    print(Dx43Wk1, Dx43Tx1)
          
if data['K'][0] <= 3.4: 
    print(Dx44Wk1, Dx44Tx1)
          
if (data['Na'][0] <= 150 and data['Na'][0] >= 130 and data['K'][0] <= 5.8 and data['K'][0] >= 3.4 and y_predESRD <.4 and y_predCKD <.4 and y_predAKI <.4):
    print(Dx49Tx1) 

print()  
print("#6 ID:", end=" ")
 
if (y_predBacteremia >.4 and y_predPneumonia >.4):
    print(Dx25Wk2, Dx25Tx1)
elif (y_predBacteremia >.4 and y_predALF >.4):
    print(Dx25Wk3, Dx25Tx1)
elif (y_predBacteremia >.4 and y_predPancreatitis >.4):
    print(Dx25Wk4, Dx25Tx2)
elif (y_predBacteremia >.4 and y_predColitis >.4):
    print(Dx25Wk4, Dx25Tx2)
elif (y_predBacteremia >.4 and y_predGastroenteritis >.4):
    print(Dx25Wk4, Dx25Tx2)
elif (y_predBacteremia >.4 and y_predmeningitis_encephalitis >.4):
    print(Dx25Wk5, Dx25Tx3)
elif (y_predBacteremia >.4 and y_predEndocarditis >.4):
    print(Dx25Wk6, Dx25Tx1)
elif (y_predBacteremia >.4 and y_predUTI >.4):
    print(Dx25Wk7, Dx25Tx1)
elif (y_predBacteremia >.4):
    print(Dx25Wk1, Dx25Tx1)

if (y_predEndocarditis >.4):
    print(Dx27Wk1, Dx27Tx1) 
elif (yhistEndocarditis >.4):
    print(hist21)

if (y_predmeningitis_encephalitis >.4 and y_predEndocarditis >.4):
    print(Dx26Wk2, Dx26Tx1)
elif (y_predmeningitis_encephalitis >.4):
    print(Dx26Wk1, Dx26Tx1)
    
if (y_predPneumonia >.4 and y_predALF >.4):
    print(Dx15Wk2, Dx15Tx1)
elif (y_predPneumonia >.4 and y_predEndocarditis >.4):
    print(Dx15Wk5, Dx15Tx2)
elif (y_predPneumonia >.4 and y_predBacteremia >.4):
    print(Dx15Wk4, Dx15Tx2)
elif (y_predPneumonia >.4):
    print(Dx15Wk1, Dx15Tx1)
    
if (y_predUTI >.4):
    print(Dx33Wk1, Dx33Tx1)
elif (yhistUTI >.4):
    print(hist22)

if (y_predCellulitis >.4) and (y_predHypotensionShock > .4 or y_predDelirium >.4):
    print(Dx34Wk2, Dx34Tx2)
elif (y_predCellulitis >.4):
    print(Dx34Wk1, Dx34Tx1)

if (y_predCholecystitis >.4):
    print(Dx75Wk1, Dx75Tx1)
    
if (y_predBacteremia >.4 or y_predCellulitis >.4 or y_predmeningitis_encephalitis >.4 or y_predPneumonia >.4 or y_predEndocarditis >.4 or y_predUTI >.4):
    print()
else: print(Dx53Tx1)

print()        
print("#7 Endo:", end=" ")

#Workup
if (y_predDKA >.4 and data['AG'][0] >= 16):
    print(Dx38Wk1, Dx38Tx1)
elif (y_predDM >.4):
    print(Dx36Wk1, Dx36Tx1)
elif (y_predDKA >.4):
    print(Dx36Wk1, Dx36Tx1)
else: 
    print(Dx52Tx1)

if (y_predDLD >=.4):
    print(Dx71Wk1, Dx71Tx1)

if (y_predhypothyroid >.4):
    print(Dx67Wk1, Dx67Tx1)
elif (y_predhyperthyroid >.4):
    print(Dx68Wk1, Dx68Tx1)

if (y_predAI >.4):
    print(Dx76Wk1, Dx76Tx1)
elif (y_predCushing >.4):
    print(Dx77Wk1, Dx77Tx1)


print()
print("#8 Heme/Onc:", end=" ")

#workup
if data['Hgb'][0] <= 9: 
    print(Dx39Wk1, Dx39Tx1)
if (data['Hgb'][0] > 9):
    print(Dx51Tx1)

if (data['AG'][0] < 0):
    print(Dx59Wk3, Dx59Tx3)

if (coag >.4 and Bleed >.4):
    print(Dx45Wk2, Dx45Tx2)
elif (coag >.4):
    print(Dx45Wk1, Dx45Tx1)

if (y_predLupus >.4 and y_predRespiratory_Failure >.4):
    print(Dx72Wk3, Dx72Tx2)
elif (y_predLupus >.4 and y_predHTN_Emergency >.4):
    print(Dx72Wk2, Dx72Tx2)
elif (y_predLupus >.4 and y_predAKI >.4):
    print(Dx72Wk5, Dx72Tx4)
elif (y_predLupus >.4 and y_predCKD >.4):
    print(Dx72Wk5, Dx72Tx4)
elif (y_predLupus >.4 and y_predESRD >.4):
    print(Dx72Wk5, Dx72Tx4)
elif (y_predLupus >.4 and y_predmeningitis_encephalitis >.4):
    print(Dx72Wk4, Dx72Tx3)
elif (y_predLupus >.4):
    print(Dx72Wk1, Dx72Tx1)

if (y_predScleroderma >.4):
    print(Dx73Wk1, Dx73Tx1)

if (y_predBurn >.4):
    print(Dx80Wk1, Dx80Tx1)

print() 
print("#9 PPx:", end=" ")
if (SUP <.4 and chemDVT <.4 and mechDVT <.4 and y_predALF <.4 and supportv['INR'][0] > 2):
    print('INR', supportv['INR'][0], 'for DVT Prophylaxis.')
elif (SUP >.4 and chemDVT <.4 and mechDVT <.4 and y_predALF <.4 and supportv['INR'][0] > 2):
    print('INR', supportv['INR'][0], 'for DVT Prophylaxis.  Cont H2 or PPI for SUP.')
elif (SUP <.4 and chemDVT <.4 and mechDVT <.4):
    print(Dx54Tx1)
elif (SUP >.4 and chemDVT <.4 and mechDVT <.4):
    print(Dx54Tx2)
elif (SUP >.4 and chemDVT <.4 and mechDVT >.4):
    print(Dx54Tx3)
elif (SUP <.4 and chemDVT <.4 and mechDVT >.4):
    print(Dx54Tx4)
elif (SUP >.4 and chemDVT >.4):
    print(Dx54Tx5)
elif (SUP <.4 and chemDVT >.4):
    print(Dx54Tx6)

print()
print("I spent a total of greater than 60 minutes formulating critical care for this patient today. I saw this patient and completed a full visual exam via audio-visual HIPAA compliant technology.")
print()
print()
print('\033[1m' + '*********      Operative Assessment and Plan     *********' + '\033[0m')  
print()  

if yprocextremity_angiogram >.4:
    print(Op1)

if yprocimpella > .4:
    print(Op2)
   
if yproccraniotomy > .4:
    print(Op3)

if yproccoronary_angiogram > .4:
    print(Op4)

if yproccabg > .4:
    print(Op5)

if yprocavr > .4:
    print(Op6)

if yprocaaa > .4:
    print(Op7)

if yprocpericardial_window > .4:
    print(Op8)

if yprocmvr > .4:
    print(Op9)

print()
print()

#ICD-10 Output
print()  
print()    
print('\033[1m' + '*********       ICD-10 Code Checker     *********' + '\033[0m')  
print()  
print() 
if (y_predmeningitis_encephalitis >.4 or yhistmeningitis_encephalitis >.4):
    print("meningitis/encephalitis G03.9/G04.90")   
if (y_predACS >.4 or yhistACS >.4):
    print("ACS I24.9")  
if (y_predAAA >.4 or yhistAAA >.4):
    print("AAA I71.4")  
if (y_predTachycardia >.4 or yhistTachycardia >.4):
     print("Tachycardia R00.0") 
if (y_predBradycardia >.4 or yhistBradycardia >.4):
    print("Bradycardia R00.1")  
if (y_predCHF >.4 or yhistCHF >.4):
    print("CHF I50.9")  
if (y_predHTN_Emergency >.4 or yhistHTN_Emergency >.4):
    print("HTN Emergency I16.1", "HTN I10") 
if (y_predHypotensionShock >.4 or yhistHypotensionShock >.4):
    print("Hypotension I95.9 Shock R57.0")  
if (y_predCardiac_arrest >.4 or yhistCardiac_arrest >.4):
    print("Cardiac arrest I46.9")   
if (y_predAirway_Obstruction >.4 or yhistAirway_Obstruction >.4):
    print("Airway_Obstruction J98.8")   
if (y_predPneumonia >.4 or yhistPneumonia >.4):
    print("Pneumonia J18.9", "Sepsis A41.9")    
if (y_predCOPD >.4 or yhistCOPD >.4):
    print("COPD J44.9")     
if (y_predPneumothorax >.4 or yhistPneumothorax >.4):
    print("Pneumothorax J93.9") 
if (y_predRespiratory_Failure >.4 or yhistRespiratory_Failure >.4):
    print("Respiratory Failure J96")    
if (y_predALF >.4 or yhistALF >.4):
    print("ALF K72.00 Cirrhosis K74.60")    
if (y_predGI_Bleed >.4 or yhistGI_Bleed >.4):
    print("GI Bleed K92.2") 
if (y_predPancreatitis >.4 or yhistPancreatitis >.4):
    print("Pancreatitis K85.90")    
if (y_predColitis >.4 or yhistColitis >.4):
    print("Colitis K52.9", "Sepsis A41.9")  
if (y_predGastroenteritis >.4 or yhistGastroenteritis >.4):
    print("Gastroenteritis K52.9")  
if (y_predBacteremia >.4 or yhistBacteremia >.4):
    print("Bacteremia R78.81", "Sepsis A41.9")  
if (y_predEndocarditis >.4 or yhistEndocarditis >.4):
    print("Endocarditis I38", "Sepsis A41.9")   
if (y_predUTI >.4 or yhistUTI >.4):
    print("UTI N39.0", "Sepsis A41.9")  
if (y_predvented >.4 or yhistvented >.4):
    print("vented ARF J96") 
if (y_predafib >.4 or yhistafib >.4):
    print("Atrial fibrillation I48.0")  
if (y_predDM >.4 or yhistDM >.4):
    print("DM E11.8")   
if (y_predDKA >.4):
    print("DM E11.8", "Acidosis E87.2", "DKA E11.10")   
if (y_predPE >.4 or yhistPE >.4):
    print("PE I26.99")  
if (y_predICH >.4 or yhistICH >.4):
    print("ICH I62.9")  
if (y_predSDH >.4 or yhistSDH >.4):
    print("SDH S06.5X9A")   
if (y_predSAH >.4 or yhistSAH >.4):
    print("SAH I60.9")  
if (y_predSeizure >.4 or yhistSeizure >.4):
    print("Seizures R56.9") 
if (y_predESRD >.4 or yhistESRD >.4):
    print("ESRD N18.6") 
if (y_predCKD >.4 or yhistCKD >.4):
    print("CKD N18.9")  
if (y_predAKI >.4 or yhistAKI >.4):
    print("AKI N17.9")  
if (y_predAsthma >.4 or yhistAsthma >.4):
    print("Asthma J45.909")     
if (y_predHTN >.4 or yhistHTN >.4) or (y_predHTN >.4 or supportv['SBP'][0] > 139):
    print("HTN I10")    
if (y_predEtOH_WD >.4 or yhistEtOH_WD >.4):
    print("EtOH Withdrawal F10.20")     
if (y_predCAD >.4 or yhistCAD >.4):
    print("CAD I25.10") 
if (y_predOSA >.4 or yhistOSA >.4):
    print("OSA G47.33") 
if (y_predhypothyroid >.4 or yhisthypothyroid >.4):
    print("Hypothyroid E03.9")  
if (y_predhyperthyroid >.4 or yhisthyperthyroid >.4):
    print("Hyperthyroid E05.90")    
if (y_predCVA >.4 or yhistCVA >.4):
    print("CVA I63.9")  
if (y_predDelirium >.4 or yhistDelirium >.4):
    print("Delirium R41.0") 
if (y_predDLD >.4 or yhistDLD >.4):
    print("HLD E78.5")      
if (y_predAI >.4 or yhistAI >.4):
    print("Adrenal Insufficiency E27.1")    
if (y_predLupus >.4 or yhistLupus >.4):
    print("Lupus M32.9")    
if (y_predCushing >.4 or yhistCushing >.4):
    print("Cushings Disease E24.0")     
if (y_predScleroderma >.4 or yhistScleroderma >.4):  
    print("Scleroderma M34.9")      
if (y_predPericardial_Effusion >.4 or yhistPericardial_Effusion >.4):
    print("Pericardial Effusion I31.39")    
if (y_predCholecystitis >.4 or yhistCholecystitis >.4):
    print("Cholecystitis  K81.9")   
if (y_predARDS >.4 or yhistARDS >.4):
    print("ARDS J80")   
if (y_predCellulitis >.4 or yhistCellulitis >.4):
    print("Cellulitis L03.90 Sepsis A41.9") 
if (y_predTBI >.4 or yhistTBI >.4):
    print("TBI/Head Trauma S06.9XAA S09.90XA")  
if (y_predBurn >.4 or yhistBurn >.4):
    print("Burn T30.0") 
if (y_predPAD >.4 or yhistPAD >.4):
    print("PAD I73.9")  
if (y_predBrain_Tumor >.4 or yhistBrain_Tumor >.4):
    print("Brain Tumor D49.6")  
if (y_predGERD >.4 or yhistGERD >.4):
    print("GERD K21.9") 
if data['Na'][0] >= 150: 
    print("Hypernatremia E87.0")
if data['Na'][0] <= 130: 
    print("Hyponatremia E87.1")
if data['K'][0] >= 5.8: 
    print("Hyperkalemia E87.5")
if data['K'][0] <= 3.4: 
    print("Hypokalemia E87.6")
if data['Hgb'][0] <= 10: 
    print("Anemia D64.9")
if data['AG'][0] >= 16: 
    print("Anion Gap Acidosis E87.2")
if (y_predCoagulopathy >.4):
    print("Coagulopathy I73.9")
if (supportv['Plt'][0] < 100):
    print("Thrombocytopenia D69.6")
if (y_predxgbpreopUEAngio >.4):
    print("Peripheral Angiopathy I73.9")
if y_predxgbpreopImpella > .4:
    print("Cardiogenic Shock R57.0")
if y_predxgbpreopCraniotomy > .4:
    print("S/P Craniotomy Z98.890")
if y_predxgbpreopCath > .4:
    print("ACS I24.9")
if y_predxgbpreopCABG > .4:
    print("CAD I25.10")
if y_predxgbpreopAVR > .4:
    print("S/P AVR Z95.2")
if y_predxgbpreopAngiogram > .4:
    print("S/P Angiogram Z98.890")
if y_predxgbpreopAAA > .4:
    print("EVAR Z86.79")   

print()
print('\033[1m' + '   Active Dx:' + '\033[0m')
print()
 
if (y_predmeningitis_encephalitis >.4):
    print("meningitis/encephalitis G03.9/G04.90")
if (y_predACS >.4):
    print("ACS I24.9")
if (y_predAAA >.4):
    print("AAA I71.4")
if (y_predTachycardia >.4):
     print("Tachycardia R00.0")
if (y_predBradycardia >.4):
    print("Bradycardia R00.1")
if (y_predCHF >.4):
    print("CHF I50.9")
if (y_predHTN_Emergency >.4):
    print("HTN Emergency I16.1", "HTN I10")
if (y_predHypotensionShock >.4):
    print("Hypotension I95.9 Shock R57.0")
if (y_predCardiac_arrest >.4):
    print("Cardiac arrest I46.9")
if (y_predAirway_Obstruction >.4):
    print("Airway_Obstruction J98.8")
if (y_predPneumonia >.4):
    print("Pneumonia J18.9", "Sepsis A41.9")
if (y_predCOPD >.4):
    print("COPD J44.9") 
if (y_predPneumothorax >.4):
    print("Pneumothorax J93.9")
if (y_predRespiratory_Failure >.4):
    print("Respiratory Failure J96")
if (y_predALF >.4):
    print("ALF K72.00 Cirrhosis K74.60")
if (y_predGI_Bleed >.4):
    print("GI Bleed K92.2")
if (y_predPancreatitis >.4):
    print("Pancreatitis K85.90")
if (y_predColitis >.4):
    print("Colitis K52.9", "Sepsis A41.9")
if (y_predGastroenteritis >.4):
    print("Gastroenteritis K52.9")
if (y_predBacteremia >.4):
    print("Bacteremia R78.81", "Sepsis A41.9")
if (y_predEndocarditis >.4):
    print("Endocarditis I38", "Sepsis A41.9")
if (y_predUTI >.4):
    print("UTI N39.0", "Sepsis A41.9")
if (y_predvented >.4):
    print("vented ARF J96")
if (y_predafib >.4):
    print("Atrial fibrillation I48.0")
if data['Na'][0] >= 150: 
    print("Hypernatremia E87.0")
if data['Na'][0] <= 130: 
    print("Hyponatremia E87.1")
if data['K'][0] >= 5.8: 
    print("Hyperkalemia E87.5")
if data['K'][0] <= 3.4: 
    print("Hypokalemia E87.6")
if data['Hgb'][0] <= 10: 
    print("Anemia D64.9")
if (y_predDM >.4):
    print("DM E11.8")
if (y_predDKA >.4):
    print("DM E11.8", "Acidosis E87.2", "DKA E11.10")
if data['AG'][0] >= 16: 
    print("Anion Gap Acidosis E87.2")
if (y_predPE >.4):
    print("PE I26.99")
if (y_predICH >.4):
    print("ICH I62.9")
if (y_predSDH >.4):
    print("SDH S06.5X9A")
if (y_predSAH >.4):
    print("SAH I60.9")
if (y_predSeizure >.4):
    print("Seizures R56.9")
if (y_predESRD >.4 and data['Cr'][0] >= 2):
    print("ESRD N18.6")
if (y_predCKD >.4 and data['Cr'][0] >= 1.4):
    print("CKD N18.9")
if (y_predAKI >.4 and data['Cr'][0] >= 1.4):
    print("AKI N17.9")
if (y_predAsthma >.4):
    print("Asthma J45.909") 
if (y_predHTN >.4 and supportv['SBP'][0] > 140):
    print("HTN I10") 
if (y_predEtOH_WD >.4):
    print("EtOH Withdrawal F10.20") 
if (y_predCAD >.4):
    print("CAD I25.10")
if (y_predOSA >.4):
    print("OSA G47.33")
if (y_predhypothyroid >.4):
    print("Hypothyroid E03.9")
if (y_predhyperthyroid >.4):
    print("Hyperthyroid E05.90")
if (y_predCVA >.4):
    print("CVA I63.9")
if (y_predDelirium >.4):
    print("Delirium R41.0")
if (y_predDLD >.4):
    print("HLD E78.5")   
if (y_predAI >.4):
    print("Adrenal Insufficiency E27.1")   
if (y_predLupus >.4):
    print("Lupus M32.9")  
if (y_predCushing >.4):
    print("Cushings Disease E24.0")  
if (y_predScleroderma >.4):     
    print("Scleroderma M34.9")  
if (y_predPericardial_Effusion >.4):
    print("Pericardial Effusion I31.39")
if (y_predCholecystitis >.4):
    print("Cholecystitis    K81.9")
if (y_predARDS >.4):
    print("ARDS J80")
if (y_predCellulitis >.4):
    print("Cellulitis   L03.90 Sepsis A41.9")
if (y_predTBI >.4):
    print("TBI/Head Trauma S06.9XAA S09.90XA")
if (y_predBurn >.4):
    print("Burn T30.0")
if (y_predPAD >.4):
    print("PAD I73.9")
if (y_predBrain_Tumor >.4):
    print("Brain Tumor D49.6")
if (y_predCoagulopathy >.4):
    print("Coagulopathy I73.9")
if (supportv['Plt'][0] < 100):
    print("Thrombocytopenia D69.6")
if (y_predGERD >.4):
    print("GERD K21.9")
if (y_predxgbpreopUEAngio >.4):
    print("Peripheral Angiopathy I73.9")
if y_predxgbpreopImpella > .4:
    print("Cardiogenic Shock R57.0")
if y_predxgbpreopCraniotomy > .4:
    print("S/P Craniotomy Z98.890")
if y_predxgbpreopCath > .4:
    print("ACS I24.9")
if y_predxgbpreopCABG > .4:
    print("CAD I25.10")
if y_predxgbpreopAVR > .4:
    print("S/P AVR Z95.2")
if y_predxgbpreopAngiogram > .4:
    print("S/P Angiogram Z98.890")
if y_predxgbpreopAAA > .4:
    print("EVAR Z86.79")   

print()
print('\033[1m' + '   Inactive Dx:' + '\033[0m')
print()
if (y_predmeningitis_encephalitis <.4 and yhistmeningitis_encephalitis >.4):
    print("meningitis/encephalitis G03.9/G04.90")   
if (y_predACS <.4 and yhistACS >.4):
    print("ACS I24.9")  
if (yhistAAA >.4):
    print("AAA I71.4")  
if (y_predTachycardia <.4 and yhistTachycardia >.4):
     print("Tachycardia R00.0") 
if (y_predBradycardia <.4 and yhistBradycardia >.4):
    print("Bradycardia R00.1")  
if (y_predCHF <.4 and yhistCHF >.4):
    print("CHF I50.9")  
if (y_predHTN_Emergency <.4 and yhistHTN_Emergency >.4):
    print("HTN Emergency I16.1", "HTN I10") 
if (y_predHypotensionShock <.4 and yhistHypotensionShock >.4):
    print("Hypotension I95.9 Shock R57.0")  
if (y_predCardiac_arrest <.4 and yhistCardiac_arrest >.4):
    print("Cardiac arrest I46.9")   
if (y_predAirway_Obstruction <.4 and yhistAirway_Obstruction >.4):
    print("Airway_Obstruction J98.8")   
if (y_predPneumonia <.4 and yhistPneumonia >.4):
    print("Pneumonia J18.9", "Sepsis A41.9")    
if (y_predCOPD <.4 and yhistCOPD >.4):
    print("COPD J44.9")     
if (y_predPneumothorax <.4 and yhistPneumothorax >.4):
    print("Pneumothorax J93.9") 
if (y_predRespiratory_Failure <.4 and yhistRespiratory_Failure >.4):
    print("Respiratory Failure J96")    
if (y_predALF <.4 and yhistALF >.4):
    print("ALF K72.00 Cirrhosis K74.60")    
if (y_predGI_Bleed <.4 and yhistGI_Bleed >.4):
    print("GI Bleed K92.2") 
if (y_predPancreatitis <.4 and yhistPancreatitis >.4):
    print("Pancreatitis K85.90")    
if (y_predColitis <.4 and yhistColitis >.4):
    print("Colitis K52.9", "Sepsis A41.9")  
if (y_predGastroenteritis <.4 and yhistGastroenteritis >.4):
    print("Gastroenteritis K52.9")  
if (y_predBacteremia <.4 and yhistBacteremia >.4):
    print("Bacteremia R78.81", "Sepsis A41.9")  
if (y_predEndocarditis <.4 and yhistEndocarditis >.4):
    print("Endocarditis I38", "Sepsis A41.9")   
if (y_predUTI <.4 and yhistUTI >.4):
    print("UTI N39.0", "Sepsis A41.9")  
if (y_predvented <.4 and yhistvented >.4):
    print("vented ARF J96") 
if (y_predafib <.4 and yhistafib >.4):
    print("Atrial fibrillation I48.0")  
if (y_predDM <.4 and yhistDM >.4):
    print("DM E11.8")   
if (y_predPE <.4 and yhistPE >.4):
    print("PE I26.99")  
if (y_predICH <.4 and yhistICH >.4):
    print("ICH I62.9")  
if (y_predSDH <.4 and yhistSDH >.4):
    print("SDH S06.5X9A")   
if (y_predSAH <.4 and yhistSAH >.4):
    print("SAH I60.9")  
if (y_predSeizure <.4 and yhistSeizure >.4):
    print("Seizures R56.9") 
if (y_predESRD <.4 and data['Cr'][0] <= 2 and yhistESRD >.4):
    print("ESRD N18.6") 
if (y_predCKD <.4 and data['Cr'][0] <= 1.4 and yhistCKD >.4):
    print("CKD N18.9")  
if (y_predAKI <.4 and data['Cr'][0] <= 1.4 and yhistAKI >.4):
    print("AKI N17.9")  
if (y_predAsthma <.4 and yhistAsthma >.4):
    print("Asthma J45.909")     
if (y_predHTN <.4 and yhistHTN >.4) or (y_predHTN >.4 and supportv['SBP'][0] < 140):
    print("HTN I10")    
if (y_predEtOH_WD <.4 and yhistEtOH_WD >.4):
    print("EtOH Withdrawal F10.20")     
if (y_predCAD <.4 and yhistCAD >.4):
    print("CAD I25.10") 
if (y_predOSA <.4 and yhistOSA >.4):
    print("OSA G47.33") 
if (y_predhypothyroid <.4 and yhisthypothyroid >.4):
    print("Hypothyroid E03.9")  
if (y_predhyperthyroid <.4 and yhisthyperthyroid >.4):
    print("Hyperthyroid E05.90")    
if (y_predCVA <.4 and yhistCVA >.4):
    print("CVA I63.9")  
if (y_predDelirium <.4 and yhistDelirium >.4):
    print("Delirium R41.0") 
if (y_predDLD <.4 and yhistDLD >.4):
    print("HLD E78.5")      
if (y_predAI <.4 and yhistAI >.4):
    print("Adrenal Insufficiency E27.1")    
if (y_predLupus <.4 and yhistLupus >.4):
    print("Lupus M32.9")    
if (y_predCushing <.4 and yhistCushing >.4):
    print("Cushings Disease E24.0")     
if (y_predScleroderma <.4 and yhistScleroderma >.4):  
    print("Scleroderma M34.9")      
if (y_predPericardial_Effusion <.4 and yhistPericardial_Effusion >.4):
    print("Pericardial Effusion I31.39")    
if (y_predCholecystitis <.4 and yhistCholecystitis >.4):
    print("Cholecystitis  K81.9")   
if (y_predARDS <.4 and yhistARDS >.4):
    print("ARDS J80")   
if (y_predCellulitis <.4 and yhistCellulitis >.4):
    print("Cellulitis L03.90 Sepsis A41.9") 
if (y_predTBI <.4 and yhistTBI >.4):
    print("TBI/Head Trauma S06.9XAA S09.90XA")  
if (y_predBurn <.4 and yhistBurn >.4):
    print("Burn T30.0") 
if (y_predPAD <.4 and yhistPAD >.4):
    print("PAD I73.9")  
if (y_predBrain_Tumor <.4 and yhistBrain_Tumor >.4):
    print("Brain Tumor D49.6")  
if (y_predGERD <.4 and yhistGERD >.4):
    print("GERD K21.9") 
print()
print()
print('\033[1m' + '*********       Prophylaxis Checker     *********' + '\033[0m')
print()
print()
#Prophylaxis Checker
if (SUP >.4):
    print('\033[1m' + 'SUP Indicated' + '\033[0m')
    print('   Reason:')
    if (y_predvented >.4): 
        print('     Ventilator')
    if (y_predRespiratory_Failure >.4 and y_predvented <.4):
        print('     NIV')
    if (y_predTBI >.4):
        print('     TBI')
    if (y_predBurn >.4):
        print('     Burns')
    if (y_predCoagulopathy >.4):
        print('     Coagulopathy')
        if (supportv['INR'][0] >= 1.6):
            print('     INR', supportv['INR'][0])
        if (supportv['Plt'][0] <= 50):
            print('     Plt', supportv['Plt'][0])
        if (supportv['aPTT'][0] >= 50):
            print('     aPTT', supportv['aPTT'][0])
    if (y_predBrain_Tumor >.4):
        print('     Brain Tumor')
    if (y_predGERD >.4):
        print('     GERD')
    if (y_predGI_Bleed >.4):
        print('     GI Bleed')
    if (y_predSAH >.4 or y_predSDH >.4 or y_predICH >.4 or y_predTBI >.4):
        print('    Brain Bleed/Trauma')
else:
    print('\033[1m' + 'No SUP Indicated' + '\033[0m')
print()
if (chemDVT <.4 and mechDVT <.4):
    print('\033[1m' + 'No Mechanical or Chemical DVT PPx Indicated' + '\033[0m')
    print('   Reason:')
    if (y_predPAD >.4):
        print('    PAD')
    if (y_predCellulitis >.4):
        print('    Cellulitis')
    if (severecoag >.4):
        print('    Severe Coagulopathy')
    elif (y_predCoagulopathy >.4):
        print('    Coagulopathy')
        if (supportv['INR'][0] >= 1.6):
            print('     INR', supportv['INR'][0])
        if (supportv['Plt'][0] <= 50):
            print('     Plt', supportv['Plt'][0])
        if (supportv['aPTT'][0] >= 50):
            print('     aPTT', supportv['aPTT'][0])
    if (y_predSAH >.4 or y_predSDH >.4 or y_predICH >.4 or y_predTBI >.4):
        print('    Brain Bleed/Trauma')
    if (y_predGI_Bleed >.4):
        print('    GI Bleed')
    if (y_predAAA >.4):
        print('    AAA/Dissection')
    if (y_predBrain_Tumor >.4):
        print('     Brain Tumor')
    if (data['Hgb'][0]  < 7):
        print('    Hgb', data['Hgb'][0])
elif (chemDVT < .4 and mechDVT >= .4):
    print('\033[1m' + 'Only Mechanical DVT PPx Indicated' + '\033[0m')
    print('   Reason:')
    if (y_predSAH >.4 or y_predSDH >.4 or y_predICH >.4 or y_predTBI >.4):
        print('    Brain Bleed/Trauma')
    if (y_predGI_Bleed >.4):
        print('     GI Bleed')
    if (y_predAAA >.4):
        print('     AAA/Dissection')
    if (y_predCoagulopathy >.4):
        print('     Coagulopathy')
        if (supportv['INR'][0] >= 1.6):
            print('     INR', supportv['INR'][0])
        if (supportv['Plt'][0] <= 50):
            print('     Plt', supportv['Plt'][0])
        if (supportv['aPTT'][0] >= 50):
            print('     aPTT', supportv['aPTT'][0])
    if (y_predBrain_Tumor >.4):
        print('     Brain Tumor')
    if (data['Hgb'][0]  < 7):
        print('     Hgb', data['Hgb'][0]) 
elif (chemDVT >= .4):
        print('\033[1m' + 'Chemical DVT PPx Indicated' + '\033[0m')
print()
print()
print()
print('\033[1m' + '*********       Variable Display     *********' + '\033[0m')
print()
print()
XXX

RuntimeWarning: ignored

In [ ]:
#@title Insurance Grabber: Copy from 'Patient Name' to end of 'Member ID' Insurance Number (Methodist, Integris, Tuomey, St Mary NJ)

Info = "cc" #@param {type:"string"}
Hospital = 'Me' #@param {type:"string"}

inslp = medspacy.load(disable={"medspacy_context"})

inpreprocess_rules = [
    PreprocessingRule(
        r"\[\*\*[\d]{1,4}-[\d]{1,2}(-[\d]{1,2})?\*\*\]",
        repl="01-01-2010",
        desc="Replace MIMIC date brackets with a generic date."
    ),
    
    PreprocessingRule(
        r"\[\*\*[\d]{4}\*\*\]",
        repl="2010",
        desc="Replace MIMIC year brackets with a generic year."
    ),
    
    PreprocessingRule(
        r"dx'd", 
        repl="Diagnosed", 
        desc="Replace abbreviation"
    ),
    
    PreprocessingRule(
        r"tx'd", 
        repl="Treated", 
        desc="Replace abbreviation"
    ),
    
        PreprocessingRule(
        r"\[\*\*[^\]]+\]", 
        desc="Remove all other bracketed placeholder text from MIMIC"
    ),    
]

intarget_matcher = inslp.get_pipe("medspacy_target_matcher")

intarget_rules = [
        TargetRule("", "words",
               pattern=[{"IS_ALPHA": True}]),
        TargetRule("", "number",
               pattern=[{"LIKE_NUM": True}]),
        TargetRule("", "alphanum",
               pattern = [{"TEXT": {"REGEX": "[A-Za-z0-9]+"}}]),
               
        TargetRule("", "dater",              
               pattern = [{"TEXT": {"REGEX": "(?:0?[1-9]|1[012])"}}, {"TEXT": {"REGEX": "(?:/|-)"}}, {"TEXT": {"REGEX": "(?:0?[1-9]|[12][0-9]|3[01])"}}, {"TEXT": {"REGEX": "(?:/|-)"}}, {"TEXT": {"REGEX": "[0-9]{4}"}}]),
 

        TargetRule("", "phoner",
               pattern = [{"LIKE_NUM": True}, {"TEXT": {"REGEX": r"\d{3}[-\.\s]\d{3}[-\.\s]\d{4}|\(\d{3}\)\s*\d{3}[-\.\s]\d{4}|\d{3}[-\.\s]\d{4}"}}]),
        TargetRule("", "phoner",
               pattern = [{"LIKE_NUM": True}, {"TEXT": {"REGEX": r"^((?:\+\d{2}[-\.\s]??|\d{4}[-\.\s]??)?(?:\d{3}[-\.\s]??\d{3}[-\.\s]??\d{4}|\(\d{3}\)\s*\d{3}[-\.\s]??\d{4}|\d{3}[-\.\s]??\d{4}))$"}}]),
        TargetRule("", "phoner",
               pattern = [{"TEXT": {"REGEX": r"^(\+\d{1,2}\s)?\(?\d{3}\)?[\s.-]?\d{3}[\s.-]?\d{4}$"}}]), 
        TargetRule("", "phoner",
               pattern = [{"TEXT": {"REGEX": r"^(\+\d{1,2}\s?)?1?\-?\.?\s?\(?\d{3}\)?[\s.-]?\d{3}[\s.-]?\d{4}$"}}]),
        TargetRule("", "phoner",
               pattern = [{"TEXT": {"REGEX": r"\d{3}-\d{3}-\d{4}"}}]),    
        TargetRule("", "phoner",
               pattern = [{"TEXT": {"REGEX": r"\d{3}[-\.\s]\d{3}[-\.\s]\d{4}|\(\d{3}\)\s*\d{3}[-\.\s]\d{4}|\d{3}[-\.\s]\d{4}"}}]),
        TargetRule("", "phoner",
               pattern = [{"TEXT": {"REGEX": r"\d{3}[-\.\s]\d{3}[-\.\s]\d{4}|\(\d{3}\)\s*\d{3}[-\.\s]\d{4}|\d{3}[-\.\s]\d{4}"}}]),
        TargetRule("", "phoner",
               pattern = [{"TEXT": {"REGEX": r"^\(?(\d{3})\)?[-.\s]?(\d{3})[-.\s]?(\d{4})$"}}]),
        TargetRule("", "phoner",
               pattern = [{"TEXT": {"REGEX": r"(\+)?([0-9]{1,3})?( )?(\([0-9]{1,3}\))?( )?[(\d+((\-\d+)+)]{10,15}"}}]),
        TargetRule("", "phoner",
               pattern = [{"TEXT": {"REGEX": r"\d{3}[-\.\s]\d{3}[-\.\s]\d{4}|\(\d{3}\)\s*\d{3}[-\.\s]\d{4}|\d{3}[-\.\s]\d{4}"}}]),
        TargetRule("", "phoner",
               pattern = [{"TEXT": {"REGEX": r"^\d{3}[-\.\s]\d{3}[-\.\s]\d{4}|\(\d{3}\)\s*\d{3}[-\.\s]\d{4}|\d{3}[-\.\s]\d{4}$"}}]),
        TargetRule("", "phoner",
               pattern = [{"TEXT": {"REGEX": r"\+?1?\s*\(?-*\.*(\d{3})\)?\.*-*\s*(\d{3})\.*-*\s*(\d{4})$"}}]),
        TargetRule("", "phoner",
               pattern = [{"TEXT": {"REGEX": r"^(?:(?:\(\d{3}\)|\d{3}))?[-\s.]?(\d{3})[-\s.]?(\d{4})$"}}]),
        TargetRule("", "phoner1",
               pattern = [{"TEXT": "(","OP": "?"}, {"SHAPE": "ddd"}, {"TEXT": ")","OP": "?"}, {"TEXT": "-", "OP": "?"}, {"SHAPE": "ddd"}, {"TEXT": "-", "OP": "?"}, {"SHAPE": "dddd"}]),
        TargetRule("", "phoner",
               pattern = [{"TEXT": {"REGEX": r"^(?:(?:\(\d{3}\)|\d{3}))?[-\s.]?(\d{3})[-\s.]?(\d{4})$"}}]),
]

intarget_matcher.add(intarget_rules)

incontext_rules = [
    ConTextRule(literal='Na', category='NA', pattern=[{'LOWER': {'IN': ['na:', 'na', 'sodium', 'sodio', 'sod', 'sodium:', 'sodio:', 'sod:']}}], max_targets = 1, direction='FORWARD'),
    ConTextRule(literal='K', category='K', pattern=[{'LOWER': {'IN': ['k+:', 'potassium', 'potassio', 'pot', 'k+', 'potassium:', 'potassio:', 'pot:']}}], max_targets = 1, direction='FORWARD'),
    ConTextRule(literal='preferred', category='PREFERRED', pattern=None, max_targets = 1, direction='BACKWARD', allowed_types={"dater"}),
    ConTextRule(literal='mobile', category='mobile', pattern=None, max_targets = 1, direction='BACKWARD', allowed_types={"dater"}),
    ConTextRule(literal='home', category='home', pattern=None, max_targets = 1, direction='BACKWARD', allowed_types={"dater"}),
    ConTextRule(literal='Payor', category='PAYOR', pattern=None, max_targets = 1, direction='BACKWARD',),
    ConTextRule(literal='Patient Name', category='PTNAME', pattern=None, max_targets = 3, direction='BACKWARD',),
    ConTextRule(literal="Member ID", category="member_id", pattern=None, max_targets = 1, direction='FORWARD'),
    ConTextRule(literal="ID", category="id", 
                pattern=[{"lower": {"in":  ["Id", "id", "iD"]}}], direction='FORWARD', allowed_types={"number", "alphanum"}),
    ConTextRule(literal="Group", category="group", 
                pattern=[{"lower": {"in":  ["group", "Group", "GROUP"]}}], direction='FORWARD', excluded_types={"dates", "number", "alphanum"}),
    ConTextRule(literal="Member ID", category="member_id", 
                pattern=[{"lower": {"in":  ["member", "Member"]}},
                         {"lower": {"in": ["id", "ID"]}}], max_targets = 1, direction='FORWARD', allowed_types={"number", "alphanum"}),
                 
]  

incontext = inslp.add_pipe("medspacy_context")
incontext.add(incontext_rules)


indoc = inslp(Info)
#indoc2 = inslp(notess)
#Name Grabber
if len(re.findall(r'{}'.format("Name"), Info)) > 0 and len(re.findall(r'{}'.format("Legal"), Info)) > 0:
  token_positionName = [token.i for token in indoc if token.text == "Name"][0] + 1
  token_positionLegal = [token.i for token in indoc if token.text == "Legal"][0] - 1
  Name = str(indoc[(token_positionName):(token_positionLegal)])
  PtName = re.sub(r'[\n\d()]', '', Name)
else:
  PtName = ""

#DOB Grabber
if Hospital == "Tuomey" or Hospital == "tuomey" or Hospital == "Tu" or Hospital == "tu":
  if len(re.findall(r'{}'.format("DOB"), Info)) > 0 and len(re.findall(r'{}'.format("Patient"), Info)) > 0:
    token_positionDOB = [token.i for token in indoc if token.text == "DOB"][0] + 1
    token_positionPatient = [token.i for token in indoc if token.text == "Date"][0]
    def extract_dates(indoc):
        dates = []
        for token in indoc[(token_positionDOB):(token_positionPatient)]:
            if token.ent_type_ == "dater":
                dates.append(token.text)
        return "".join(dates)
    PtDOB = extract_dates(indoc)
  else:
    PtDOB = ""
else:
  if len(re.findall(r'{}'.format("DOB"), Info)) > 0 and len(re.findall(r'{}'.format("Patient"), Info)) > 0:
    token_positionDOB = [token.i for token in indoc if token.text == "DOB"][0] + 1
    token_positionPatient = [token.i for token in indoc if token.text == "Patient"][1] - 1
    def extract_dates(indoc):
        dates = []
        for token in indoc[(token_positionDOB):(token_positionPatient)]:
            if token.ent_type_ == "dater":
                dates.append(token.text)
        return "".join(dates)
    PtDOB = extract_dates(indoc)
  else:
    PtDOB = ""

#Phone Grabber
if Hospital == "Tuomey" or Hospital == "tuomey" or Hospital == "Tu" or Hospital == "tu":
  if [(len(re.findall(r'{}'.format("Phone"), Info)) > 0 or len(re.findall(r'{}'.format("Contact Numbers"), Info)) > 0) and len(re.findall(r'{}'.format("Date"), Info)) > 0]:
    token_positionPhone = [token.i for token in indoc if token.text == "Phone" or token.text == "Contact Numbers"][0] - 1
    token_positionDate = [token.i for token in indoc if token.text == "Active"][0] + 1
    phonepos = indoc[token_positionPhone:token_positionDate].text
    indocq = inslp(phonepos)
    pp = pd.DataFrame(indocq._.context_graph.edges)
    pp.columns=['column1', 'column2']
    pp['column1'] = pp['column1'].astype(str)
    pp['column2'] = pp['column2'].astype(str)
    preferrednumber = pp.loc[pp['column2'].str.contains("PREFERRED")]
    mobilenumber = pp.loc[pp['column2'].str.contains("MOBILE")]
    homenumber = pp.loc[pp['column2'].str.contains("HOME")]
    if len(preferrednumber) > 0:
      preferrednumber = preferrednumber 
    elif len(mobilenumber) > 0:
      preferrednumber = mobilenumber
    elif len(homenumber) > 0:
      preferrednumber = homenumber
    else:
      preferrednumber = ""
    preferrednumber['column1'] = preferrednumber['column1'].astype(str)
    preferrednumber = list(preferrednumber['column1'])
    preferrednumber = [string for string in preferrednumber if any(char.isdigit() for char in string)]
    preferrednumber = "".join(preferrednumber)
  else:
    preferrednumber = ""
else:
  if [(len(re.findall(r'{}'.format("Phone"), Info)) > 0 or len(re.findall(r'{}'.format("Contact Numbers"), Info)) > 0) and len(re.findall(r'{}'.format("Date"), Info)) > 0]:
    token_positionPhone = [token.i for token in indoc if token.text == "Phone" or token.text == "Contact Numbers"][0] - 1
    token_positionDate = [token.i for token in indoc if token.text == "Date"][0] + 1
    phonepos = indoc[token_positionPhone:token_positionDate].text
    indocq = inslp(phonepos)
    pp = pd.DataFrame(indocq._.context_graph.edges)
    pp.columns=['column1', 'column2']
    pp['column1'] = pp['column1'].astype(str)
    pp['column2'] = pp['column2'].astype(str)
    preferrednumber = pp.loc[pp['column2'].str.contains("PREFERRED")]
    mobilenumber = pp.loc[pp['column2'].str.contains("MOBILE")]
    homenumber = pp.loc[pp['column2'].str.contains("HOME")]
    if len(preferrednumber) > 0:
      preferrednumber = preferrednumber 
    elif len(mobilenumber) > 0:
      preferrednumber = mobilenumber
    elif len(homenumber) > 0:
      preferrednumber = homenumber
    else:
      preferrednumber = ""
    preferrednumber['column1'] = preferrednumber['column1'].astype(str)
    preferrednumber = list(preferrednumber['column1'])
    preferrednumber = [string for string in preferrednumber if any(char.isdigit() for char in string)]
    preferrednumber = "".join(preferrednumber)
  else:
    preferrednumber = ""

 #Insurance Carrier Grabber
pi1 = len(re.findall(r'Primary Coverage', Info))
pi2 = len(re.findall(r'Secondary Coverage', Info))
pi3 = len(re.findall(r'Tertiary Coverage', Info))
pi4 = len(re.findall(r'Quarternary Coverage', Info))
pi2a = (pi1 + pi2) 
pi3a = (pi1 + pi2 + pi3) 
pi4a = (pi1 + pi2 + pi3 + pi4) 

if Hospital == "Methodist" or Hospital == "Me" or Hospital == "me" or Hospital == "Tuomey" or Hospital == "tuomey" or Hospital == "Tu" or Hospital == "tu" or Hospital == "Integris" or Hospital == "integris" or Hospital == "Bass" or Hospital == "bass":
  if len(re.findall(r'{}'.format("Group"), Info)) > 0 and len(re.findall(r'{}'.format("Payor"), Info)) > 0:
    token_positionGroup = [token.i for token in indoc if token.text == "Group"]
    token_positionPayor = [token.i for token in indoc if token.text == "Payor" or token.text == "Coverage"]
    if len(token_positionGroup) > 7:
      token_positionGroup1 = token_positionGroup[2]
      token_positionPayor1 = token_positionPayor[1 + pi1]
      token_positionGroup2 = token_positionGroup[4]
      token_positionPayor2 = token_positionPayor[5 + pi2a]
      token_positionGroup3 = token_positionGroup[6]
      token_positionPayor3 = token_positionPayor[9 + pi3a]
      token_positionGroup4 = token_positionGroup[8]
      token_positionPayor4 = token_positionPayor[13 + pi4a]  
      Ins4 = indoc[token_positionGroup4:token_positionPayor4].text
      Ins3 = indoc[token_positionGroup3:token_positionPayor3].text
      Ins2 = indoc[token_positionGroup2:token_positionPayor2].text
      Ins1 = indoc[token_positionGroup1:token_positionPayor1].text
    elif len(token_positionGroup) <= 7 and len(token_positionGroup) > 5:
      token_positionGroup1 = token_positionGroup[2]
      token_positionPayor1 = token_positionPayor[1 + pi1]
      token_positionGroup2 = token_positionGroup[4]
      token_positionPayor2 = token_positionPayor[5 + pi2a]
      token_positionGroup3 = token_positionGroup[6]
      token_positionPayor3 = token_positionPayor[9 + pi3a]
      Ins3 = indoc[token_positionGroup3:token_positionPayor3].text
      Ins2 = indoc[token_positionGroup2:token_positionPayor2].text
      Ins1 = indoc[token_positionGroup1:token_positionPayor1].text
    elif len(token_positionGroup) <= 5 and len(token_positionGroup) > 3:
      token_positionGroup1 = token_positionGroup[2]
      token_positionPayor1 = token_positionPayor[1 + pi1]
      token_positionGroup2 = token_positionGroup[4]
      token_positionPayor2 = token_positionPayor[5 + pi2a]
      Ins2 = indoc[token_positionGroup2:token_positionPayor2].text
      Ins1 = indoc[token_positionGroup1:token_positionPayor1].text
    elif len(token_positionGroup) <= 3:
      token_positionGroup1 = token_positionGroup[2]
      token_positionPayor1 = token_positionPayor[1 + pi1]
      Ins1 = indoc[token_positionGroup1:token_positionPayor1].text
    if len(token_positionGroup) > 7:
      indocd4 = inslp(Ins4)
      indocc3 = inslp(Ins3)
      indocb2 = inslp(Ins2)
      indoca1 = inslp(Ins1)
    elif len(token_positionGroup) <= 7 and len(token_positionGroup) > 5:
      indocc3 = inslp(Ins3)
      indocb2 = inslp(Ins2)
      indoca1 = inslp(Ins1)
    elif len(token_positionGroup) <= 5 and len(token_positionGroup) > 3:
      indocb2 = inslp(Ins2)
      indoca1 = inslp(Ins1)
    else:
      indoca1 = inslp(Ins1)
    if len(token_positionGroup) > 7:
      pp1a = pd.DataFrame(indoca1._.context_graph.edges)
      pp1a.columns=['column1', 'column2']
      pp1a['column1'] = pp1a['column1'].astype(str)
      pp1a['column2'] = pp1a['column2'].astype(str)
      selected_rowsIns1a = pp1a.loc[pp1a['column2'].str.contains("GROUP")]
      selected_rowsIns1a['column1'] = selected_rowsIns1a['column1'].astype(str)
      Inspol1 = ' '.join(list(selected_rowsIns1a['column1']))
      pp2b = pd.DataFrame(indocb2._.context_graph.edges)
      pp2b.columns=['column1', 'column2']
      pp2b['column1'] = pp2b['column1'].astype(str)
      pp2b['column2'] = pp2b['column2'].astype(str)
      selected_rowsIns2b = pp2b.loc[pp2b['column2'].str.contains("GROUP")]
      selected_rowsIns2b['column1'] = selected_rowsIns2b['column1'].astype(str)
      Inspol2 = ' '.join(list(selected_rowsIns2b['column1']))
      pp3c = pd.DataFrame(indocc3._.context_graph.edges)
      pp3c.columns=['column1', 'column2']
      pp3c['column1'] = pp3c['column1'].astype(str)
      pp3c['column2'] = pp3c['column2'].astype(str)
      selected_rowsIns3c = pp3c.loc[pp3c['column2'].str.contains("GROUP")]
      selected_rowsIns3c['column1'] = selected_rowsIns3c['column1'].astype(str)
      Inspol3 = ' '.join(list(selected_rowsIns3c['column1']))
      pp4d = pd.DataFrame(indocd4._.context_graph.edges)
      pp4d.columns=['column1', 'column2']
      pp4d['column1'] = pp4d['column1'].astype(str)
      pp4d['column2'] = pp4d['column2'].astype(str)
      selected_rowsIns4d = pp4d.loc[pp4d['column2'].str.contains("GROUP")]
      selected_rowsIns4d['column1'] = selected_rowsIns4d['column1'].astype(str)
      Inspol4 = ' '.join(list(selected_rowsIns4d['column1']))
      Inspol1 = my_string = ''.join(Inspol1)
      Inspol2 = my_string = ''.join(Inspol2)
      Inspol3 = my_string = ''.join(Inspol3)
      Inspol4 = my_string = ''.join(Inspol4)
    elif len(token_positionGroup) <= 7 and len(token_positionGroup) > 5:
      pp1a = pd.DataFrame(indoca1._.context_graph.edges)
      pp1a.columns=['column1', 'column2']
      pp1a['column1'] = pp1a['column1'].astype(str)
      pp1a['column2'] = pp1a['column2'].astype(str)
      selected_rowsIns1a = pp1a.loc[pp1a['column2'].str.contains("GROUP")]
      selected_rowsIns1a['column1'] = selected_rowsIns1a['column1'].astype(str)
      Inspol1 = ' '.join(list(selected_rowsIns1a['column1']))
      pp2b = pd.DataFrame(indocb2._.context_graph.edges)
      pp2b.columns=['column1', 'column2']
      pp2b['column1'] = pp2b['column1'].astype(str)
      pp2b['column2'] = pp2b['column2'].astype(str)
      selected_rowsIns2b = pp2b.loc[pp2b['column2'].str.contains("GROUP")]
      selected_rowsIns2b['column1'] = selected_rowsIns2b['column1'].astype(str)
      Inspol2 = ' '.join(list(selected_rowsIns2b['column1']))
      pp3c = pd.DataFrame(indocc3._.context_graph.edges)
      pp3c.columns=['column1', 'column2']
      pp3c['column1'] = pp3c['column1'].astype(str)
      pp3c['column2'] = pp3c['column2'].astype(str)
      selected_rowsIns3c = pp3c.loc[pp3c['column2'].str.contains("GROUP")]
      selected_rowsIns3c['column1'] = selected_rowsIns3c['column1'].astype(str)
      Inspol3 = ' '.join(list(selected_rowsIns3c['column1']))
      Inspol1 = my_string = ''.join(Inspol1)
      Inspol2 = my_string = ''.join(Inspol2)
      Inspol3 = my_string = ''.join(Inspol3)
    elif len(token_positionGroup) <= 5 and len(token_positionGroup) > 3:
      pp1a = pd.DataFrame(indoca1._.context_graph.edges)
      pp1a.columns=['column1', 'column2']
      pp1a['column1'] = pp1a['column1'].astype(str)
      pp1a['column2'] = pp1a['column2'].astype(str)
      selected_rowsIns1a = pp1a.loc[pp1a['column2'].str.contains("GROUP")]
      selected_rowsIns1a['column1'] = selected_rowsIns1a['column1'].astype(str)
      Inspol1 = ' '.join(list(selected_rowsIns1a['column1']))
      pp2b = pd.DataFrame(indocb2._.context_graph.edges)
      pp2b.columns=['column1', 'column2']
      pp2b['column1'] = pp2b['column1'].astype(str)
      pp2b['column2'] = pp2b['column2'].astype(str)
      selected_rowsIns2b = pp2b.loc[pp2b['column2'].str.contains("GROUP")]
      selected_rowsIns2b['column1'] = selected_rowsIns2b['column1'].astype(str)
      Inspol2 = ' '.join(list(selected_rowsIns2b['column1']))
      Inspol1 = my_string = ''.join(Inspol1)
      Inspol2 = my_string = ''.join(Inspol2)
    elif len(token_positionGroup) <= 3:
      pp1a = pd.DataFrame(indoca1._.context_graph.edges)
      pp1a.columns=['column1', 'column2']
      pp1a['column1'] = pp1a['column1'].astype(str)
      pp1a['column2'] = pp1a['column2'].astype(str)
      selected_rowsIns1a = pp1a.loc[pp1a['column2'].str.contains("GROUP")]
      selected_rowsIns1a['column1'] = selected_rowsIns1a['column1'].astype(str)
      Inspol1 = ' '.join(list(selected_rowsIns1a['column1']))
      Inspol1 = my_string = ''.join(Inspol1)
    if len(token_positionGroup) > 7:
      InsurancePolicy = [Inspol1, Inspol2, Inspol3, Inspol4]
      InsurancePolicy = '/'.join(InsurancePolicy)  
    elif len(token_positionGroup) <= 7 and len(token_positionGroup) > 5:
      InsurancePolicy = [Inspol1, Inspol2, Inspol3]
      InsurancePolicy = '/'.join(InsurancePolicy)
    elif len(token_positionGroup) <= 5 and len(token_positionGroup) > 3:
      InsurancePolicy = [Inspol1, Inspol2]
      InsurancePolicy = '/'.join(InsurancePolicy)
    else:
      InsurancePolicy = Inspol1
  else:
    InsurancePolicy = ""
else:
  if len(re.findall(r'{}'.format("Group"), Info)) > 0 and len(re.findall(r'{}'.format("Payor"), Info)) > 0:
    token_positionGroup = [token.i for token in indoc if token.text == "Group"]
    token_positionPayor = [token.i for token in indoc if token.text == "Payor" or token.text == "Coverage"]
    if len(token_positionGroup) > 7:
      token_positionGroup1 = token_positionGroup[1]
      token_positionPayor1 = token_positionPayor[1 + pi1]
      token_positionGroup2 = token_positionGroup[3]
      token_positionPayor2 = token_positionPayor[5 + pi2a]
      token_positionGroup3 = token_positionGroup[5]
      token_positionPayor3 = token_positionPayor[9 + pi3a]
      token_positionGroup4 = token_positionGroup[7]
      token_positionPayor4 = token_positionPayor[13 + pi4a]  
      Ins4 = indoc[token_positionGroup4:token_positionPayor4].text
      Ins3 = indoc[token_positionGroup3:token_positionPayor3].text
      Ins2 = indoc[token_positionGroup2:token_positionPayor2].text
      Ins1 = indoc[token_positionGroup1:token_positionPayor1].text
    elif len(token_positionGroup) <= 7 and len(token_positionGroup) > 5:
      token_positionGroup1 = token_positionGroup[1]
      token_positionPayor1 = token_positionPayor[1 + pi1]
      token_positionGroup2 = token_positionGroup[3]
      token_positionPayor2 = token_positionPayor[5 + pi2a]
      token_positionGroup3 = token_positionGroup[5]
      token_positionPayor3 = token_positionPayor[9 + pi3a]
      Ins3 = indoc[token_positionGroup3:token_positionPayor3].text
      Ins2 = indoc[token_positionGroup2:token_positionPayor2].text
      Ins1 = indoc[token_positionGroup1:token_positionPayor1].text
    elif len(token_positionGroup) <= 5 and len(token_positionGroup) > 3:
      token_positionGroup1 = token_positionGroup[1]
      token_positionPayor1 = token_positionPayor[1 + pi1]
      token_positionGroup2 = token_positionGroup[3]
      token_positionPayor2 = token_positionPayor[5 + pi2a]
      Ins2 = indoc[token_positionGroup2:token_positionPayor2].text
      Ins1 = indoc[token_positionGroup1:token_positionPayor1].text
    elif len(token_positionGroup) <= 3:
      token_positionGroup1 = token_positionGroup[1]
      token_positionPayor1 = token_positionPayor[1 + pi1]
      Ins1 = indoc[token_positionGroup1:token_positionPayor1].text
    if len(token_positionGroup) > 7:
      indocd4 = inslp(Ins4)
      indocc3 = inslp(Ins3)
      indocb2 = inslp(Ins2)
      indoca1 = inslp(Ins1)
    elif len(token_positionGroup) <= 7 and len(token_positionGroup) > 5:
      indocc3 = inslp(Ins3)
      indocb2 = inslp(Ins2)
      indoca1 = inslp(Ins1)
    elif len(token_positionGroup) <= 5 and len(token_positionGroup) > 3:
      indocb2 = inslp(Ins2)
      indoca1 = inslp(Ins1)
    else:
      indoca1 = inslp(Ins1)
    if len(token_positionGroup) > 7:
      pp1a = pd.DataFrame(indoca1._.context_graph.edges)
      pp1a.columns=['column1', 'column2']
      pp1a['column1'] = pp1a['column1'].astype(str)
      pp1a['column2'] = pp1a['column2'].astype(str)
      selected_rowsIns1a = pp1a.loc[pp1a['column2'].str.contains("GROUP")]
      selected_rowsIns1a['column1'] = selected_rowsIns1a['column1'].astype(str)
      Inspol1 = ' '.join(list(selected_rowsIns1a['column1']))
      pp2b = pd.DataFrame(indocb2._.context_graph.edges)
      pp2b.columns=['column1', 'column2']
      pp2b['column1'] = pp2b['column1'].astype(str)
      pp2b['column2'] = pp2b['column2'].astype(str)
      selected_rowsIns2b = pp2b.loc[pp2b['column2'].str.contains("GROUP")]
      selected_rowsIns2b['column1'] = selected_rowsIns2b['column1'].astype(str)
      Inspol2 = ' '.join(list(selected_rowsIns2b['column1']))
      pp3c = pd.DataFrame(indocc3._.context_graph.edges)
      pp3c.columns=['column1', 'column2']
      pp3c['column1'] = pp3c['column1'].astype(str)
      pp3c['column2'] = pp3c['column2'].astype(str)
      selected_rowsIns3c = pp3c.loc[pp3c['column2'].str.contains("GROUP")]
      selected_rowsIns3c['column1'] = selected_rowsIns3c['column1'].astype(str)
      Inspol3 = ' '.join(list(selected_rowsIns3c['column1']))
      pp4d = pd.DataFrame(indocd4._.context_graph.edges)
      pp4d.columns=['column1', 'column2']
      pp4d['column1'] = pp4d['column1'].astype(str)
      pp4d['column2'] = pp4d['column2'].astype(str)
      selected_rowsIns4d = pp4d.loc[pp4d['column2'].str.contains("GROUP")]
      selected_rowsIns4d['column1'] = selected_rowsIns4d['column1'].astype(str)
      Inspol4 = ' '.join(list(selected_rowsIns4d['column1']))
      Inspol1 = my_string = ''.join(Inspol1)
      Inspol2 = my_string = ''.join(Inspol2)
      Inspol3 = my_string = ''.join(Inspol3)
      Inspol4 = my_string = ''.join(Inspol4)
    elif len(token_positionGroup) <= 7 and len(token_positionGroup) > 5:
      pp1a = pd.DataFrame(indoca1._.context_graph.edges)
      pp1a.columns=['column1', 'column2']
      pp1a['column1'] = pp1a['column1'].astype(str)
      pp1a['column2'] = pp1a['column2'].astype(str)
      selected_rowsIns1a = pp1a.loc[pp1a['column2'].str.contains("GROUP")]
      selected_rowsIns1a['column1'] = selected_rowsIns1a['column1'].astype(str)
      Inspol1 = ' '.join(list(selected_rowsIns1a['column1']))
      pp2b = pd.DataFrame(indocb2._.context_graph.edges)
      pp2b.columns=['column1', 'column2']
      pp2b['column1'] = pp2b['column1'].astype(str)
      pp2b['column2'] = pp2b['column2'].astype(str)
      selected_rowsIns2b = pp2b.loc[pp2b['column2'].str.contains("GROUP")]
      selected_rowsIns2b['column1'] = selected_rowsIns2b['column1'].astype(str)
      Inspol2 = ' '.join(list(selected_rowsIns2b['column1']))
      pp3c = pd.DataFrame(indocc3._.context_graph.edges)
      pp3c.columns=['column1', 'column2']
      pp3c['column1'] = pp3c['column1'].astype(str)
      pp3c['column2'] = pp3c['column2'].astype(str)
      selected_rowsIns3c = pp3c.loc[pp3c['column2'].str.contains("GROUP")]
      selected_rowsIns3c['column1'] = selected_rowsIns3c['column1'].astype(str)
      Inspol3 = ' '.join(list(selected_rowsIns3c['column1']))
      Inspol1 = my_string = ''.join(Inspol1)
      Inspol2 = my_string = ''.join(Inspol2)
      Inspol3 = my_string = ''.join(Inspol3)
    elif len(token_positionGroup) <= 5 and len(token_positionGroup) > 3:
      pp1a = pd.DataFrame(indoca1._.context_graph.edges)
      pp1a.columns=['column1', 'column2']
      pp1a['column1'] = pp1a['column1'].astype(str)
      pp1a['column2'] = pp1a['column2'].astype(str)
      selected_rowsIns1a = pp1a.loc[pp1a['column2'].str.contains("GROUP")]
      selected_rowsIns1a['column1'] = selected_rowsIns1a['column1'].astype(str)
      Inspol1 = ' '.join(list(selected_rowsIns1a['column1']))
      pp2b = pd.DataFrame(indocb2._.context_graph.edges)
      pp2b.columns=['column1', 'column2']
      pp2b['column1'] = pp2b['column1'].astype(str)
      pp2b['column2'] = pp2b['column2'].astype(str)
      selected_rowsIns2b = pp2b.loc[pp2b['column2'].str.contains("GROUP")]
      selected_rowsIns2b['column1'] = selected_rowsIns2b['column1'].astype(str)
      Inspol2 = ' '.join(list(selected_rowsIns2b['column1']))
      Inspol1 = my_string = ''.join(Inspol1)
      Inspol2 = my_string = ''.join(Inspol2)
    elif len(token_positionGroup) <= 3:
      pp1a = pd.DataFrame(indoca1._.context_graph.edges)
      pp1a.columns=['column1', 'column2']
      pp1a['column1'] = pp1a['column1'].astype(str)
      pp1a['column2'] = pp1a['column2'].astype(str)
      selected_rowsIns1a = pp1a.loc[pp1a['column2'].str.contains("GROUP")]
      selected_rowsIns1a['column1'] = selected_rowsIns1a['column1'].astype(str)
      Inspol1 = ' '.join(list(selected_rowsIns1a['column1']))
      Inspol1 = my_string = ''.join(Inspol1)
    if len(token_positionGroup) > 7:
      InsurancePolicy = [Inspol1, Inspol2, Inspol3, Inspol4]
      InsurancePolicy = '/'.join(InsurancePolicy)  
    elif len(token_positionGroup) <= 7 and len(token_positionGroup) > 5:
      InsurancePolicy = [Inspol1, Inspol2, Inspol3]
      InsurancePolicy = '/'.join(InsurancePolicy)
    elif len(token_positionGroup) <= 5 and len(token_positionGroup) > 3:
      InsurancePolicy = [Inspol1, Inspol2]
      InsurancePolicy = '/'.join(InsurancePolicy)
    else:
      InsurancePolicy = Inspol1
  else:
    InsurancePolicy = ""
    
#Insurance Number Grabber
if len(re.findall(r'{}'.format("Payor"), Info)) > 0 and len(re.findall(r'{}'.format("ID"), Info)) > 0:
  token_positionPayor = [token.i for token in indoc if token.text == "Payor"]
  token_positionID = [token.i for token in indoc if token.text == "ID"]
  if len(token_positionGroup) > 7:
    token_positionID1 = [token.i for token in indoc if token.text == "ID"][0]
    token_positionPayor1a = [token.i for token in indoc if token.text == "Payor"][4]
    token_positionID2 = [token.i for token in indoc if token.text == "ID"][1]
    token_positionPayor2a = [token.i for token in indoc if token.text == "Payor"][8]
    token_positionID3 = [token.i for token in indoc if token.text == "ID"][2]
    token_positionPayor3a = [token.i for token in indoc if token.text == "Payor"][12]
    token_positionID4 = [token.i for token in indoc if token.text == "ID"][3]
    InsNum4 = indoc[token_positionID4:].text
    InsNum3 = indoc[token_positionID3:token_positionPayor3a].text
    InsNum2 = indoc[token_positionID2:token_positionPayor2a].text
    InsNum1 = indoc[token_positionID1:token_positionPayor1a].text
  elif len(token_positionGroup) <= 7 and len(token_positionGroup) > 5:
    token_positionID1 = [token.i for token in indoc if token.text == "ID"][0]
    token_positionPayor1a = [token.i for token in indoc if token.text == "Payor"][4]
    token_positionID2 = [token.i for token in indoc if token.text == "ID"][1]
    token_positionPayor2a = [token.i for token in indoc if token.text == "Payor"][8]
    token_positionID3 = [token.i for token in indoc if token.text == "ID"][2]
    InsNum3 = indoc[token_positionID3:].text
    InsNum2 = indoc[token_positionID2:token_positionPayor2a].text
    InsNum1 = indoc[token_positionID1:token_positionPayor1a].text
  elif len(token_positionGroup) <= 5 and len(token_positionGroup) > 3:
    token_positionID1 = [token.i for token in indoc if token.text == "ID"][0]
    token_positionPayor1a = [token.i for token in indoc if token.text == "Payor"][4]
    token_positionID2 = [token.i for token in indoc if token.text == "ID"][1]
    InsNum2 = indoc[token_positionID2:].text
    InsNum1 = indoc[token_positionID1:token_positionPayor1a].text
  elif len(token_positionGroup) <= 3:
    token_positionID1 = [token.i for token in indoc if token.text == "ID"][0]
    InsNum1 = indoc[token_positionID1:].text
  if len(token_positionGroup) > 7:
    indoca = inslp(InsNum1)
    indocb = inslp(InsNum2)
    indocc = inslp(InsNum3)
    indocd = inslp(InsNum4)
  elif len(token_positionGroup) <= 7 and len(token_positionGroup) > 5:
    indoca = inslp(InsNum1)
    indocb = inslp(InsNum2)
    indocc = inslp(InsNum3)
  elif len(token_positionGroup) <= 5 and len(token_positionGroup) > 3:
    indoca = inslp(InsNum1)
    indocb = inslp(InsNum2)
  else:
    indoca = inslp(InsNum1)
  if len(token_positionGroup) > 7:
    pp1 = pd.DataFrame(indoca._.context_graph.edges)
    pp1.columns=['column1', 'column2']
    pp1['column1'] = pp1['column1'].astype(str)
    pp1['column2'] = pp1['column2'].astype(str)
    selected_rowsIns1 = pp1.loc[pp1['column2'].str.contains("ID")]
    selected_rowsIns1['column1'] = selected_rowsIns1['column1'].astype(str)
    InsNumm1 = ''.join(list(selected_rowsIns1['column1']))
    pp2 = pd.DataFrame(indocb._.context_graph.edges)
    pp2.columns=['column1', 'column2']
    pp2['column1'] = pp2['column1'].astype(str)
    pp2['column2'] = pp2['column2'].astype(str)
    selected_rowsIns2 = pp2.loc[pp2['column2'].str.contains("ID")]
    selected_rowsIns2['column1'] = selected_rowsIns2['column1'].astype(str)
    InsNumm2 = ''.join(list(selected_rowsIns2['column1']))
    pp3 = pd.DataFrame(indocc._.context_graph.edges)
    pp3.columns=['column1', 'column2']
    pp3['column1'] = pp3['column1'].astype(str)
    pp3['column2'] = pp3['column2'].astype(str)
    selected_rowsIns3 = pp3.loc[pp3['column2'].str.contains("ID")]
    selected_rowsIns3['column1'] = selected_rowsIns3['column1'].astype(str)
    InsNumm3 = ''.join(list(selected_rowsIns3['column1']))
    pp4 = pd.DataFrame(indocd._.context_graph.edges)
    pp4.columns=['column1', 'column2']
    pp4['column1'] = pp4['column1'].astype(str)
    pp4['column2'] = pp4['column2'].astype(str)
    selected_rowsIns4 = pp4.loc[pp4['column2'].str.contains("ID")]
    selected_rowsIns4['column1'] = selected_rowsIns4['column1'].astype(str)
    InsNumm4 = ''.join(list(selected_rowsIns4['column1']))
  elif len(token_positionGroup) <= 7 and len(token_positionGroup) > 5:
    pp1 = pd.DataFrame(indoca._.context_graph.edges)
    pp1.columns=['column1', 'column2']
    pp1['column1'] = pp1['column1'].astype(str)
    pp1['column2'] = pp1['column2'].astype(str)
    selected_rowsIns1 = pp1.loc[pp1['column2'].str.contains("ID")]
    selected_rowsIns1['column1'] = selected_rowsIns1['column1'].astype(str)
    InsNumm1 = ''.join(list(selected_rowsIns1['column1']))
    pp2 = pd.DataFrame(indocb._.context_graph.edges)
    pp2.columns=['column1', 'column2']
    pp2['column1'] = pp2['column1'].astype(str)
    pp2['column2'] = pp2['column2'].astype(str)
    selected_rowsIns2 = pp2.loc[pp2['column2'].str.contains("ID")]
    selected_rowsIns2['column1'] = selected_rowsIns2['column1'].astype(str)
    InsNumm2 = ''.join(list(selected_rowsIns2['column1']))
    pp3 = pd.DataFrame(indocc._.context_graph.edges)
    pp3.columns=['column1', 'column2']
    pp3['column1'] = pp3['column1'].astype(str)
    pp3['column2'] = pp3['column2'].astype(str)
    selected_rowsIns3 = pp3.loc[pp3['column2'].str.contains("ID")]
    selected_rowsIns3['column1'] = selected_rowsIns3['column1'].astype(str)
    InsNumm3 = ''.join(list(selected_rowsIns3['column1']))
  elif len(token_positionGroup) <= 5 and len(token_positionGroup) > 3:
    pp1 = pd.DataFrame(indoca._.context_graph.edges)
    pp1.columns=['column1', 'column2']
    pp1['column1'] = pp1['column1'].astype(str)
    pp1['column2'] = pp1['column2'].astype(str)
    selected_rowsIns1 = pp1.loc[pp1['column2'].str.contains("ID")]
    selected_rowsIns1['column1'] = selected_rowsIns1['column1'].astype(str)
    InsNumm1 = ''.join(list(selected_rowsIns1['column1']))
    pp2 = pd.DataFrame(indocb._.context_graph.edges)
    pp2.columns=['column1', 'column2']
    pp2['column1'] = pp2['column1'].astype(str)
    pp2['column2'] = pp2['column2'].astype(str)
    selected_rowsIns2 = pp2.loc[pp2['column2'].str.contains("ID")]
    selected_rowsIns2['column1'] = selected_rowsIns2['column1'].astype(str)
    InsNumm2 = ''.join(list(selected_rowsIns2['column1']))
  elif len(token_positionGroup) <= 3:
    pp1 = pd.DataFrame(indoca._.context_graph.edges)
    pp1.columns=['column1', 'column2']
    pp1['column1'] = pp1['column1'].astype(str)
    pp1['column2'] = pp1['column2'].astype(str)
    selected_rowsIns1 = pp1.loc[pp1['column2'].str.contains("ID")]
    selected_rowsIns1['column1'] = selected_rowsIns1['column1'].astype(str)
    InsNumm1 = ''.join(list(selected_rowsIns1['column1']))
  if len(token_positionGroup) > 7:
    InsuranceNo = [InsNumm1, InsNumm2, InsNumm3, InsNumm4]
    InsuranceNo = '/'.join(InsuranceNo)  
  elif len(token_positionGroup) <= 7 and len(token_positionGroup) > 5:
    InsuranceNo = [InsNumm1, InsNumm2, InsNumm3]
    InsuranceNo = '/'.join(InsuranceNo)  
  elif len(token_positionGroup) <= 5 and len(token_positionGroup) > 3:
    InsuranceNo = [InsNumm1, InsNumm2]
    InsuranceNo = '/'.join(InsuranceNo) 
  else:
    InsuranceNo = InsNumm1
else:
  InsuranceNo = ""
  
#Address Grabber
if len(re.findall(r'{}'.format("Address"), Info)) > 0 and len(re.findall(r'{}'.format("Phone"), Info)) > 0:
  token_positionAddress = [token.i for token in indoc if token.text == "Address"][0] + 1
  token_positionPhone = [token.i for token in indoc if token.text == "Phone"][0] - 1
  Address = indoc[(token_positionAddress):(token_positionPhone)]
else:
  Address = ""

#Address = Address.translate(str.maketrans('', '', string.punctuation))
indata = {'Patient Name': [PtName],
        'Patient DOB': [PtDOB],
        'Preferred Number': [preferrednumber],
        'Insurance Policy': [InsurancePolicy],
        'Insurance Number': [InsuranceNo], 
         'Address': [Address]}

dfins = pd.DataFrame(indata)
dfins

IndexError: ignored

In [ ]:
#@markdown Signout Tool

Ages = zp.loc[zp['column2'].str.contains("AGE")]
Ages['column1'] = Ages['column1'].astype(int)
Ages = list(Ages['column1'])
Agemod = [num for num in Ages if num > 1 and num < 120]

if len(Agemod) > 1:
  Agevec = get_number_farthest_from_stdev(Agemod)
else:
  if len(Agemod) == 1:
    Agevec = int(Agemod[0])
  else:
    Agevec = ""


Age = Agevec 


for ent in doc2.ents:
  if ent.label_ == "male":
    Sex = "Male"
  elif ent.label_ == "female":
    Sex = "Female"


#Sex = "M" #@param {type:"string"}

print("The patient is a", end=" ")
if Sex in ['Female', 'Woman', 'W', 'female', 'woman', 'w']:
    print(Age, "YOW", end=" with a PMHx of ")
elif Sex in ['Male', 'Man', 'M', 'male', 'man', 'm']:
    print(Age, "YOM", end=" with a PMHx of ")
else:
    print(Age, "YO", Sex, end=" with a PMHx of ")
if (y_predCHF >.4):
    print("CHF", end=", ")
if (y_predCOPD >.4 ):
    print("COPD", end=", ") 
if (y_predafib >.4):
    print("Atrial fibrillation", end=", ")
if (y_predDM >.4):
    print("DM", end=", ")
if (y_predESRD >.4 and data['Cr'][0] >= 2):
    print("ESRD", end=", ")
if (y_predCKD >.4 and data['Cr'][0] >= 1.4):
    print("CKD", end=", ")
if (y_predAsthma >.4):
    print("Asthma", end=", ") 
if (y_predHTN >.4):
    print("HTN", end=", ") 
if (y_predCAD >.4):
    print("CAD", end=", ")
if (y_predOSA >.4):
    print("OSA", end=", ")
if (y_predhypothyroid >.4):
    print("Hypothyroid",  end=", ")
if (y_predhyperthyroid >.4):
    print("Hyperthyroid",  end=", ")
if (y_predDLD >.4):
    print("HLD",  end=", ")   
if (y_predAI >.4):
    print("Adrenal Insufficiency", end=", ")   
if (y_predLupus >.4):
    print("Lupus",  end=", ")  
if (y_predCushing >.4):
    print("Cushings Disease",  end=", ")  
if (y_predScleroderma >.4):   
    print("Scleroderma",  end=", ")  
if (y_predPAD >.4):
    print("PAD",  end=", ")
if (y_predBrain_Tumor >.4):
    print("Brain Tumor",  end=", ")
if (y_predGERD >.4):
    print("GERD ",  end=", ")
print("who now presents with", end=" ")
if (y_predmeningitis_encephalitis >.4):
    print("meningitis/encephalitis",  end=", ")
if (y_predACS >.4):
    print("ACS",  end=", ")
if (y_predAAA >.4):
    print("AAA",  end=", ")
if (y_predTachycardia >.4):
     print("Tachycardia",  end=", ")
if (y_predBradycardia >.4):
    print("Bradycardia",  end=", ")
if (y_predHTN_Emergency >.4):
    print("HTN Emergency",  end=", ")
if (y_predHypotensionShock >.4):
    print("Hypotension/Shock",  end=", ")
if (y_predCardiac_arrest >.4):
    print("Cardiac arrest",  end=", ")
if (y_predAirway_Obstruction >.4):
    print("Airway_Obstruction",  end=", ")
if (y_predPneumonia >.4):
    print("Pneumonia",  end=", ")
if (y_predPneumothorax >.4):
    print("Pneumothorax",  end=", ")
if (y_predRespiratory_Failure >.4):
    print("Respiratory Failure",  end=", ")
if (y_predALF >.4):
    print("ALF Cirrhosis",  end=", ")
if (y_predGI_Bleed >.4):
    print("GI Bleed ",  end=", ")
if (y_predPancreatitis >.4):
    print("Pancreatitis",  end=", ")
if (y_predColitis >.4):
    print("Colitis",  end=", ")
if (y_predCellulitis >.4):
    print("Cellulitis",  end=", ")
if (y_predGastroenteritis >.4):
    print("Gastroenteritis",  end=", ")
if (y_predBacteremia >.4):
    print("Bacteremia",  end=", ")
if (y_predEndocarditis >.4):
    print("Endocarditis",  end=", ")
if (y_predUTI >.4):
    print("UTI",  end=", ")
if (y_predDelirium >.4):
    print("Delirium",  end=", ")
if (y_predDKA >.4):
    print("DKA",  end=", ")
if (y_predPE >.4):
    print("PE",  end=", ")
if (y_predICH >.4):
    print("ICH",  end=", ")
if (y_predSDH >.4):
    print("SDH",  end=", ")
if (y_predSAH >.4):
    print("SAH",  end=", ")
if (y_predCVA >.4):
    print("CVA",  end=", ")
if (y_predSeizure >.4):
    print("Seizures",  end=", ")
if (y_predAKI >.4 and data['Cr'][0] >= 1.4):
    print("AKI",  end=", ")
if (y_predEtOH_WD >.4):
    print("EtOH Withdrawal",  end=", ") 
print("", end=".  \n")
if (y_predvented >.4 and y_predHypotensionShock <.4):
    print('\033[1m' + 'The patient is intubated' + '\033[0m', end=".")
if (y_predvented >.4 and y_predHypotensionShock >.4):
    print('\033[1m' + 'The patient is intubated and on pressors' + '\033[0m', end=".  ")
if (y_predvented <.4 and y_predHypotensionShock >.4):
    print('\033[1m' + 'The patient is on pressors' + '\033[0m', end=".  ")
if Sex in ['Male', 'Man', 'M', 'male', 'man', 'm']:
    print("He is", end=" ")
elif Sex in ['Female', 'Woman', 'W', 'female', 'woman', 'w']:
    print("She is", end=" ")
else: 
    print("They are", end=" ")
print("also noted to have", end=" ")
if data['Na'][0] >= 150: 
    print("Hypernatremia", end=" ")
if data['Na'][0] <= 130: 
    print("Hyponatremia", end=" ")
if data['K'][0] >= 5.8: 
    print("Hyperkalemia", end=" ")
if data['K'][0] <= 3.4: 
    print("Hypokalemia", end=" ")
if data['Hgb'][0] <= 10: 
    print("Anemia", end=" ")
if data['AG'][0] >= 16: 
    print("Anion Gap Acidosis", end=" ")
if (y_predCoagulopathy >.4):
    print("Coagulopathy", end=" ")
if (supportv['Plt'][0] < 100):
    print("Thrombocytopenia", end=" ")
print("", end=".  \n")
print("Notable labs include", end=": ")
if data['Na'][0] >= 150: 
    print("Na=", data['Na'][0], end=", ")
if data['Na'][0] <= 130: 
    print("Na=", data['Na'][0], end=", ")
if data['K'][0] >= 5.8: 
    print("K=", data['K'][0], end=" ")
if data['K'][0] <= 3.4: 
    print("K=", data['K'][0], end=" ")
if data['Hgb'][0] <= 8: 
    print("Hgb=", data['Hgb'][0], end=" ")
if (supportv['Plt'][0] < 100):
    print("Plt=", data['Plt'][0], end=" ")
if data['AG'][0] >= 16: 
    print("AG=", data['AG'][0], end=" ")

The patient is a 46 YOM with a PMHx of who now presents with GI Bleed , .  
He is also noted to have Anemia .  
Notable labs include: 

In [ ]:
#@title Site Recommendations

Hospital = 'Cortland' #@param {type:"string"}


if Hospital == "Corning":
  if (y_predICH == 1 or y_predSDH == 1 or y_predSAH == 1 or y_predBrain_Tumor == 1):
    print("Use caution, Corning does not have Neurosurgery available.\n")
  if (y_predAKI == 1 or y_predESRD == 1 or y_predCKD == 1 or data['K'][0] > 6.1):
    print("Use caution, there is no renal consult or dialysis available at Corning.\n")
  if (y_predAAA == 1 or y_predEndocarditis == 1 or y_predPericardial_Effusion == 1):
    print("Use caution, Corning does not have cardiothoracic surgery available.\n")
  if (y_predTBI == 1 or y_predBurn == 1):
    print("Use caution, Corning does not have trauma surgery available.\n")
  if (y_predBradycardia == 1):
    print("If patient requires pacemaker, they will likely need to transfer.\n")
  if (y_predTachycardia == 1 or y_predHTN_Emergency == 1 or y_predHypotensionShock == 1):
    print("If the patient requires vasoactive drips, they will need ICU at Corning.\n")
  if (y_predDelirium == 1):
    print("If the patient requires 4 point restraints, they will need ICU transfer at Corning.\n")
  if (y_predAirway_Obstruction == 1 or y_predCOPD == 1 or y_predPneumonia == 1 or y_predPneumothorax == 1 or y_predRespiratory_Failure == 1 or y_predvented == 1 or y_predAsthma == 1 or y_predARDS == 1):  
    print("At Corning, if patient is intubated or on BiPAP/CPAP or high-flow for respiratory distress, they will need ICU.\n")
  if (y_predEtOH_WD == 1):
    print("If patient requires CIWA and IV benzo therapy for EtOH Withdrawal, they will need Corning ICU transfer.\n")
  if (y_predPE == 1):
    print("At Corning, submassive PE and thrombolytics will require ICU transfer.\n")
  if (y_predAbdominal_Pain == 1 or y_predGI_Bleed == 1 or y_predPancreatitis == 1 or y_predColitis == 1 or y_predGastroenteritis == 1):
    print("Use caution, at Corning GI consult coverage may be limited.\n")
  if (y_predDKA == 1):
    print("At Corning, DKA patients will need ICU transfer.\n")

if Hospital == "Cortland":
  if (y_predICH == 1 or y_predSDH == 1 or y_predSAH == 1 or y_predBrain_Tumor == 1):
    print("Use caution, Cortland does not have Neurosurgery available.\n")
  if (y_predAKI == 1 or y_predESRD == 1 or y_predCKD == 1 or data['K'][0] > 6.1):
    print("Use caution, there is no renal consult or dialysis available at Cortland.\n")
  if (y_predAAA == 1 or y_predEndocarditis == 1 or y_predPericardial_Effusion == 1):
    print("Use caution, Cortland does not have cardiothoracic surgery available.\n")
  if (y_predTBI == 1 or y_predBurn == 1):
    print("Use caution, Cortland does not have trauma surgery available.\n")
  if (y_predBradycardia == 1):
    print("If patient requires pacemaker, they will likely need to transfer.\n")
  if (y_predTachycardia == 1 or y_predHTN_Emergency == 1 or y_predHypotensionShock == 1):
    print("If the patient requires vasoactive drips, they will need ICU at Cortland.\n")
  if (y_predDelirium == 1):
    print("If the patient requires 4 point restraints, they will need ICU admission at Cortland.\n")
  if (y_predAirway_Obstruction == 1 or y_predCOPD == 1 or y_predPneumonia == 1 or y_predPneumothorax == 1 or y_predRespiratory_Failure == 1 or y_predvented == 1 or y_predAsthma == 1 or y_predARDS == 1):  
    print("At Cortland, if patient is intubated or on BiPAP/CPAP or high-flow for respiratory distress, they will need ICU.\n")
  if (y_predEtOH_WD == 1):
    print("If patient requires CIWA and IV benzo therapy for EtOH Withdrawal, they will need Cortland ICU transfer.\n")
  if (y_predPE == 1):
    print("At Cortland, submassive PE and thrombolytics will require ICU admission. Caution, there is limited interventional radiology availability.\n")
  if (y_predDKA == 1):
    print("At Cortland, DKA patients will need ICU transfer.\n")

Use caution, Cortland does not have Neurosurgery available.

Use caution, there is no renal consult or dialysis available at Cortland.



In [ ]:
visualize_ent(docop)